
# Benchmark Analysis 

This notebook contains code for producing charts (and soon, tables) for GC benchmarks.  It can currently process data
from the ASP.NET benchmarks obtained using crank as well as ETL data.  One of the design points of this notebook is
that the different operations have a similar "feel"; they have many optional parameters that build on default settings.
The parameters are intended to be identical (or at least similar) across operations.

The data is organized in a hierarchy.  (See `TopLevelData`.)
- A "run" consists of multiple "configurations".  (See `RunData`.)
- A "configuration" consists of multiple "benchmarks".  (See `ConfigData`.)
- A "benchmark" consists of multiple "iterations".  (See `BenchmarkData`.)
- An "iteration" consists of multiple GCs.  (See `IterationData`.)

In addition to multiple instances of the next lower level, each level contains data appropriate for that level.
For example, an iteration of an ASP.NET benchmark will have an RPS (requests per second) score.  The overall
benchmark could have the average RPS score across the iterations (though this can also be computed at presentation-time -
more on that later).

Data is stored in a `DataManager` object.  This class has a number of `Create...` and `Add...` methods.  They process
data identically; a `Create` method is simply shorthand for `new` and `Add` and is the common usage.

`CreateAspNetData` expects the directory structure that is produced by the GC infrastructure for ASP.NET.  For example:
``` xml
<run>\<configA>_0\<benchmarkX>.<configA>_0.log
                 \<benchmarkX>.gc.etl
                 \<benchmarkY>.<configA>_0.log
                 \<benchmarkY>.gc.etl
     \<configA>_1\...
     \<configA>_2\...
     \<configA>_3\...
     \<configB>_0\...
     \<configB>_1\...
     \<configB>_2\...
     \<configB>_3\...
```
Because of the way these names are generated, do not put `.` in any name or `_` in configuration names.  The `_0`, `_1`,
etc., are the iterations.

Many operations including `CreateAspNetData` use the `Filter` class.  It is a consistent way to specify names to
include or exclude and can be done by listing names or by regular expression.  `CreateAspNetData` can filter by
config or benchmark.  (To filter by run, simply don't pass that directory to the method.)  By default, it has a list
of process names that it will look for in the ETL data, but the optional parameter `pertinentProcesses` can override
that.

`CreateGCTrace(s)` only loads ETL files.  Since there is no context for a default value, `pertinentProcesses` must be
specified.  GC traces can be loaded in two ways.  The first expects one relevant process per trace and defaults to
setting the run as blank, the config as the enclosing directory name, and the iteration as zero.  The
benchmark name is extracted from the ETL filename but can be overridden or filtered.  The second allows multiple
processes per trace.  It uses the process as the benchmark name and promotes the other values up one level (e.g.,
the ETL filename becomes the config).  This behavior is controlled by the `loadMultipleProcesses` parameter.

The data is stored in nested dictionaries that can be directly modified or accessed through a number of `Get...`
helpers.  However, typically charting (and soon tabling) methods will be called next.  There are charting methods
for each of the three levels (the "run" level is not included since aggregating across configurations is not
expected), and at each level there are two overloads that only differ based on whether they expect one metric or
a list of metrics.
- `ChartBenchmarks` will chart benchmarks across the x-axis using aggregation of data from the iterations.  Each
  run/configuration will be a data series.
- `ChartIterations` will chart benchmarks across the x-axis using data from each iteration.  Each
  run/configuration/iteration will be a data series.
- `ChartGCData` will chart GCs across the x-axis using data from each iteration.  Each run/configuration/iteration
  will be a data series, and by default each benchmark will be on a different chart.

Each charting method requires one or more metrics to include in the chart.  These are represented by the `Metric`
class, which encapsulates a way to extract the metric from the data source, a label for that data, and the unit
for that data.  Many examples of metrics are provided in the `Metrics` class.  Data from one level can be
aggregated to the next level via the `Metrics.Promote` methods and the `Aggregation` class.  For example, the
average GC pause time for the execution of a single iteration can be extracted using
`Metrics.Promote(Metrics.G.PauseDuration, Aggregation.Max)`, though this particular example is already available as
`Metrics.I.MaxPauseDuration`.  Sample GC metrics are in `Metrics.G`.  Sample iteration metrics are in `Metrics.I`.
Sample benchmark metrics are in `Metrics.B`.

For typical cases, x-axis values are handled automatically (the GC index or the benchmark name as appropriate), but
the start time of the GC can be used instead by passing `Metrics.X.StartRelativeMSec` as the optional `xMetric`
argument.  (See the class `BaseMetric` for more details on how this works.)

Each charting method accepts `Filter`s for the runs, configs, and benchmarks and a predicate `dataFilter` for the
data itself (`BenchmarkData`, `IterationData`, or `TraceGC`).

In addition, some more advanced arguments are available:
- `xArrangement` - controls how the x-axis is arranged
  - `XArrangements.Default` - normal sorting by x values
  - `XArrangements.Sorted` - each series is sorted (highest-to-lowest), and the x-axis values are changed to ranks
  - `XArrangements.CombinedSorted` - the first series is sorted (highest-to-lowest), then other series are updated
    to match the resulting ordering of x values found from that sort
  - `XArrangements.Percentile` - similar to sorted except lower-to-highest, and the x-axis values are the
    percentiles of the data within that series - `Sorted` is useful for a small number of items where the x values
    have specific meanings (such as benchmark names), whereas `Percentile` is useful when considering the x values
    as a distribution.
  - Alternatively, create a new subclass of the `XArrangement` class
- `configNameSimplifier` - XPlot has trouble if the series' names (and thus the chart legend) get too large.  The
  configuration names can be long and repetitive, so this option can be used to display shorter values.
  - `NameSimplifier.PrefixDashed` - a predefined strategy that considers configurations as a series of names
    separated by dashes.  Common prefixes are removed.  For example, `a`, `a-b-d`, `a-b-e`, and `a-c` will be
    simplified to `<>`, `b-d`, `b-e`, and `c`.  The blank value and delimiter can be adjusted by creating a new
    `PrefixSimplifier`.
  - `ListSimplifier` - applies key-value pairs to the names
  - Alternatively, create a new subclass of the `NameSimplifier` class
- `includeRunName` - By default, the run name is discarded when charting under the assumption that the typical
  case is multiple configurations under the same run.  Setting this parameter concatenates the run and configuration
  together.
- `display` - By default, generated chart(s) will be displayed.  Clearing this parameters prevents that behavior.
  Charts are always returned to the caller for possible further processing.
- `debug` - Enables a bit of debug spew.

Upcoming:
- Add the ability to specify a primary data series and add metrics that compare against it.
- Fill out the predefined metrics.
- Add requested features (specify width of chart).
- Add more aggregations, including adding the aggregation of iterations to an iteration-level chart/table.
  (e.g., b1_1, b1_2, b1_3, b1_max, b1_avg, b2_1, b2_2, b2_3, b2_max, b2_avg)
- Consider splitting `SeriesInfo` into level-specific versions and make methods such as `ChartInternal` generic
  on the series information.

## Building and Using The GC Analysis API

In [ ]:
dotnet build -c Release "..\GC.Analysis.API"

In [ ]:
#i "nuget: https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-public/nuget/v3/index.json"

#r "nuget: Microsoft.Diagnostics.Tracing.TraceEvent"
#r "nuget: YamlDotnet"
#r "nuget: XPlot.Plotly"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: Microsoft.Data.Analysis, 0.19.1"
#r "nuget: Newtonsoft.Json"

// TODO: Ensure you are pointing to the right artifacts folder.
#r "..\..\..\..\..\artifacts\bin\GC.Analysis.API\Release\net7.0\GC.Analysis.API.dll"

using Etlx = Microsoft.Diagnostics.Tracing.Etlx;
using GC.Analysis.API;
using Microsoft.Data.Analysis;
using Microsoft.Diagnostics.Tracing.Analysis.GC;
using Microsoft.Diagnostics.Tracing.Analysis;
using Microsoft.Diagnostics.Tracing.Parsers.Clr;
using Microsoft.Diagnostics.Tracing;
using System.Diagnostics;
using XPlot.Plotly;

using System.IO;
using System.Text.RegularExpressions;
using Newtonsoft.Json;

// Very basic utilities

// ML and MA are convenience syntax for making lists and arrays.
public static List<T> ML<T>(params T[] elems) => new List<T>(elems);
public static T[] MA<T>(params T[] elems) => elems;

public static V GetOrAdd<K,V>(this Dictionary<K,V> dict, K key, V value)
    => dict.TryAdd(key, value) ? value : dict[key];

public static void SetWithExtend<T>(this List<T> list, int index, T value)
{
    int count = list.Count;
    int needed = index + 1;
    for (int i = 0; i < (needed - count); ++i)
    {
        list.Add(default(T));
    }
    list[index] = value;
}

public static IEnumerable<(T, int)> WithIndex<T>(this IEnumerable<T> list) => list.Select((value, index) => (value, index));
public static bool NotNull<T>(T x) => x != null;


## Data Acquisition

The next few cells detail how to retrieve the data from a base path. The run name below is the name of the folder generated from running the ``aspnetbenchmarks`` command from the GC.Infrastructure API. 

In [ ]:
// The LoadInfo class consists of all the pertinent fields needed to represent both the result from a particular benchmark
// as well as the the comparison between two runs where the Data2 represents the GCProcessData of the comparand.
public sealed class LoadInfo
{
    public double MaxWorkingSetMB {get;set;} = double.NaN;
    public double P99WorkingSetMB {get;set;} = double.NaN;
    public double P95WorkingSetMB {get;set;} = double.NaN;
    public double P90WorkingSetMB {get;set;} = double.NaN;
    public double P75WorkingSetMB {get;set;} = double.NaN;
    public double P50WorkingSetMB {get;set;} = double.NaN;

    public double MaxPrivateMemoryMB {get;set;} = double.NaN;
    public double P99PrivateMemoryMB {get;set;} = double.NaN;
    public double P95PrivateMemoryMB {get;set;} = double.NaN;
    public double P90PrivateMemoryMB {get;set;} = double.NaN;
    public double P75PrivateMemoryMB {get;set;} = double.NaN;
    public double P50PrivateMemoryMB {get;set;} = double.NaN;

    public double RequestsPerMSec {get; set;} = double.NaN;
    public double MeanLatencyMS {get; set;} = double.NaN;
    public double Latency99thMS {get; set;} = double.NaN;
    public double Latency90thMS {get; set;} = double.NaN;
    public double Latency75thMS {get; set;} = double.NaN;
    public double Latency50thMS {get; set;} = double.NaN;

    // Do these need to be stored on the LoadInfo?  Context should already have this information.
    public string Run {get; set;}
    public string Config {get; set;}
    public string Benchmark {get; set;}
    public int Iteration {get; set;} = -1;
}

public class GCSummaryInfo
{
    public double TotalSuspensionTimeMSec {get;set;} = double.NaN;
    public double PercentPauseTimeInGC {get; set;} = double.NaN;
    public double PercentTimeInGC {get; set;} = double.NaN;
    public double MeanHeapSizeBeforeMB {get; set;} = double.NaN;
    public double MaxHeapSizeMB {get; set;} = double.NaN;
    public double TotalAllocationsMB {get;set;} = double.NaN;
    public double GCScore {get;set;} = double.NaN;

    public double MaxHeapCount {get;set;} = double.NaN;
    public double NumberOfHeapCountSwitches {get;set;} = double.NaN;
    public double NumberOfHeapCountDirectionChanges {get;set;} = double.NaN;

    // Consider removing
    public GCProcessData Data {get;set;}
    public GCProcessData? Data2 {get;set;}

    public int ProcessId {get;set;}
    public string CommandLine {get;set;}
    public string TracePath {get; set;}
    public string ProcessName {get;set;}
}

public class BenchmarkSummaryData
{
    public double MaxWorkingSetMB {get;set;} = double.NaN;
    public double P99WorkingSetMB {get;set;} = double.NaN;
    public double P95WorkingSetMB {get;set;} = double.NaN;
    public double P90WorkingSetMB {get;set;} = double.NaN;
    public double P75WorkingSetMB {get;set;} = double.NaN;
    public double P50WorkingSetMB {get;set;} = double.NaN;

    public double MaxPrivateMemoryMB {get;set;} = double.NaN;
    public double P99PrivateMemoryMB {get;set;} = double.NaN;
    public double P95PrivateMemoryMB {get;set;} = double.NaN;
    public double P90PrivateMemoryMB {get;set;} = double.NaN;
    public double P75PrivateMemoryMB {get;set;} = double.NaN;
    public double P50PrivateMemoryMB {get;set;} = double.NaN;

    public double RequestsPerMSec {get;set;} = double.NaN;
    public double MeanLatencyMS {get; set;} = double.NaN;
    public double Latency50thMS {get; set;} = double.NaN;
    public double Latency75thMS {get; set;} = double.NaN;
    public double Latency90thMS {get; set;} = double.NaN;
    public double Latency99thMS {get; set;} = double.NaN;

    public string Benchmark {get; set;}
}

// XXXData is the Data for an XXX, not a mapping from XXX to data.
// For example, BenchmarkData is a mapping from iterations to data because a benchmark can have multiple iterations.
public record IterationData(LoadInfo LoadInfo, GCSummaryInfo GCSummaryInfo, GCProcessData GCProcessData)
{
    public LoadInfo LoadInfo { get; set; } = LoadInfo;
    public GCSummaryInfo GCSummaryInfo { get; set; } = GCSummaryInfo;
    public GCProcessData GCProcessData  { get; set; } = GCProcessData;
    // GCLogInfo GCLogInfo;
    // Dictionary<string, double> Other;
}
public record BenchmarkData(LoadInfo SummaryLoadInfo, List<IterationData> Iterations); // Iteration # -> data
public record ConfigData(Dictionary<string, BenchmarkData> Benchmarks); // Benchmark name -> data
public record RunData(Dictionary<string, ConfigData> Configs); // Config name -> data
public record TopLevelData(Dictionary<string, RunData> Runs); // Run name -> data

public class Filter // abstraction used whenever names should be filtered
{
    private string[] _includeNames;
    private string[] _excludeNames;
    private Regex _includeRE;
    private Regex _excludeRE;

    public Filter(params string[] includeNames) : this(includeNames: includeNames, excludeNames: null) {}
    public Filter(IEnumerable<string> includeNames = null, IEnumerable<string> excludeNames = null,
                  string includeRE = null, string excludeRE = null)
        : this(
            includeNames: includeNames?.ToArray(),
            excludeNames: excludeNames?.ToArray(),
            includeRE: (includeRE != null) ? (new Regex(includeRE)) : null,
            excludeRE: (excludeRE != null) ? (new Regex(excludeRE)) : null
        )
        {}

    private Filter(string[] includeNames = null, string[] excludeNames = null,
                   Regex includeRE = null, Regex excludeRE = null)
    {
        _includeNames = includeNames;
        _excludeNames = excludeNames;
        _includeRE = includeRE;
        _excludeRE = excludeRE;
    }

    public static Filter Names(params string[] includeNames) => new(includeNames: includeNames);
    public static Filter ExcludeNames(params string[] includeNames) => new(excludeNames: includeNames);
    public static Filter RE(string includeRE) => new(includeRE: includeRE);
    public static Filter ExcludeRE(string includeRE) => new(excludeRE: includeRE);
    public static Filter All { get; } = new(null);

    public bool Include(string candidate)
        => (((_includeNames != null) || (_includeRE != null))
                ? ((_includeNames?.Contains(candidate) ?? false) || ((_includeRE?.Match(candidate).Success ?? false)))
                : true)
            && (!_excludeNames?.Contains(candidate) ?? true)
            && (!_excludeRE?.Match(candidate).Success ?? true);
}

public class IntFilter
{
    private (int min, int max)[] _includeRanges;
    private (int min, int max)[] _excludeRanges;

    private static IEnumerable<T> EmptyIfNull<T>(IEnumerable<T> enumerable)
        => enumerable ?? Enumerable.Empty<T>();

    public IntFilter(params int[] includeValues) : this(includeValues: includeValues, excludeRanges: null) {}
    public IntFilter(params (int min, int max)[] includeRanges) : this(includeRanges: includeRanges, excludeRanges: null) {}
    public IntFilter(IEnumerable<int> includeValues = null, IEnumerable<int> excludeValues = null,
        IEnumerable<(int min, int max)> includeRanges = null, IEnumerable<(int min, int max)> excludeRanges = null)
        : this(
            includeRanges:
                (includeValues != null || includeRanges != null)
                ? (EmptyIfNull(includeValues).Select(v => (v,v))).Concat(EmptyIfNull(includeRanges)).ToArray()
                : null,
            excludeRanges:
                (excludeValues != null || excludeRanges != null)
                ? (EmptyIfNull(excludeValues).Select(v => (v,v))).Concat(EmptyIfNull(excludeRanges)).ToArray()
                : null
        )
        {}

    private IntFilter((int min, int max)[] includeRanges = null, (int min, int max)[] excludeRanges = null)
    {
        _includeRanges = includeRanges;
        _excludeRanges = excludeRanges;
    }

    public static IntFilter Values(params int[] includeValues) => new(includeValues: includeValues);
    public static IntFilter Ranges(params (int min, int max)[] includeRanges) => new(includeRanges: includeRanges);
    public static IntFilter ExcludeValues(params int[] excludeValues) => new(excludeValues: excludeValues);
    public static IntFilter ExcludeRanges(params (int min, int max)[] excludeRanges) => new(excludeRanges: excludeRanges);
    public static IntFilter All { get; } = new(includeValues: null);

    public bool Include(int candidate)
        => (_includeRanges?.Any(pair => pair.min <= candidate && candidate <= pair.max) ?? true)
            && (!_excludeRanges?.Any(pair => pair.min <= candidate && candidate <= pair.max) ?? true);
}

In [ ]:
// Filter tests
int failed = 0;
void Assert(bool b, string message)
{
    if (!b)
    {
        failed++;
        Console.WriteLine($"Failed: {message}");
    }
}

{
    foreach (Filter fa in ML(new("a"), new (includeNames: ML("a")), Filter.Names("a"), new(includeRE: "a"), Filter.RE("a")))
    {
        Assert(fa.Include("a"), "a~a");
        Assert(!fa.Include("b"), "a~!b");
    }

    foreach (Filter fab in ML(new("a", "b"), new(includeNames: ML("a", "b")), Filter.Names("a", "b"), new(includeRE: "a|b"), Filter.RE("a|b"),
        new(includeNames: ML("a"), includeRE: "b")))
    {
        Assert(fab.Include("a"), "ab~a");
        Assert(fab.Include("b"), "ab~b");
        Assert(!fab.Include("c"), "ab~!c");
    }

    foreach (Filter fna in ML(new(excludeNames: ML("a")), Filter.ExcludeNames("a"), new(excludeRE: "a"), Filter.ExcludeRE("a")))
    {
        Assert(!fna.Include("a"), "!a~!a");
        Assert(fna.Include("b"), "!a~b");
    }

    foreach (Filter fnab in ML(new(excludeNames: ML("a", "b")), Filter.ExcludeNames("a", "b"), new(excludeRE: "a|b"), Filter.ExcludeRE("a|b"),
        new(excludeNames: ML("a"), excludeRE: "b")))
    {
        Assert(!fnab.Include("a"), "!ab~!a");
        Assert(!fnab.Include("b"), "!ab~!b");
        Assert(fnab.Include("c"), "!ab~c");
    }

    foreach (Filter fanb in ML<Filter>(new(includeNames: ML("a", "b"), excludeNames: ML("b")), new(includeRE: "a|b", excludeRE: "b")))
    {
        Assert(fanb.Include("a"), "a!b~a");
        Assert(!fanb.Include("b"), "a!b~!b");
    }

    Assert(Filter.All.Include("a"), "all~a");

    foreach (IntFilter f1 in ML(new(1), new((1,1)), new (includeValues: ML(1)), new (includeRanges: ML((1,1))),
        IntFilter.Values(1), IntFilter.Ranges((1,1))))
    {
        Assert(f1.Include(1), "1~1");
        Assert(!f1.Include(2), "1~!2");
    }

    foreach (IntFilter f12 in ML(new((1,2)), new((1,1), (2,2)),
        new (includeValues: ML(1,2)), new (includeRanges: ML((1,1), (2,2))), new (includeRanges: ML((1,2))),
        IntFilter.Values(1, 2), IntFilter.Ranges((1,2)), IntFilter.Ranges((1,1), (2,2))))
    {
        Assert(f12.Include(1), "1~1");
        Assert(f12.Include(2), "1~!2");
    }

    foreach (IntFilter fn1 in ML(new (excludeValues: ML(1)), new (excludeRanges: ML((1,1))),
        IntFilter.ExcludeValues(1), IntFilter.ExcludeRanges((1,1))))
    {
        Assert(!fn1.Include(1), "!1~!1");
        Assert(fn1.Include(2), "!1~2");
    }

    foreach (IntFilter fn12 in ML(new(excludeValues: ML(1,2)), IntFilter.ExcludeValues(1,2),
        new(excludeRanges: ML((1,1),(2,2))), new(excludeRanges: ML((1,2))), IntFilter.ExcludeRanges((1,2)), IntFilter.ExcludeRanges((1,2))))
    {
        Assert(!fn12.Include(1), "!12~!1");
        Assert(!fn12.Include(2), "!12~!2");
        Assert(fn12.Include(3), "!12~3");
    }

    foreach (IntFilter f1n2 in ML<IntFilter>(new(includeValues: ML(1,2), excludeValues: ML(2)), new(includeRanges: ML((1,2)), excludeRanges: ML((2,2)))))
    {
        Assert(f1n2.Include(1), "1!2~1");
        Assert(!f1n2.Include(2), "1!2~!2");
    }

    Assert(IntFilter.All.Include(1), "all~1");
}
if (failed > 0) throw new Exception($"Failed {failed} test(s)");

In [ ]:
using ConfigIterationFilter = System.Collections.Generic.IReadOnlyDictionary<string, IntFilter>;

public static bool MightInclude(this ConfigIterationFilter configIterationFilter, string config)
    => (configIterationFilter == null) || configIterationFilter.ContainsKey(config);

public static bool Include(this ConfigIterationFilter configIterationFilter, string config, int iteration)
    => (configIterationFilter == null) || (configIterationFilter.GetValueOrDefault(config)?.Include(iteration) ?? true);

public class DataManager
{
    public readonly TopLevelData _data;

    public DataManager() => _data = new(new());

    public static DataManager CreateAspNetData(string basePath,
        Filter configFilter = null, Filter benchmarkFilter = null, IntFilter iterationFilter = null, ConfigIterationFilter configIterationFilter = null,
        List<string> pertinentProcesses = null)
        => CreateAspNetData(MA(basePath),
            configFilter: configFilter, benchmarkFilter: benchmarkFilter, iterationFilter: iterationFilter, configIterationFilter: configIterationFilter,
            pertinentProcesses: pertinentProcesses);

    public static DataManager CreateAspNetData(IEnumerable<string> basePaths,
        Filter configFilter = null, Filter benchmarkFilter = null, IntFilter iterationFilter = null, ConfigIterationFilter configIterationFilter = null,
        List<string> pertinentProcesses = null)
    {
        DataManager dataManager = new();
        dataManager.AddAspNetData(basePaths: basePaths,
            configFilter: configFilter, benchmarkFilter: benchmarkFilter, iterationFilter: iterationFilter, configIterationFilter: configIterationFilter,
            pertinentProcesses: pertinentProcesses);
        return dataManager;
    }

    public static DataManager CreateGCTrace(string file, List<string> pertinentProcesses, string run = null, string config = null, int? iteration = null,
        bool loadMultipleProcesses = true)
    {
        DataManager dataManager = new();
        dataManager.AddGCTrace(file: file, pertinentProcesses: pertinentProcesses, run: run, config: config, iteration: iteration,
            loadMultipleProcesses: loadMultipleProcesses);
        return dataManager;
    }

    public static DataManager CreateGCTraces(string basePath, List<string> pertinentProcesses, SearchOption searchOption = SearchOption.TopDirectoryOnly,
        Filter benchmarkFilter = null, string run = null, string config = null, int? iteration = null, bool loadMultipleProcesses = true)
    {
        DataManager dataManager = new();
        dataManager.AddGCTraces(basePath: basePath, pertinentProcesses: pertinentProcesses, searchOption: searchOption,
            benchmarkFilter: benchmarkFilter, run: run, config: config, iteration: iteration, loadMultipleProcesses: loadMultipleProcesses);
        return dataManager;

    }

    public void AddAspNetData(string basePath,
        Filter configFilter = null, Filter benchmarkFilter = null, IntFilter iterationFilter = null, ConfigIterationFilter configIterationFilter = null,
        List<string> pertinentProcesses = null)
        => AddAspNetData(basePaths: MA(basePath),
            configFilter: configFilter, benchmarkFilter: benchmarkFilter, iterationFilter: iterationFilter, configIterationFilter: configIterationFilter,
            pertinentProcesses: pertinentProcesses);

    public void AddAspNetData(IEnumerable<string> basePaths,
        Filter configFilter = null, Filter benchmarkFilter = null, IntFilter iterationFilter = null, ConfigIterationFilter configIterationFilter = null,
        List<string> pertinentProcesses = null)
    {
        configFilter = configFilter ?? Filter.All;
        benchmarkFilter = benchmarkFilter ?? Filter.All;
        iterationFilter = iterationFilter ?? IntFilter.All;
        // configIterationFilter is not set to an empty dictionary as that would exclude everything

        foreach (var basePath in basePaths)
        {
            LoadAspNetDataFromBasePath(basePath: basePath,
                configFilter: configFilter, benchmarkFilter: benchmarkFilter, iterationFilter: iterationFilter, configIterationFilter: configIterationFilter,
                pertinentProcesses: pertinentProcesses);
        }
    }

    public void AddGCTrace(string file, List<string> pertinentProcesses, string run = null, string config = null, int? iteration = null, bool loadMultipleProcesses = true)
    {
        LoadGCTrace(file: file, configFilter: Filter.All, benchmarkFilter: Filter.All, run: run, config: config, iteration: iteration, pertinentProcesses: pertinentProcesses,
            expectAspNetData: false, loadMultipleProcesses: loadMultipleProcesses);
    }

    public void AddGCTraces(string basePath, List<string> pertinentProcesses, SearchOption searchOption = SearchOption.TopDirectoryOnly, Filter configFilter = null, Filter benchmarkFilter = null,
        string run = null, string config = null, int? iteration = null, bool loadMultipleProcesses = true)
    {
        configFilter = configFilter ?? Filter.All;
        benchmarkFilter = benchmarkFilter ?? Filter.All;

        LoadGCTracesFromPath(path: basePath, searchOption: searchOption, configFilter: configFilter, benchmarkFilter: benchmarkFilter,
            run: run, config: config, iteration: iteration, pertinentProcesses: pertinentProcesses,
            expectAspNetData: false, loadMultipleProcesses: loadMultipleProcesses);
    }

    public static double DeltaPercent (double baseline, double comparand) => Math.Round((comparand - baseline) / baseline * 100, 2);

    public TopLevelData Data => _data; 

    //public static LoadInfo LoadLogFile(string file)
    //{
    //    
    //}

    // Consider generalizing the error reporting here
    private (string, int) ParseConfigIterName(string dir)
    {
        int lastUnderscore = dir.LastIndexOf("_");
        string config;
        int iteration;
        if ((lastUnderscore != -1)
            && int.TryParse(dir.AsSpan(lastUnderscore + 1), out iteration))
        {
            config = dir.Substring(0, lastUnderscore);
        }
        else
        {
            Console.WriteLine($"{dir} is not in the form <config>_<iteration>");
            config = dir;
            iteration = 0;
        }

        return (config, iteration);
    }

    private (string, string, int) ParseBenchmarkLogFileName(string logName)
    {
        string[] split = Path.GetFileName(logName).Split(".");
        if ((split.Length != 3) || (split[2] != "log"))
        {
            Console.WriteLine($"{logName} is not in the form <benchmark>.<config>_<iteration>.log");
        }
        // TODO: Store these suffixes
        string benchmark = Path.GetFileName( split[0] ).Replace("_Windows", "").Replace("_Linux", "").Replace(".gc", "").Replace(".nettrace", "");
        (string config, int iteration) = ParseConfigIterName(split[1]);
        return (config, benchmark, iteration);
    }

    private List<string> AspNetProcesses = new()
    {
        "PlatformBenchmarks",
        "Benchmarks",
        "MapAction",
        "TodosApi",
        "BasicGrpc",
        "BasicMinimalApi",
    };

    private void LoadAspNetDataFromBasePath(string basePath,
        Filter configFilter, Filter benchmarkFilter, IntFilter iterationFilter, ConfigIterationFilter configIterationFilter,
        List<string> pertinentProcesses)
    {
        pertinentProcesses = pertinentProcesses ?? AspNetProcesses;

        string run = Path.GetFileName(basePath);

        foreach (string fullDir in Directory.GetDirectories(basePath))
        {
            string subDir = Path.GetFileName(fullDir);
            (string config, int iteration) = ParseConfigIterName(subDir);
            if (configFilter.Include(config) && iterationFilter.Include(iteration) && configIterationFilter.Include(config, iteration))
            {
                LoadAspNetDataFromPath(fullDir, benchmarkFilter, run, config, iteration);
                // configFilter has alreay been done- LoadGCTracesFromPath needs it for the case where loadMultipleProcesses is true
                // and the filenames become the configs
                LoadGCTracesFromPath(fullDir, SearchOption.TopDirectoryOnly, configFilter: Filter.All, benchmarkFilter: benchmarkFilter,
                    run: run, config: config, iteration: iteration,
                    pertinentProcesses: pertinentProcesses, expectAspNetData: true, loadMultipleProcesses: false);
            }
        }
    }

    // Returns a LoadInfo with information extracted from the log file.
    // Does not populate the Benchmark, etc., fields.
    private LoadInfo LoadAspNetLogFile(string file)
    {
        LoadInfo info = new();

        int idxOfApplication = Int32.MaxValue;
        int idxOfLoad = Int32.MaxValue;
        int idx = 0;

        foreach (var line in File.ReadLines(file))
        {
            string[] sp = line.Split("|", StringSplitOptions.TrimEntries);
            if (line.Contains("| application"))
            {
                idxOfApplication = idx;
            }
            else if (line.Contains("| load"))
            {
                idxOfLoad = idx;
            }
            else if (line.Contains("| Latency 50th"))
            {
                info.Latency50thMS = double.Parse(sp[2]);
            }
            else if (line.Contains("| Latency 75th"))
            {
                info.Latency75thMS = double.Parse(sp[2]);
            }
            else if (line.Contains("| Latency 90th"))
            {
                info.Latency90thMS = double.Parse(sp[2]);
            }
            else if (line.Contains("| Latency 99th"))
            {
                info.Latency99thMS = double.Parse(sp[2]);
            }
            else if (line.Contains("Requests/sec"))
            {
                info.RequestsPerMSec = double.Parse(sp[2]) / 1000;
            }
            else if (line.Contains("Mean latency"))
            {
                info.MeanLatencyMS = double.Parse(sp[2]);
            }
            else if (line.Contains("Max Working Set") && (idxOfApplication < idx && idx < idxOfLoad)) 
            {
                info.MaxWorkingSetMB = double.Parse(sp[2]);
            }
            else if (line.Contains("Working Set P99") && (idxOfApplication < idx && idx < idxOfLoad)) 
            {
                info.P99WorkingSetMB = double.Parse(sp[2]);
            }
            else if (line.Contains("Working Set P95") && (idxOfApplication < idx && idx < idxOfLoad)) 
            {
                info.P95WorkingSetMB = double.Parse(sp[2]);
            }
            else if (line.Contains("Working Set P90") && (idxOfApplication < idx && idx < idxOfLoad)) 
            {
                info.P90WorkingSetMB = double.Parse(sp[2]);
            }                
            else if (line.Contains("Working Set P75") && (idxOfApplication < idx && idx < idxOfLoad)) 
            {
                info.P75WorkingSetMB = double.Parse(sp[2]);
            }
            else if (line.Contains("Working Set P50") && (idxOfApplication < idx && idx < idxOfLoad)) 
            {
                info.P50WorkingSetMB = double.Parse(sp[2]);
            }                
            else if (line.Contains("Max Private Memory") && (idxOfApplication < idx && idx < idxOfLoad)) 
            {
                info.MaxPrivateMemoryMB  = double.Parse(sp[2]);
            }
            else if (line.Contains("Private Memory P99") && (idxOfApplication < idx && idx < idxOfLoad)) 
            {
                info.P99PrivateMemoryMB = double.Parse(sp[2]);
            }
            else if (line.Contains("Private Memory P95") && (idxOfApplication < idx && idx < idxOfLoad)) 
            {
                info.P95PrivateMemoryMB = double.Parse(sp[2]);
            }
            else if (line.Contains("Private Memory P90") && (idxOfApplication < idx && idx < idxOfLoad)) 
            {
                info.P90PrivateMemoryMB = double.Parse(sp[2]);
            }                
            else if (line.Contains("Private Memory P75") && (idxOfApplication < idx && idx < idxOfLoad)) 
            {
                info.P75PrivateMemoryMB = double.Parse(sp[2]);
            }
            else if (line.Contains("Private Memory P50") && (idxOfApplication < idx && idx < idxOfLoad)) 
            {
                info.P50PrivateMemoryMB = double.Parse(sp[2]);
            }

            ++idx;
        }

        return info;
    }

    private void LoadAspNetDataFromPath(string path, Filter benchmarkFilter, string run, string config, int iteration)
    {
        var files = Directory.GetFiles(path, "*.log", SearchOption.AllDirectories);

        foreach (var file in files)
        {
            if (file.Contains("build.log") || file.Contains("output.log") || file.Contains("_GCLog"))
            {
                continue;
            }

            (string logConfig, string benchmark, int logIteration) = ParseBenchmarkLogFileName(file);

            if (!benchmarkFilter.Include(benchmark))
            {
                continue;
            }

            if ((config != logConfig) || (iteration != logIteration))
            {
                Console.WriteLine($"Directory name and log filename in {file} disagree on config/iteration");
            }

            LoadInfo info = LoadAspNetLogFile(file);

            info.Run = run;
            info.Config = config;
            info.Benchmark = benchmark;
            info.Iteration = iteration;

            RunData runData = _data.Runs.GetOrAdd(run, new(new()));
            ConfigData configData = runData.Configs.GetOrAdd(config, new(new()));
            BenchmarkData benchmarkData = configData.Benchmarks.GetOrAdd(benchmark, new(null, new()));
            if ((benchmarkData.Iterations.Count > iteration)
                && (benchmarkData.Iterations[iteration] != null))
            {
                Console.WriteLine($"WARNING: Duplicate iteration '{run} / {config} / {benchmark} / {iteration}' found");
                benchmarkData.Iterations[iteration].LoadInfo = info;
            }
            else
            {
                benchmarkData.Iterations.SetWithExtend(iteration, new(info, null, null));
            }
        }
    }

    private void LoadGCTracesFromPath(string path, SearchOption searchOption, Filter configFilter, Filter benchmarkFilter, string run, string config, int? iteration, List<string> pertinentProcesses,
        bool expectAspNetData, bool loadMultipleProcesses)
    {
        var traceFiles = Directory.GetFiles(path, "*.etl.zip", searchOption).ToList();
        var nettraceFiles = Directory.GetFiles(path, "*.nettrace", searchOption);
        traceFiles.AddRange(nettraceFiles);

        Parallel.ForEach(traceFiles,
            file => LoadGCTrace(file: file, configFilter: configFilter, benchmarkFilter: benchmarkFilter, run: run, config: config, iteration: iteration,
                pertinentProcesses: pertinentProcesses, expectAspNetData: expectAspNetData, loadMultipleProcesses: loadMultipleProcesses));
    }

    private void LoadGCTrace(string file, Filter configFilter, Filter benchmarkFilter, string run, string config, string benchmark, int? iteration, List<string> pertinentProcesses, bool expectAspNetData, bool loadMultipleProcesses)
    {
        string dir = Path.GetFileName(Path.GetDirectoryName(file));
        //string[] sp = file.Split("\\");
        //sp[sp.Length - 1]
        string fileBaseName = Path.GetFileNameWithoutExtension(file)
            .Replace("_Windows", "")
            .Replace(".gc.etl", "")
            .Replace("_Linux", "")
            .Replace(".nettrace", "")
            .Replace(".gc", "")
            .Replace(".etl", "");

        if (loadMultipleProcesses && (benchmark != null))
        {
            throw new ArgumentException("LoadGCTrace: loadMultipleProcesses and setting a benchmark name are not compatible");
        }

        run = run ?? (loadMultipleProcesses ? dir : "");
        config = config ?? (loadMultipleProcesses ? fileBaseName : dir);
        if (!configFilter.Include(config)) return;

        Analyzer analyzer = AnalyzerManager.GetAnalyzer(file);
        List<GCProcessData> allData;

        //foreach (var pair in analyzer.AllGCProcessData)
        //{
        //    Console.WriteLine($"{pair.Key}: {pair.Value.Count}");
        //}
        //if (file.Contains(".nettrace"))
        //{
        //    data = analyzer.AllGCProcessData.First().Value.First();
        //}
        //else
        {
            allData = pertinentProcesses.SelectMany(p => analyzer.GetProcessGCData(p)).ToList(); //.Where(NotNull).FirstOrDefault();
        }

        if (allData.Count == 0)
        {
            Console.WriteLine($"The following trace doesn't have a pertinent process: {file}");
            Console.WriteLine($"Processes: {string.Join(", ", analyzer.AllGCProcessData.Keys)}");
            Console.WriteLine($"Check: {string.Join(", ", analyzer.AllGCProcessData.Keys.Select(k => k == pertinentProcesses[0]))}");
            return;
        }
        if (!loadMultipleProcesses && (allData.Count > 1))
        {
            Console.WriteLine($"The following trace has more than one pertinent process: {file}");
            Console.WriteLine($"Found processes: {string.Join(", ", allData.Select(d => d.ProcessName))}'");
            return;
        }

        foreach (GCProcessData data in allData)
        {
            benchmark = benchmark ?? (loadMultipleProcesses ? data.ProcessName : fileBaseName);
            if (!benchmarkFilter.Include(benchmark)) continue;
            LoadGCTraceOneProcess(file, data, run, config, benchmark, iteration, expectAspNetData);
        }
    }

    private void LoadGCTraceOneProcess(string file, GCProcessData data, string run, string config, string benchmark, int? iteration, bool expectAspNetData)
    {
        GCSummaryInfo gcSummaryInfo = new();
        gcSummaryInfo.MeanHeapSizeBeforeMB = data.Stats.MeanSizePeakMB;
        gcSummaryInfo.MaxHeapSizeMB = data.Stats.MaxSizePeakMB;
        gcSummaryInfo.PercentTimeInGC = (data.GCs.Sum(gc => gc.PauseDurationMSec - gc.SuspendDurationMSec) / (data.Stats.ProcessDuration) ) * 100;
        gcSummaryInfo.TracePath = data.Parent.TraceLogPath;
        gcSummaryInfo.TotalAllocationsMB = data.Stats.TotalAllocatedMB;
        gcSummaryInfo.CommandLine = data.CommandLine;
        gcSummaryInfo.PercentPauseTimeInGC = data.Stats.GetGCPauseTimePercentage();
        gcSummaryInfo.GCScore = (gcSummaryInfo.MaxHeapSizeMB * gcSummaryInfo.PercentPauseTimeInGC);
        gcSummaryInfo.ProcessId = data.ProcessID;
        gcSummaryInfo.Data = data;
        gcSummaryInfo.ProcessName = data.ProcessName;
        gcSummaryInfo.TotalSuspensionTimeMSec = data.GCs.Sum(gc => gc.SuspendDurationMSec);

        gcSummaryInfo.MaxHeapCount = 0;
        gcSummaryInfo.NumberOfHeapCountSwitches = 0;
        gcSummaryInfo.NumberOfHeapCountDirectionChanges = 0;

        int? prevNumHeapsOption = null;
        bool prevChangeUp = true; // don't want to count the initial 1->n change as a change in direction
        for (int i = 0; i < data.GCs.Count; i++)
        {
            if (data.GCs[i].GlobalHeapHistory == null) continue;
            int thisNumHeaps = data.GCs[i].GlobalHeapHistory.NumHeaps;
            gcSummaryInfo.MaxHeapCount = Math.Max(gcSummaryInfo.MaxHeapCount, thisNumHeaps);
            if (prevNumHeapsOption.HasValue)
            {
                int prevNumHeaps = prevNumHeapsOption.Value;
                if (prevNumHeaps != thisNumHeaps)
                {
                    gcSummaryInfo.NumberOfHeapCountSwitches++;
                    bool thisChangeUp = thisNumHeaps > prevNumHeaps;
                    if (prevChangeUp != thisChangeUp)
                    {
                        gcSummaryInfo.NumberOfHeapCountDirectionChanges++;
                    }
                    prevChangeUp = thisChangeUp;
                }
            }
            prevNumHeapsOption = thisNumHeaps;
        }

        lock (_data)
        {
            RunData runData = _data.Runs.GetOrAdd(run, new(new()));
            ConfigData configData = runData.Configs.GetOrAdd(config, new(new()));
            BenchmarkData benchmarkData = configData.Benchmarks.GetOrAdd(benchmark, new(null, new()));

            int iterationToUse = iteration ?? benchmarkData.Iterations.FindIndex(iterationData => iterationData == null);
            if (iterationToUse == -1) iterationToUse = benchmarkData.Iterations.Count;

            if ((benchmarkData.Iterations.Count > iterationToUse)
                && (benchmarkData.Iterations[iterationToUse] != null))
            {
                if (benchmarkData.Iterations[iterationToUse].GCSummaryInfo != null)
                {
                    Console.WriteLine($"Replacing existing GC information for '{run} / {config} / {benchmark} / {iterationToUse}' - {file}");
                }
                benchmarkData.Iterations[iterationToUse].GCSummaryInfo = gcSummaryInfo;
                benchmarkData.Iterations[iterationToUse].GCProcessData = data;
            }
            else
            {
                if (expectAspNetData)
                {
                    Console.WriteLine($"The following trace doesn't have a corresponding ASP.NET log '{run} / {config} / {benchmark} / {iterationToUse}' - {file}");
                }

                benchmarkData.Iterations.SetWithExtend(iterationToUse, new(null, gcSummaryInfo, data));
            }
        }
    }
}

In [ ]:
// Huge block of code that operates on DataManager
// -----------------------------------------------

// Notebook cells are already in implicit classes, so this isn't needed (and doesn't work):
// public static class DataManagerExtensions

public static IEnumerable<(string run, string config, ConfigData configData)> GetConfigsWithData(this DataManager dataManager, Filter runFilter, Filter configFilter)
{
    foreach ((string run, RunData runData) in dataManager.Data.Runs)
    {
        if (!runFilter.Include(run)) continue;
        foreach ((string config, ConfigData configData) in runData.Configs)
        {
            if (!configFilter.Include(config)) continue;
            yield return (run, config, configData);
        }
    }
}

public static IEnumerable<(string run, string config)> GetConfigs(this DataManager dataManager, Filter runFilter, Filter configFilter)
    => dataManager.GetConfigsWithData(runFilter, configFilter).Select(tuple => (tuple.run, tuple.config));

public static IEnumerable<(string run, string config, string benchmark, BenchmarkData benchmarkData)> GetBenchmarksWithData(
    this DataManager dataManager, Filter runFilter, Filter configFilter, Filter benchmarkFilter, IntFilter iterationFilter, ConfigIterationFilter configIterationFilter)
{
    foreach ((string run, string config, ConfigData configData) in dataManager.GetConfigsWithData(runFilter, configFilter))
    {
        if (!configIterationFilter.MightInclude(config)) continue;

        foreach ((string benchmark, BenchmarkData benchmarkData) in configData.Benchmarks)
        {
            if (!benchmarkFilter.Include(benchmark)) continue;
            if (!benchmarkData.Iterations.WithIndex()
                .Where(pair => pair.Item1 != null)
                .Select(pair => pair.Item2)
                .Any(iteration => iterationFilter.Include(iteration) && configIterationFilter.Include(config, iteration))) continue;
            yield return (run, config, benchmark, benchmarkData);
        }
    }
}

public static IEnumerable<(string run, string config, string benchmark)> GetBenchmarks(this DataManager dataManager, Filter runFilter, Filter configFilter,
    Filter benchmarkFilter, IntFilter iterationFilter, ConfigIterationFilter configIterationFilter)
    => dataManager.GetBenchmarksWithData(runFilter, configFilter, benchmarkFilter, iterationFilter, configIterationFilter)
        .Select(tuple => (tuple.run, tuple.config, tuple.benchmark));

public static IEnumerable<(string run, string config, int iteration, IterationData data)> GetIterationsForBenchmark(this DataManager dataManager,
    Filter runFilter, Filter configFilter, IntFilter iterationFilter, ConfigIterationFilter configIterationFilter, string benchmark)
{
    foreach ((string run, string config, ConfigData configData) in dataManager.GetConfigsWithData(runFilter, configFilter))
    {
        if (!configIterationFilter.MightInclude(config)) continue;
        if (!configData.Benchmarks.TryGetValue(benchmark, out BenchmarkData benchmarkData)) continue;

        foreach ((IterationData iterationData, int iteration) in benchmarkData.Iterations.WithIndex())
        {
            if (!iterationFilter.Include(iteration)) continue;
            if (!configIterationFilter.Include(config, iteration)) continue;
            if (iterationData == null) continue;
            yield return (run, config, iteration, iterationData);
        }
    }
}

public static IEnumerable<int> GetIterations(this ConfigData data, string config,
    Filter benchmarkFilter, IntFilter iterationFilter, ConfigIterationFilter configIterationFilter)
    // May need to improve efficiency here
    => data.Benchmarks
        .Where((b, _) => benchmarkFilter.Include(b.Key))
        .SelectMany(b =>
            b.Value.Iterations
                .WithIndex()
                .Where(pair => pair.Item1 != null)
                .Select(pair => pair.Item2)
                .Where(iteration => iterationFilter.Include(iteration) && configIterationFilter.Include(config, iteration)))
        .Distinct()
        .OrderBy(x => x);

// Utilities

// https://stackoverflow.com/a/49058506 
public static IEnumerable<(T PrevItem, T CurrentItem, T NextItem)>
        SlidingWindow<T>(this IEnumerable<T> source, T emptyValue = default)
{
    using (var iter = source.GetEnumerator())
    {
        if (!iter.MoveNext())
            yield break;
        var prevItem = emptyValue;
        var currentItem = iter.Current;
        while (iter.MoveNext())
        {
            var nextItem = iter.Current;
            yield return (prevItem, currentItem, nextItem);
            prevItem = currentItem;
            currentItem = nextItem;
        }
        yield return (prevItem, currentItem, emptyValue);
    }
}

// overkill for what is needed now but leftover

public struct CircularListAccess<T> : IReadOnlyList<T>
{
    private IList<T> _list;
    private int _start;
    private int _length;

    public CircularListAccess(IList<T> list, int start, int length)
    {
        if (list == null) throw new ArgumentException("list");
        if (start < 0 || start >= list.Count) throw new ArgumentException("start");
        if (length < 0 || length > list.Count) throw new ArgumentException("length");

        _list = list;
        _start = start;
        _length = length;
    }

    public T this[int index]
    {
        get
        {
            if (index >= _length) throw new IndexOutOfRangeException();
            return _list[(_start + index) % _list.Count];
        }
    }

    public int Count => _length;

    public struct Enumerator : IEnumerator<T>
    {
        private CircularListAccess<T> _list;
        private int _index;
        private T _current;

        public Enumerator(CircularListAccess<T> list)
        {
            _list = list;
            _index = 0;
            _current = default;
        }
        public T Current => _current;
        object IEnumerator.Current => Current;
        public bool MoveNext()
        {
            int count = _list.Count;
            if (_index < count)
            {
                _current = _list[_index++];
                return true;
            }
            else
            {
                _current = default;
                return false;
            }
        }
        public void Reset() { _index = 0; _current = default; }
        public void Dispose() {}
    }

    public IEnumerator<T> GetEnumerator() => new Enumerator(this);
    IEnumerator IEnumerable.GetEnumerator() => new Enumerator(this);
}

public static IEnumerable<IReadOnlyList<T>>
        SlidingRange<T>(this List<T> source, int size)
{
    for (int i = 0; i <= source.Count - size; ++i)
    {
        // don't actually need CircularListAccess - was from an earlier idea
        yield return new CircularListAccess<T>(source, i, size);
    }
}

public class ColorProvider
{
    // Families of gradients
    // 80 00 00 -> ff 00 00 -> ff 80 80 (3)
    // 80 80 00 -> ff ff 00 -> ff ff 80 (3)
    // 80 40 00 -> ff 80 00 -> ff c0 80 (6)
    // 40 40 40 -> 80 80 80 -> c0 c0 c0 (1)
    // 80 2A 00 -> ff 55 00 -> ff aa 80 (6)
    // 80 55 00 -> ff aa 00 -> ff d4 80 (6)
    enum Scale
    {
        Zero,
        Full,
        Half,
        OneThird,
        TwoThird,
    }
    
    static (int first, int mid, int last) GetScale(Scale scale)
        => scale switch
        {
            Scale.Zero => (0, 0, 0x80),
            Scale.Full => (0x80, 0xFF, 0xFF),
            Scale.Half => (0x40, 0x80, 0xC0),
            Scale.OneThird => (0x2A, 0x55, 0xAA),
            Scale.TwoThird => (0x55, 0xAA, 0xD4),
            _ => throw new Exception("Unknown Scale")
        };

    public record RGB(int R, int G, int B);
    record ScaleRGB(Scale R, Scale G, Scale B);

    static ScaleRGB[] _colorFamilies =
    {
        new ScaleRGB(Scale.Full, Scale.Zero, Scale.Zero),
        new ScaleRGB(Scale.Zero, Scale.Full, Scale.Zero),
        new ScaleRGB(Scale.Zero, Scale.Zero, Scale.Full),

        new ScaleRGB(Scale.Half, Scale.Half, Scale.Half),

        //new ScaleRGB(Scale.Full, Scale.Full, Scale.Zero), // yellow isn't scaling very well
        new ScaleRGB(Scale.Full, Scale.Zero, Scale.Full),
        new ScaleRGB(Scale.Zero, Scale.Full, Scale.Full),
        
        new ScaleRGB(Scale.Full, Scale.Half, Scale.Zero),
        new ScaleRGB(Scale.Zero, Scale.Full, Scale.Half),
        new ScaleRGB(Scale.Half, Scale.Zero, Scale.Full),

        new ScaleRGB(Scale.Full, Scale.Zero, Scale.Half),
        new ScaleRGB(Scale.Half, Scale.Full, Scale.Zero),
        new ScaleRGB(Scale.Zero, Scale.Half, Scale.Full),

        new ScaleRGB(Scale.Full, Scale.OneThird, Scale.Zero),
        new ScaleRGB(Scale.Zero, Scale.Full, Scale.OneThird),
        new ScaleRGB(Scale.OneThird, Scale.Zero, Scale.Full),

        new ScaleRGB(Scale.Full, Scale.Zero, Scale.OneThird),
        new ScaleRGB(Scale.OneThird, Scale.Full, Scale.Zero),
        new ScaleRGB(Scale.Zero, Scale.OneThird, Scale.Full),

        new ScaleRGB(Scale.Full, Scale.TwoThird, Scale.Zero),
        new ScaleRGB(Scale.Zero, Scale.Full, Scale.TwoThird),
        new ScaleRGB(Scale.TwoThird, Scale.Zero, Scale.Full),

        new ScaleRGB(Scale.Full, Scale.Zero, Scale.TwoThird),
        new ScaleRGB(Scale.TwoThird, Scale.Full, Scale.Zero),
        new ScaleRGB(Scale.Zero, Scale.TwoThird, Scale.Full),
    };

    int GetComponent(Scale scale, int index, int count)
    {
        int max = count - 1;
        float half = max / 2.0f;
        var scaleValue = GetScale(scale);
        if (max == 0) return scaleValue.first;
        (int baseValue, int topValue, float fraction) =
            (index > half)
            ? (scaleValue.mid, scaleValue.last, (index - half) / half)
            : (scaleValue.first, scaleValue.mid, (index / half));
        return (int)(baseValue + fraction * (topValue - baseValue));
    }

    public static Marker GetMarker(RGB rgb) => (rgb != null) ? (new Marker { color = $"rgb({rgb.R}, {rgb.G}, {rgb.B})" }) : null;

    RGB GetColor(int colorIndex, int groupIndex, int numInBuild)
    {
        if (colorIndex >= _colorFamilies.Length) return null;

        var RGB = _colorFamilies[colorIndex];
        var R = GetComponent(RGB.R, groupIndex, numInBuild);
        var G = GetComponent(RGB.G, groupIndex, numInBuild);
        var B = GetComponent(RGB.B, groupIndex, numInBuild);
        return new RGB(R, G, B);
    }

    record ColorGroup(int FamilyIndex, int GroupIndex, int GroupSize, Dictionary<string, RGB> GroupColorMap)
    {
        public int GroupIndex { get; set; } = GroupIndex;
    }

    Dictionary<string, ColorGroup>? _groups; // name of build -> (color index, next index in group)

    public ColorProvider(Dictionary<string, int> groups)
    {
        if (groups.Count <= 1) return;

        _groups = groups
            .Take(_colorFamilies.Length)
            .Select((kvp, index) => (kvp.Key, new ColorGroup(index, 0, kvp.Value, new())))
            .ToDictionary();
    }

    public RGB GetColor(string buildName, string id = null)
    {
        //Console.WriteLine($"- '{buildName}' '{id}'");
        if (_groups == null) return null;
        ColorGroup group = _groups[buildName];
        if (group.FamilyIndex >= _colorFamilies.Length) return null;

        if ((id != null) && group.GroupColorMap.TryGetValue(id, out RGB color))
        {
            return color;
        }
        //Console.WriteLine($"--- '{group}'");
        color = GetColor(group.FamilyIndex, group.GroupIndex++, group.GroupSize);
        //Console.WriteLine($"----- '{color}'");
        if (id != null) group.GroupColorMap[id] = color;
        return color;
    }

    public void SetMarker(Scatter scatter, string buildName, string id = null)
    {
        Marker marker = GetMarker(GetColor(buildName, id));
        if (marker != null) scatter.marker = marker;
    }

    public void DumpColorGroups()
    {
        if (_groups == null)
        {
            Console.WriteLine("No groups");
            return;
        }
        Console.WriteLine($"Number of groups: {_groups.Count}");
        foreach (var (name, group) in _groups)
        {
            Console.WriteLine($"  '{name}': {group.FamilyIndex}, {group.GroupIndex}/{group.GroupSize}");
        }
    }
}

public class Aggregation
{
    public Func<IEnumerable<double>, double> Func;
    public string Title;
    public string UnitOverride;

    public Aggregation(Func<IEnumerable<double>, double> func, string title, string unitOverride)
    {
        Func = func;
        Title = title;
        UnitOverride = unitOverride;
    }

    public static class Funcs
    {
        public static double Min(IEnumerable<double> data) => data.Min();
        public static double Max(IEnumerable<double> data) => data.Max();

        public static double Volatility(IEnumerable<double> data)
        {
            var max = data.Max();
            var min = data.Min();
            return Math.Round(((max - min) / min) * 100, 2);
        }

        public static double Average(IEnumerable<double> data) => data.Average();
        public static double Range(IEnumerable<double> data) => data.Max() - data.Min();

        public static double GeoMean(IEnumerable<double> data)
        {
            double mult = 1;
            int count = 0;
            foreach (double value in data)
            {
                mult *= value;
                count++;
            }
            return Math.Pow(mult, 1.0 / count);
        }
    }

    public static Aggregation Min { get; } = new Aggregation(Funcs.Min, "Min", null);
    public static Aggregation Max { get; } = new Aggregation(Funcs.Max, "Max", null);
    public static Aggregation Volatility { get; } = new Aggregation(Funcs.Volatility, "Volatility", "?");
    public static Aggregation Average { get; } = new Aggregation(Funcs.Average, "Average", null);
    public static Aggregation Range { get; } = new Aggregation(Funcs.Range, "Range", null);
    public static Aggregation GeoMean { get; } = new Aggregation(Funcs.GeoMean, "GeoMean", null);
}

public class BaseMetric<TSource, TValue>
{
    protected Func<TSource, TValue?> ExtractFunc;
    public string Title;

    public BaseMetric(Func<TSource, TValue?> extract, string title)
    {
        ExtractFunc = extract;
        Title = title;
    }

    public TValue? DoExtract(TSource gc)
    {
        TValue? value;
        try
        {
            value = ExtractFunc(gc);
        }
        catch (Exception e)
        {
            //Console.WriteLine($"Exception processing {Title}");
            //Console.WriteLine($"   {e}");
            value = default;
        }
        return value;
    }
}

public class Metric<TSource> : BaseMetric<TSource, double?>
{
    public string Unit;
    public double? Cap;
    private int _capExceededCount;
    private double _capExceededMin;
    private double _capExceededMax;
    public double? AxisCountOffset;

    public Metric(Func<TSource, double> extract, string title, string unit, double? cap = null, double? axisCountOffset = null)
        : base((s => extract(s)), title)
    {
        Unit = unit;
        Cap = cap;
        AxisCountOffset = axisCountOffset;
    }

    public Metric(Func<TSource, double?> extract, string title, string unit, double? cap = null, double? axisCountOffset = null)
        : base(extract, title)
    {
        Unit = unit;
        Cap = cap;
        AxisCountOffset = axisCountOffset;
    }

    public double? DoExtract(TSource gc, int count)
    {
        double? value = base.DoExtract(gc);
        if (value.HasValue)
        {
            if (value > Cap)
            {
                _capExceededCount++;
                _capExceededMin = Math.Min(_capExceededMin, value.Value);
                _capExceededMax = Math.Max(_capExceededMax, value.Value);
                value = Cap;
            }
            if (AxisCountOffset.HasValue) value += AxisCountOffset * count;
        }
        return value;
    }

    private Metric<TSource> Copy() => new(ExtractFunc, Title, Unit, Cap);
    public Metric<TSource> WithCap(double cap) => new(ExtractFunc, Title, Unit, cap, AxisCountOffset);
    public Metric<TSource> WithOffset(double offset) => new(ExtractFunc, Title, Unit, Cap, offset);

    public void ResetDiagnostics()
    {
        _capExceededCount = 0;
        _capExceededMin = double.MaxValue;
        _capExceededMax = double.MinValue;
    }

    public void DisplayDiagnostics(string context)
    {
        if (_capExceededCount > 0)
        {
            Console.WriteLine($"Cap ({Cap.Value}) exceeded {_capExceededCount} times (min={_capExceededMin:N2}, max={_capExceededMax:N2}) for {context}");
        }
    }

    public static Metric<TSource> Promote<TOldSource>(Metric<TOldSource> metric, Func<TSource, IEnumerable<TOldSource>> oldExtract, Aggregation aggregation)
        => new(extract: source => aggregation.Func(oldExtract(source).Select(metric.ExtractFunc).Where(NotNull).Select(value => value.Value)),
            title: $"{aggregation.Title} of {metric.Title}",
            unit: aggregation.UnitOverride ?? metric.Unit);
}

public static class Metrics
{
    public static Metric<IterationData> Promote(Metric<TraceGC> metric, Aggregation aggregation)
        => Metric<IterationData>.Promote(metric, iterationData => iterationData.GCProcessData.GCs, aggregation);
    public static Metric<BenchmarkData> Promote(Metric<IterationData> metric, Aggregation aggregation)
        => Metric<BenchmarkData>.Promote(metric, benchmarkData => benchmarkData.Iterations, aggregation);
    public static Metric<ConfigData> Promote(Metric<BenchmarkData> metric, Aggregation aggregation)
        => Metric<ConfigData>.Promote(metric, configData => configData.Benchmarks.Values, aggregation);
    public static Metric<RunData> Promote(Metric<ConfigData> metric, Aggregation aggregation)
        => Metric<RunData>.Promote(metric, runData => runData.Configs.Values, aggregation);
    public static Metric<TopLevelData> Promote(Metric<RunData> metric, Aggregation aggregation)
        => Metric<TopLevelData>.Promote(metric, data => data.Runs.Values, aggregation);

    public static class X
    {
        public static BaseMetric<(string, TraceGC), XValue> GCIndex { get; } = new(pair => new XValue(pair.Item2.Number), "GC Index");
        public static BaseMetric<(string, TraceGC), XValue> StartRelativeMSec { get; } = new(pair => new XValue(pair.Item2.StartRelativeMSec), "GC Start");
        public static BaseMetric<(string, BenchmarkData), XValue> BenchmarkName { get; } = new(pair => new XValue(pair.Item1), "Benchmark Name");
        public static BaseMetric<(string, IterationData), XValue> IterationBenchmarkName { get; } = new(pair => new XValue(pair.Item1), "Benchmark Name");
    }

    public static class G
    {
        public static Metric<TraceGC> AllocedSinceLastGCMB = new(gc => gc.AllocedSinceLastGCMB, title: "Allocated", unit: "MB");
        // AllocRateMBSec is MB/s but this puts it on same y-axis as plain MB
        public static Metric<TraceGC> AllocRateMBSec = new(gc => gc.AllocRateMBSec, title: "Allocation rate", unit: "MB");
        public static Metric<TraceGC> CommittedAfterTotalBookkeeping = new(gc => gc.CommittedUsageAfter.TotalBookkeepingCommitted, title: "Committed Book (after)", unit: "MB");
        public static Metric<TraceGC> CommittedAfterInFree = new(gc => gc.CommittedUsageAfter.TotalCommittedInFree, title: "Committed In Free (after)", unit: "MB");
        public static Metric<TraceGC> CommittedAfterInGlobalDecommit = new(gc => gc.CommittedUsageAfter.TotalCommittedInGlobalDecommit, title: "Committed In Global Decommit (after)", unit: "MB");
        public static Metric<TraceGC> CommittedAfterInGlobalFree = new(gc => gc.CommittedUsageAfter.TotalCommittedInGlobalFree, title: "Committed In Global Free (after)", unit: "MB");
        public static Metric<TraceGC> CommittedAfterInUse = new(gc => gc.CommittedUsageAfter.TotalCommittedInUse, title: "Committed In Use (after)", unit: "MB");
        public static List<Metric<TraceGC>> CommittedAfterMetrics = ML(CommittedAfterTotalBookkeeping, CommittedAfterInFree, CommittedAfterInGlobalDecommit, CommittedAfterInGlobalFree, CommittedAfterInUse);
        public static Metric<TraceGC> CommittedBeforeTotalBookkeeping = new(gc => gc.CommittedUsageBefore.TotalBookkeepingCommitted, title: "Committed Book (before)", unit: "MB");
        public static Metric<TraceGC> CommittedBeforeInFree = new(gc => gc.CommittedUsageBefore.TotalCommittedInFree, title: "Committed In Free (before)", unit: "MB");
        public static Metric<TraceGC> CommittedBeforeInGlobalDecommit = new(gc => gc.CommittedUsageBefore.TotalCommittedInGlobalDecommit, title: "Committed In Global Decommit (before)", unit: "MB");
        public static Metric<TraceGC> CommittedBeforeInGlobalFree = new(gc => gc.CommittedUsageBefore.TotalCommittedInGlobalFree, title: "Committed In Global Free (before)", unit: "MB");
        public static Metric<TraceGC> CommittedBeforeInUse = new(gc => gc.CommittedUsageBefore.TotalCommittedInUse, title: "Committed In Use (before)", unit: "MB");
        public static List<Metric<TraceGC>> CommittedBeforeMetrics = ML(CommittedBeforeTotalBookkeeping, CommittedBeforeInFree, CommittedBeforeInGlobalDecommit, CommittedBeforeInGlobalFree, CommittedBeforeInUse);
        public static Metric<TraceGC> DurationMSec = new(gc => gc.DurationMSec, "Duration", "ms");
        public static Metric<TraceGC> GCCpuMSec = new(gc => gc.GCCpuMSec, "GC CPU", "ms");
        public static Metric<TraceGC> Gen0Budget = new(gc => gc.GenBudgetMB(Gens.Gen0), "Gen0 budget", "MB");
        public static Metric<TraceGC> Gen1Budget = new(gc => gc.GenBudgetMB(Gens.Gen1), "Gen1 budget", "MB");
        public static Metric<TraceGC> Gen2Budget = new(gc => gc.GenBudgetMB(Gens.Gen2), "Gen2 budget", "MB");
        public static Metric<TraceGC> GenLargeBudget = new(gc => gc.GenBudgetMB(Gens.GenLargeObj), "GenLarge budget", "MB");
        public static Metric<TraceGC> GenPinBudget = new(gc => gc.GenBudgetMB(Gens.GenPinObj), "GenPin budget", "MB");
        public static Metric<TraceGC> Generation = new(gc => gc.Generation, "Generation", "gen");
        public static Metric<TraceGC> Gen0Fragmentation = new(gc => gc.GenFragmentationMB(Gens.Gen0), "Gen0 fragmentation", "MB");
        public static Metric<TraceGC> Gen1Fragmentation = new(gc => gc.GenFragmentationMB(Gens.Gen1), "Gen1 fragmentation", "MB");
        public static Metric<TraceGC> Gen2Fragmentation = new(gc => gc.GenFragmentationMB(Gens.Gen2), "Gen2 fragmentation", "MB");
        public static Metric<TraceGC> GenLargeFragmentation = new(gc => gc.GenFragmentationMB(Gens.GenLargeObj), "GenLarge fragmentation", "MB");
        public static Metric<TraceGC> GenPinFragmentation = new(gc => gc.GenFragmentationMB(Gens.GenPinObj), "GenPin fragmentation", "MB");
        public static Metric<TraceGC> Gen0FragmentationPercent = new(gc => gc.GenFragmentationPercent(Gens.Gen0), "Gen0 fragmentation %", "%");
        public static Metric<TraceGC> Gen1FragmentationPercent = new(gc => gc.GenFragmentationPercent(Gens.Gen1), "Gen1 fragmentation %", "%");
        public static Metric<TraceGC> Gen2FragmentationPercent = new(gc => gc.GenFragmentationPercent(Gens.Gen2), "Gen2 fragmentation %", "%");
        public static Metric<TraceGC> GenLargeFragmentationPercent = new(gc => gc.GenFragmentationPercent(Gens.GenLargeObj), "GenLarge fragmentation %", "%");
        public static Metric<TraceGC> GenPinFragmentationPercent = new(gc => gc.GenFragmentationPercent(Gens.GenPinObj), "GenPin fragmentation %", "%");
        public static Metric<TraceGC> Gen0In = new(gc => gc.GenInMB(Gens.Gen0), "Gen0 Memory (in)", "MB");
        public static Metric<TraceGC> Gen1In = new(gc => gc.GenInMB(Gens.Gen1), "Gen1 Memory (in)", "MB");
        public static Metric<TraceGC> Gen2In = new(gc => gc.GenInMB(Gens.Gen2), "Gen2 Memory (in)", "MB");
        public static Metric<TraceGC> GenLargeIn = new(gc => gc.GenInMB(Gens.GenLargeObj), "GenLarge Memory (in)", "MB");
        public static Metric<TraceGC> GenPinIn = new(gc => gc.GenInMB(Gens.GenPinObj), "GenPin Memory (in)", "MB");
        public static Metric<TraceGC> Gen0ObjSizeAfter = new(gc => gc.GenObjSizeAfterMB(Gens.Gen0), "Gen0 object size (after)", "MB");
        public static Metric<TraceGC> Gen1ObjSizeAfter = new(gc => gc.GenObjSizeAfterMB(Gens.Gen1), "Gen1 object size (after)", "MB");
        public static Metric<TraceGC> Gen2ObjSizeAfter = new(gc => gc.GenObjSizeAfterMB(Gens.Gen2), "Gen2 object size (after)", "MB");
        public static Metric<TraceGC> GenLargeObjSizeAfter = new(gc => gc.GenObjSizeAfterMB(Gens.GenLargeObj), "GenLarge object size (after)", "MB");
        public static Metric<TraceGC> GenPinObjSizeAfter = new(gc => gc.GenObjSizeAfterMB(Gens.GenPinObj), "GenPin object size (after)", "MB");
        public static Metric<TraceGC> Gen0Out = new(gc => gc.GenOutMB(Gens.Gen0), "Gen0 Memory (out)", "MB");
        public static Metric<TraceGC> Gen1Out = new(gc => gc.GenOutMB(Gens.Gen1), "Gen1 Memory (out)", "MB");
        public static Metric<TraceGC> Gen2Out = new(gc => gc.GenOutMB(Gens.Gen2), "Gen2 Memory (out)", "MB");
        public static Metric<TraceGC> GenLargeOut = new(gc => gc.GenOutMB(Gens.GenLargeObj), "GenLarge Memory (out)", "MB");
        public static Metric<TraceGC> GenPinOut = new(gc => gc.GenOutMB(Gens.GenPinObj), "GenPin Memory (out)", "MB");
        public static Metric<TraceGC> Gen0Promoted = new(gc => gc.GenPromotedMB(Gens.Gen0), "Gen0 Promoted", "MB");
        public static Metric<TraceGC> Gen1Promoted = new(gc => gc.GenPromotedMB(Gens.Gen1), "Gen1 Promoted", "MB");
        public static Metric<TraceGC> Gen2Promoted = new(gc => gc.GenPromotedMB(Gens.Gen2), "Gen2 Promoted", "MB");
        public static Metric<TraceGC> GenLargePromoted = new(gc => gc.GenPromotedMB(Gens.GenLargeObj), "GenLarge Promoted", "MB");
        public static Metric<TraceGC> GenPinPromoted = new(gc => gc.GenPromotedMB(Gens.GenPinObj), "GenPin Promoted", "MB");
        public static Metric<TraceGC> Gen0SizeAfter = new(gc => gc.GenSizeAfterMB(Gens.Gen0), "Gen0 size (after)", "MB");
        public static Metric<TraceGC> Gen1SizeAfter = new(gc => gc.GenSizeAfterMB(Gens.Gen1), "Gen1 size (after)", "MB");
        public static Metric<TraceGC> Gen2SizeAfter = new(gc => gc.GenSizeAfterMB(Gens.Gen2), "Gen2 size (after)", "MB");
        public static Metric<TraceGC> GenLargeSizeAfter = new(gc => gc.GenSizeAfterMB(Gens.GenLargeObj), "GenLarge size (after)", "MB");
        public static Metric<TraceGC> GenPinSizeAfter = new(gc => gc.GenSizeAfterMB(Gens.GenPinObj), "GenPin size (after)", "MB");
        public static Metric<TraceGC> Gen0SizeBefore = new(gc => gc.GenSizeBeforeMB[(int) Gens.Gen0], "Gen0 size (before)", "MB");
        public static Metric<TraceGC> Gen1SizeBefore = new(gc => gc.GenSizeBeforeMB[(int) Gens.Gen1], "Gen1 size (before)", "MB");
        public static Metric<TraceGC> Gen2SizeBefore = new(gc => gc.GenSizeBeforeMB[(int) Gens.Gen2], "Gen2 size (before)", "MB");
        public static Metric<TraceGC> GenLargeSizeBefore = new(gc => gc.GenSizeBeforeMB[(int) Gens.GenLargeObj], "GenLarge size (before)", "MB");
        public static Metric<TraceGC> GenPinSizeBefore = new(gc => gc.GenSizeBeforeMB[(int) Gens.GenPinObj], "GenPin size (before)", "MB");
        //public static Metric<TraceGC> Condemned = new(gc => gc.GetCondemnedReasons());

        // TODO: GlobalHeapHistory.*
        //public static Metric<TraceGC> Ghh = new(gc => gc.GlobalHeapHistory., "", "");
        public static Metric<TraceGC> IsConcurrent = new (gc => Convert.ToDouble((gc.GlobalHeapHistory.GlobalMechanisms & GCGlobalMechanisms.Concurrent) != 0), "Is concurrent", "Y/N");
        public static Metric<TraceGC> IsCompaction = new (gc => Convert.ToDouble((gc.GlobalHeapHistory.GlobalMechanisms & GCGlobalMechanisms.Compaction) != 0), "Is compaction", "Y/N");
        public static Metric<TraceGC> IsPromotion = new (gc => Convert.ToDouble((gc.GlobalHeapHistory.GlobalMechanisms & GCGlobalMechanisms.Promotion) != 0), "Is promotion", "Y/N");
        public static Metric<TraceGC> IsDemotion = new (gc => Convert.ToDouble((gc.GlobalHeapHistory.GlobalMechanisms & GCGlobalMechanisms.Demotion) != 0), "Is demotion", "Y/N");
        public static Metric<TraceGC> IsCardBundles = new (gc => Convert.ToDouble((gc.GlobalHeapHistory.GlobalMechanisms & GCGlobalMechanisms.CardBundles) != 0), "Is cardbundles", "Y/N");
        public static Metric<TraceGC> NumHeaps = new((gc => gc.GlobalHeapHistory.NumHeaps), "GC Heaps", "#");
        public static Metric<TraceGC> NumHeapsWithOffset = NumHeaps.WithOffset(0.05);

        public static Metric<TraceGC> HeapCount = new(gc => gc.HeapCount, "Heap count", "#");

        // HeapCountSample
        public static Metric<TraceGC> HcsElapsedTimeBetweenGCs = new(gc => gc.HeapCountSample.ElapsedTimeBetweenGCsMSec, "HCSampleElapsed", "ms");
        public static Metric<TraceGC> HcsGCIndex = new(gc => gc.HeapCountSample.GCIndex, "HCSampleGCIndex", "#");
        public static Metric<TraceGC> HcsGCPauseTime = new(gc => gc.HeapCountSample.GCPauseTimeMSec, "HCSampleGCPause", "ms");
        public static Metric<TraceGC> HcsMslWaitTime = new(gc => gc.HeapCountSample.MslWaitTimeMSec, "HCSampleGCMslWait", "ms");

        // HeapCountTuning
        public static Metric<TraceGC> HctGCIndex = new(gc => gc.HeapCountTuning?.GCIndex, "HCTuningGCIndex", "#");
        public static Metric<TraceGC> HctMtcp = new((gc => gc.HeapCountTuning?.MedianThroughputCostPercent), "Median TCP", "%");
        public static Metric<TraceGC> HctMtcpCap15 = HctMtcp.WithCap(15);
        public static Metric<TraceGC> HctNewHeapCount = new(gc => gc.HeapCountTuning?.NewHeapCount, "HCTuningNewHeapCount", "#");
        public static Metric<TraceGC> HctSmtcp = new(gc => gc.HeapCountTuning?.SmoothedMedianThroughputCostPercent, "Smoothed MTCP", "%");
        public static Metric<TraceGC> HctSpaceCostDown = new(gc => gc.HeapCountTuning?.SpaceCostPercentDecreasePerStepDown, "Space cost (down)", "%");
        public static Metric<TraceGC> HctSpaceCostUp = new(gc => gc.HeapCountTuning?.SpaceCostPercentIncreasePerStepUp, "Space cost (up)TCP", "%");
        public static Metric<TraceGC> HctTPCostDown = new(gc => gc.HeapCountTuning?.ThroughputCostPercentIncreasePerStepDown, "TP cost (down)", "%");
        public static Metric<TraceGC> HctTPCostUp = new(gc => gc.HeapCountTuning?.ThroughputCostPercentReductionPerStepUp, "TP cost (up)", "%");

        public static Metric<TraceGC> HeapSizeAfter = new(gc => gc.HeapSizeAfterMB, "Heap size (after)", "MB");
        public static Metric<TraceGC> HeapSizeBefore = new(gc => gc.HeapSizeBeforeMB, "Heap size (before)", "MB");
        public static Metric<TraceGC> HeapSizePeak = new(gc => gc.HeapSizePeakMB, "Heap size (peak)", "MB");
        
        // TODO: HeapStats.*
        //public static Metric<TraceGC> Hs = new(gc => gc.HeapStats., "", "");

        // TODO: Remaining are less comprehensive
        public static Metric<TraceGC> PauseDuration = new((gc => gc.PauseDurationMSec), "GC pause", "ms");
        public static Metric<TraceGC> PausePercent = new((gc => gc.PauseTimePercentageSinceLastGC), "GC pause %", "%");
        public static Metric<TraceGC> EndOfSegAllocated = new(gc => gc.PerHeapHistories.Sum(p => p.EndOfSegAllocated), title: "EndOfSegAllocated", unit: "?");
        public static Metric<TraceGC> PauseStack = new(gc => gc.PerHeapMarkTimes.Values.Select(mi => mi.MarkTimes[(int) MarkRootType.MarkStack]).Sum(), "Pause (stack)", "ms");
        public static Metric<TraceGC> PauseFQ = new(gc => gc.PerHeapMarkTimes.Values.Select(mi => mi.MarkTimes[(int) MarkRootType.MarkFQ]).Sum(), "Pause (FQ)", "ms");
        public static Metric<TraceGC> PauseHandles = new(gc => gc.PerHeapMarkTimes.Values.Select(mi => mi.MarkTimes[(int) MarkRootType.MarkHandles]).Sum(), "Pause (handles)", "ms");
        public static Metric<TraceGC> PauseCards = new(gc => gc.PerHeapMarkTimes.Values.Select(mi => mi.MarkTimes[(int) MarkRootType.MarkOlder]).Sum(), "Pause (cards)", "ms");
        public static Metric<TraceGC> ObjectSpaceStack = new(gc => gc.PerHeapMarkTimes.Values.Select(mi => mi.MarkPromoted[(int) MarkRootType.MarkStack]).Sum(), "Obj space (stack)", "bytes");
        public static Metric<TraceGC> ObjectSpaceFQ = new(gc => gc.PerHeapMarkTimes.Values.Select(mi => mi.MarkPromoted[(int) MarkRootType.MarkFQ]).Sum(), "Obj space (FQ)", "bytes");
        public static Metric<TraceGC> ObjectSpaceHandles = new(gc => gc.PerHeapMarkTimes.Values.Select(mi => mi.MarkPromoted[(int) MarkRootType.MarkHandles]).Sum(), "Obj space (handles)", "bytes");
        public static Metric<TraceGC> ObjectSpaceCards = new(gc => gc.PerHeapMarkTimes.Values.Select(mi => mi.MarkPromoted[(int) MarkRootType.MarkOlder]).Sum(), "Obj space (cards)", "bytes");
        public static Metric<TraceGC> Suspend = new(gc => gc.SuspendDurationMSec, "Suspend", "ms");
        public static Metric<TraceGC> UserAllocated = new(gc => gc.UserAllocated.Sum(), "UserAllocated", "bytes");
    }

    public static class I
    {
        public static Metric<IterationData> MaxNumHeaps = Promote(Metrics.G.NumHeaps, Aggregation.Max);
        public static Metric<IterationData> MaxPauseDuration = Promote(Metrics.G.PauseDuration, Aggregation.Max);

        public static Metric<IterationData> TotalSuspensionTime = new (iterationData => iterationData.GCSummaryInfo.TotalSuspensionTimeMSec, "Total suspension time", "ms");
        public static Metric<IterationData> PercentPauseTimeInGC = new (iterationData => iterationData.GCSummaryInfo.PercentPauseTimeInGC, "% pause GC", "%");
        public static Metric<IterationData> PercentTimeInGC = new (iterationData => iterationData.GCSummaryInfo.PercentTimeInGC, "% GC", "%");
        public static Metric<IterationData> MeanHeapSizeBeforeMB = new (iterationData => iterationData.GCSummaryInfo.MeanHeapSizeBeforeMB, "Mean heap size (before)", "MB");
        public static Metric<IterationData> MaxHeapSizeMB = new (iterationData => iterationData.GCSummaryInfo.MaxHeapSizeMB, "Max heap size", "MB");
        public static Metric<IterationData> TotalAllocationsMB = new (iterationData => iterationData.GCSummaryInfo.TotalAllocationsMB, "Total allocations", "MB");
        public static Metric<IterationData> GCScore = new (iterationData => iterationData.GCSummaryInfo.GCScore, "GC score", "score"); // MB * %

        public static Metric<IterationData> MaxHeapCount = new (iterationData => iterationData.GCSummaryInfo.MaxHeapCount, "Max heap count", "#");
        public static Metric<IterationData> NumberOfHeapCountSwitches = new (iterationData => iterationData.GCSummaryInfo.NumberOfHeapCountSwitches, "# hc changes", "#");
        public static Metric<IterationData> NumberOfHeapCountDirectionChanges = new (iterationData => iterationData.GCSummaryInfo.NumberOfHeapCountDirectionChanges, "# hc dir changes", "#");

        public static Metric<IterationData> MaxWorkingSetMB = new (iterationData => iterationData.LoadInfo.MaxWorkingSetMB, "Max working set", "MB");
        public static Metric<IterationData> P99WorkingSetMB = new (iterationData => iterationData.LoadInfo.P99WorkingSetMB, "P99 working set", "MB");
        public static Metric<IterationData> P95WorkingSetMB = new (iterationData => iterationData.LoadInfo.P95WorkingSetMB, "P95 working set", "MB");
        public static Metric<IterationData> P90WorkingSetMB = new (iterationData => iterationData.LoadInfo.P90WorkingSetMB, "P90 working set", "MB");
        public static Metric<IterationData> P75WorkingSetMB = new (iterationData => iterationData.LoadInfo.P75WorkingSetMB, "P75 working set", "MB");
        public static Metric<IterationData> P50WorkingSetMB = new (iterationData => iterationData.LoadInfo.P50WorkingSetMB, "P50 working set", "MB");
        public static List<Metric<IterationData>> WorkingSetMBList = ML(MaxWorkingSetMB, P99PrivateMemoryMB, P95PrivateMemoryMB, P90PrivateMemoryMB, P75PrivateMemoryMB, P50PrivateMemoryMB);

        public static Metric<IterationData> MaxPrivateMemoryMB = new (iterationData => iterationData.LoadInfo.MaxPrivateMemoryMB, "Max private memory", "MB");
        public static Metric<IterationData> P99PrivateMemoryMB = new (iterationData => iterationData.LoadInfo.P99PrivateMemoryMB, "P99 private memory", "MB");
        public static Metric<IterationData> P95PrivateMemoryMB = new (iterationData => iterationData.LoadInfo.P95PrivateMemoryMB, "P95 private memory", "MB");
        public static Metric<IterationData> P90PrivateMemoryMB = new (iterationData => iterationData.LoadInfo.P90PrivateMemoryMB, "P90 private memory", "MB");
        public static Metric<IterationData> P75PrivateMemoryMB = new (iterationData => iterationData.LoadInfo.P75PrivateMemoryMB, "P75 private memory", "MB");
        public static Metric<IterationData> P50PrivateMemoryMB = new (iterationData => iterationData.LoadInfo.P50PrivateMemoryMB, "P50 private memory", "MB");
        public static List<Metric<IterationData>> PrivateMemoryMBList = ML(MaxPrivateMemoryMB, P99PrivateMemoryMB, P95PrivateMemoryMB, P90PrivateMemoryMB, P75PrivateMemoryMB, P50PrivateMemoryMB);

        public static Metric<IterationData> RequestsPerMSec = new (iterationData => iterationData.LoadInfo.RequestsPerMSec, "RPS", "RPS");
        public static Metric<IterationData> MeanLatencyMS = new (iterationData => iterationData.LoadInfo.MeanLatencyMS, "Mean latency", "ms");
        public static Metric<IterationData> Latency99thMS = new (iterationData => iterationData.LoadInfo.Latency99thMS, "Latency 99th", "ms");
        public static Metric<IterationData> Latency90thMS = new (iterationData => iterationData.LoadInfo.Latency90thMS, "Latency 90th", "ms");
        public static Metric<IterationData> Latency75thMS = new (iterationData => iterationData.LoadInfo.Latency75thMS, "Latency 75th", "ms");
        public static Metric<IterationData> Latency50thMS = new (iterationData => iterationData.LoadInfo.Latency50thMS, "Latency 50th", "ms");
        public static List<Metric<IterationData>> LatencyMSList = ML(MeanLatencyMS, Latency99thMS, Latency90thMS, Latency75thMS, Latency50thMS);
    }

    public static class B
    {
        public static Metric<BenchmarkData> MaxHeapCount = Promote(Metrics.I.MaxHeapCount, Aggregation.Max);
        public static Metric<BenchmarkData> MaxPauseDurationBenchmark = Promote(Metrics.I.MaxPauseDuration, Aggregation.Max);
        public static Metric<BenchmarkData> MaxPercentPauseTimeInGC = Promote(Metrics.I.PercentPauseTimeInGC, Aggregation.Max);
        public static Metric<BenchmarkData> AveragePercentPauseTimeInGC = Promote(Metrics.I.PercentPauseTimeInGC, Aggregation.Average);
    }

}

// Exploratory
public abstract class NameSimplifier
{
    public abstract (string title, Dictionary<string, string>) Simplify(List<string> names);

    public static PrefixSimplifier PrefixDashed { get; } = new PrefixSimplifier('-');
}

public class ListSimplifier : NameSimplifier
{
    private Dictionary<string, string> _nameMap;

    public ListSimplifier(params (string inData, string toDisplay)[] names)
        : this((IEnumerable<(string, string)>) names) {}

    public ListSimplifier(IEnumerable<(string inData, string toDisplay)> names)
        => _nameMap = names.ToDictionary();

    public override (string title, Dictionary<string, string>) Simplify(List<string> names) => (null, _nameMap);
}

public class PrefixSimplifier : NameSimplifier
{
    private char _delimiter;
    private string _emptyResult;

    public PrefixSimplifier(char delimiter, string emptyResult = "<>")
    {
        _delimiter = delimiter;
        _emptyResult = emptyResult;
    }

    public override (string title, Dictionary<string, string>) Simplify(List<string> names)
    {
        if (names.Count == 0) return (null, null);
        List<string> namesToScan = names;
        int longestMatch = namesToScan.Select(n => n.Length).Min();
        bool allContinueWithDelimiter = namesToScan.All(n => (n.Length == longestMatch) || (n[longestMatch] == _delimiter));
        if (allContinueWithDelimiter)
        {
            namesToScan = namesToScan.Select(n => ((allContinueWithDelimiter && (n.Length == longestMatch)) ? (n + _delimiter) : n)).ToList();
            longestMatch++;
        }
        foreach (string name in namesToScan)
        {
            int overlap = name.TakeWhile((ch, i) => (i < longestMatch) && (ch == namesToScan[0][i])).Count();
            longestMatch = (overlap == 0) ? 0 : name.LastIndexOf(_delimiter, overlap - 1) + 1;
            if (longestMatch == 0) break;
        }
        if (longestMatch > 0)
        {
            return (
                names[0].Substring(0, longestMatch - 1),
                names.Select(config => (config, (longestMatch >= config.Length) ? _emptyResult : config.Substring(longestMatch)))
                    .ToDictionary()
            );
        }
        return (null, null);
    }
}

// Some will be null depending on the chart type
record SeriesInfo<TData>(Metric<TData> Metric, string Run, string Config, ConfigData ConfigData, string Benchmark, int? Iteration, IterationData IterationData);

abstract class ChartType<TData>
{
    public abstract BaseMetric<(string, TData), XValue> DefaultXMetric { get; }
    public abstract string DefaultBenchmarkMap(string benchmark);

    public abstract IEnumerable<SeriesInfo<TData>> GetSeries(DataManager dataManager, List<Metric<TData>> metrics, Filter runFilter, Filter configFilter,
        Filter benchmarkFilter, IntFilter iterationFilter, ConfigIterationFilter configIterationFilter, IEnumerable<string> benchmarkList);
    public abstract string GetColorFamilyKey(SeriesInfo<TData> info, bool multipleMetrics, bool includeRunName, bool multipleConfigs,
        Dictionary<string, string> configDisplayNames, bool multipleBenchmarks);
    public abstract string GetColorFamilyId(SeriesInfo<TData> info, bool multipleMetrics);
    public abstract string GetSeriesTitle(SeriesInfo<TData> info, string colorFamilyKey, bool multipleMetrics);
    public abstract string GetChartTitle();
    public abstract List<KeyValuePair<string, TData>> GetDataSource(SeriesInfo<TData> info,
        Filter benchmarkFilter, IntFilter iterationFilter, ConfigIterationFilter configIterationFilter, Func<TData, bool> dataFilter);
}

class BenchmarksChartType : ChartType<BenchmarkData>
{
    public override BaseMetric<(string, BenchmarkData), XValue> DefaultXMetric { get; } = Metrics.X.BenchmarkName;
    public override string DefaultBenchmarkMap(string benchmark) => "";

    public override IEnumerable<SeriesInfo<BenchmarkData>> GetSeries(DataManager dataManager, List<Metric<BenchmarkData>> metrics, Filter runFilter, Filter configFilter,
        Filter benchmarkFilter, IntFilter iterationFilter, ConfigIterationFilter configIterationFilter, IEnumerable<string> benchmarkList)
    {
        foreach (var metric in metrics)
        {
            foreach ((string run, string config, ConfigData configData) in dataManager.GetConfigsWithData(runFilter, configFilter))
            {
                if (!configIterationFilter.MightInclude((config))) continue;

                // Note - could filter out configs that don't have a relevant benchmark/iteration
                yield return new (metric, run, config, configData, null, null, null);
            }
        }
    }

    public override string GetColorFamilyKey(SeriesInfo<BenchmarkData> info, bool multipleMetrics, bool includeRunName, bool multipleConfigs,
        Dictionary<string, string> configDisplayNames, bool multipleBenchmarks)
    {
        string runDisplay = includeRunName ? $"{info.Run}, " : "";
        string configDisplay = multipleConfigs ? (configDisplayNames?.GetValueOrDefault(info.Config) ?? info.Config) : "";
        string colorFamilyKey = $"{runDisplay}{configDisplay}";
        return colorFamilyKey;
    }

    public override string GetColorFamilyId(SeriesInfo<BenchmarkData> info, bool multipleMetrics) => multipleMetrics ? $"{info.Metric.Title} / " : "";
    public override string GetSeriesTitle(SeriesInfo<BenchmarkData> info, string colorFamilyKey, bool multipleMetrics) => $"{GetColorFamilyId(info, multipleMetrics)}{colorFamilyKey}";

    public override string GetChartTitle() => "Per-benchmark behavior";

    public override List<KeyValuePair<string, BenchmarkData>> GetDataSource(SeriesInfo<BenchmarkData> info,
        Filter benchmarkFilter, IntFilter iterationFilter, ConfigIterationFilter configIterationFilter, Func<BenchmarkData, bool> dataFilter)
        => info.ConfigData.Benchmarks
            .Where(benchmark => benchmarkFilter.Include(benchmark.Key)
                && benchmark.Value.Iterations.WithIndex()
                    .Any(pair => (pair.Item1 != null)
                        && iterationFilter.Include(pair.Item2)
                        && configIterationFilter.Include(info.Config, pair.Item2)));
}

class IterationsChartType : ChartType<IterationData>
{
    public override BaseMetric<(string, IterationData), XValue> DefaultXMetric { get; } = Metrics.X.IterationBenchmarkName;
    public override string DefaultBenchmarkMap(string benchmark) => "";

    public override IEnumerable<SeriesInfo<IterationData>> GetSeries(DataManager dataManager, List<Metric<IterationData>> metrics, Filter runFilter, Filter configFilter,
        Filter benchmarkFilter, IntFilter iterationFilter, ConfigIterationFilter configIterationFilter, IEnumerable<string> benchmarkList)
    {
        foreach (var metric in metrics)
        {
            foreach ((string run, string config, ConfigData configData) in dataManager.GetConfigsWithData(runFilter, configFilter))
            {
                foreach (int iteration in configData.GetIterations(config, benchmarkFilter, iterationFilter, configIterationFilter))
                {
                    yield return new (metric, run, config, configData, null, iteration, null);
                }
            }
        }
    }
        
    public override string GetColorFamilyKey(SeriesInfo<IterationData> info, bool multipleMetrics, bool includeRunName, bool multipleConfigs,
        Dictionary<string, string> configDisplayNames, bool multipleBenchmarks)
    {
        string metricDisplay = multipleMetrics ? $"{info.Metric.Title}, " : "";
        string runDisplay = includeRunName ? $"{info.Run}, " : "";
        string configDisplay = multipleConfigs ? (configDisplayNames?.GetValueOrDefault(info.Config) ?? info.Config) : "";
        string colorFamilyKey = $"{metricDisplay}{runDisplay}{configDisplay}";

        return colorFamilyKey;
    }

    public override string GetColorFamilyId(SeriesInfo<IterationData> info, bool multipleMetrics) => $"_{info.Iteration}";
    public override string GetSeriesTitle(SeriesInfo<IterationData> info, string colorFamilyKey, bool multipleMetrics) => $"{colorFamilyKey}{GetColorFamilyId(info, multipleMetrics)}";

    public override string GetChartTitle() => "Per-iteration behavior";

    public override List<KeyValuePair<string, IterationData>> GetDataSource(SeriesInfo<IterationData> info,
        Filter benchmarkFilter, IntFilter iterationFilter, ConfigIterationFilter configIterationFilter, Func<IterationData, bool> dataFilter)
    {
        if (!iterationFilter.Include(info.Iteration.Value)
            || !configIterationFilter.Include(info.Config, info.Iteration.Value))
        {
            throw new Exception("IterationsChartType.GetDataSource expected GetSeries to filter iterations");
        }

        return info.ConfigData.Benchmarks
            .Where(benchmark => benchmarkFilter.Include(benchmark.Key))
            .Where(benchmark => info.Iteration < benchmark.Value.Iterations.Count)
            .Select(benchmark => KeyValuePair.Create(benchmark.Key, benchmark.Value.Iterations[info.Iteration.Value]))
            .Where(kvp => kvp.Value != null);
    }
}

class TraceGCChartType : ChartType<TraceGC>
{
    public override BaseMetric<(string, TraceGC), XValue> DefaultXMetric { get; } = Metrics.X.GCIndex;
    public override string DefaultBenchmarkMap(string benchmark) => benchmark;
    
    public override IEnumerable<SeriesInfo<TraceGC>> GetSeries(DataManager dataManager, List<Metric<TraceGC>> metrics, Filter runFilter, Filter configFilter,
        Filter benchmarkFilter, IntFilter iterationFilter, ConfigIterationFilter configIterationFilter, IEnumerable<string> benchmarkList)
    {
        foreach (var metric in metrics)
        {
            foreach (string benchmark in benchmarkList)
            {
                foreach ((string run, string config, int iteration, IterationData iterationData) in
                    dataManager.GetIterationsForBenchmark(runFilter, configFilter, iterationFilter, configIterationFilter, benchmark))
                {
                    yield return new (metric, run, config, null, benchmark, iteration, iterationData);
                }
            }
        }
    }

    public override string GetColorFamilyKey(SeriesInfo<TraceGC> info, bool multipleMetrics, bool includeRunName, bool multipleConfigs,
        Dictionary<string, string> configDisplayNames, bool multipleBenchmarks)
    {
        string benchmarkDisplay = multipleBenchmarks ? $"{info.Benchmark}, " : "";
        string metricDisplay = multipleMetrics ? $"{info.Metric.Title}, " : "";
        string runDisplay = includeRunName ? $"{info.Run}, " : "";
        string configDisplay = multipleConfigs ? (configDisplayNames?.GetValueOrDefault(info.Config) ?? info.Config) : "";
        string colorFamilyKey = $"{benchmarkDisplay}{metricDisplay}{runDisplay}{configDisplay}";

        return colorFamilyKey;
    }

    public override string GetColorFamilyId(SeriesInfo<TraceGC> info, bool multipleMetrics) => $"_{info.Iteration}";
    public override string GetSeriesTitle(SeriesInfo<TraceGC> info, string colorFamilyKey, bool multipleMetrics) => $"{colorFamilyKey}{GetColorFamilyId(info, multipleMetrics)}";

    public override string GetChartTitle() => "Per-run behavior";

    public override List<KeyValuePair<string, TraceGC>> GetDataSource(SeriesInfo<TraceGC> info,
        Filter benchmarkFilter, IntFilter iterationFilter, ConfigIterationFilter configIterationFilter, Func<TraceGC, bool> dataFilter)
        => info.IterationData.GCProcessData?.GCs.Where(gc => gc.GlobalHeapHistory != null).Where(dataFilter).Select(gc => KeyValuePair.Create("", gc));
}

public struct XValue : IComparable<XValue>, IEquatable<XValue>
{
    private double _value;
    private string _name;

    public XValue(double value) { _value = value; _name = null; }
    public XValue(string name) { _value = 0; _name = name; }

    public bool HasValue => _name == null;
    public bool HasName => _name != null;

    public double GetValue() => HasValue ? _value : throw new Exception("XValue.GetValue on a named value");
    public string GetName() => HasName ? _name : throw new Exception("XValue.GetName on a numerical value");

    public override int GetHashCode() => HasValue ? GetValue().GetHashCode() : GetName().GetHashCode();
    public bool Equals(XValue other) => HasValue ? (other.HasValue && (GetValue() == other.GetValue())) : (other.HasName && (GetName() == other.GetName()));
    public override bool Equals(object other) => other is XValue otherX && Equals(otherX);

    public int CompareTo(XValue other)
        => (HasValue && other.HasName) ? 1
            : (HasName && other.HasValue) ? -1
            : HasValue ? GetValue().CompareTo(other.GetValue())
            : GetName().CompareTo(other.GetName());

    public override string ToString() => HasValue ? _value.ToString() : _name;
    public string ToString(string format) => HasValue ? _value.ToString(format) : _name;
}

public abstract class XArrangement
{
    private string _titleOverride;

    public XArrangement(string titleOverride) { _titleOverride = titleOverride; }

    public string GetNewTitle(string oldTitle) => _titleOverride ?? oldTitle;
    public abstract List<(XValue x, double? y)> Arrange(List<(XValue x, double? y)> data, List<(XValue x, double? y)> firstDataPreSorted);
    // This interface probably needs some work.  The idea is that, given the next xvalue in each series, this selects which one
    // should be next overall.
    public abstract XValue? ChooseNext(IEnumerable<XValue?> xavlues);

    public class DefaultXArrangement : XArrangement
    {
        public DefaultXArrangement() : base(null) {}
        public override List<(XValue x, double? y)> Arrange(List<(XValue x, double? y)> data, List<(XValue x, double? y)> firstDataPreSorted) => data;
        public override XValue? ChooseNext(IEnumerable<XValue?> xvalues) => xvalues.FirstOrDefault(x => x.HasValue);
    }

    public class PercentileXArrangement : XArrangement
    {
        private bool _descending;
        public PercentileXArrangement(bool descending = false) : base("Percentile") { _descending = descending; }
        public override List<(XValue x, double? y)> Arrange(List<(XValue x, double? y)> data, List<(XValue x, double? y)> firstDataPreSorted)
        {
            var sortedData = _descending
                ? data.Select(d => d.y).OrderByDescending(y => y)
                : data.Select(d => d.y).OrderBy(y => y);
            return sortedData.Select((d, i) => (new XValue(i / (double) data.Count), d)).ToList();
        }
        public override XValue? ChooseNext(IEnumerable<XValue?> xvalues) => xvalues.Min();
    }

    public class SortedXArrangement : XArrangement
    {
        private bool _descending;
        public SortedXArrangement(bool descending = true) : base("Metric Rank") { _descending = descending; }
        public override List<(XValue x, double? y)> Arrange(List<(XValue x, double? y)> data, List<(XValue x, double? y)> firstDataPreSorted)
        {
            var sortedData = _descending
                ? data.Select(d => d.y).OrderByDescending(y => y)
                : data.Select(d => d.y).OrderBy(y => y);
            return sortedData.Select((d, i) => (new XValue(i), d)).ToList();
        }
        public override XValue? ChooseNext(IEnumerable<XValue?> xvalues) => xvalues.Min();
    }

    public class CombinedSortedXArrangement : XArrangement
    {
        public CombinedSortedXArrangement() : base(null) {}
        public override List<(XValue x, double? y)> Arrange(List<(XValue x, double? y)> data, List<(XValue x, double? y)> firstDataPreSorted)
            => data.Join(firstDataPreSorted, d => d.x, d => d.x, ((d, sortedEntry) => (d.x, d.y, sortedEntry.y)))
                .OrderByDescending(triple => triple.Item3)
                .Select(triple => (triple.x, triple.Item2));
        public override XValue? ChooseNext(IEnumerable<XValue?> xvalues) => xvalues.FirstOrDefault(x => x.HasValue);
    }

    public class RelativeXArrangement : XArrangement
    {
        public RelativeXArrangement() : base(null) {}
        public override List<(XValue x, double? y)> Arrange(List<(XValue x, double? y)> data, List<(XValue x, double? y)> firstDataPreSorted)
        {
            if (data.Count == 0) return data;
            if (data[0].x.HasName)
            {
                Console.WriteLine($"Applying {nameof(RelativeXArrangement)} on non-numeric x values (first is {data[0].x})");
                return data;
            }
            double firstValue = data[0].x.GetValue();
            return data.Select(d => (new XValue(d.x.GetValue() - firstValue), d.y));
        }
        public override XValue? ChooseNext(IEnumerable<XValue?> xvalues) => xvalues.Min(); // not necessarily?
    }
}

public static class XArrangements
{
    public static XArrangement.DefaultXArrangement Default { get; } = new ();
    public static XArrangement.PercentileXArrangement Percentile { get; } = new();
    public static XArrangement.SortedXArrangement Sorted { get; } = new();
    public static XArrangement.CombinedSortedXArrangement CombinedSorted { get; } = new();
    public static XArrangement.RelativeXArrangement Relative { get; } = new();
}

public abstract class DataPresenter<TResult>
{
    public bool Debug;

    public abstract void Clear();

    // true if ok
    public abstract bool PrepareUnits(IEnumerable<string> units);
    public abstract void SetColorGroups(Dictionary<string, int> colorGroups);
    public abstract void Display();
    public abstract TResult Result { get; }

    // Below members are per-chart

    public abstract void Start(string title, string xlabel);
    public abstract void AddSeries(string title, string unit, string colorFamilyKey, string colorFamilyId, List<(XValue x, double? y)> data);
    public abstract void Finish(XArrangement xArrangement);
}

public abstract class TextPresenter : DataPresenter<List<List<string>>>
{
    public static RawTextPresenter RawText { get; } = new RawTextPresenter();
    public static MarkdownPresenter Markdown { get; } = new MarkdownPresenter();
    public static HtmlPresenter Html { get; } = new HtmlPresenter();
    public static CsvPresenter Csv { get; } = new CsvPresenter();

    protected record struct DataPoint(XValue x, double? y);
    private record Series(string title, string unit, List<DataPoint> data);
    private record Table(string title, string xlabel, List<Series> series);

    private List<List<string>> _result = new();
    public override void Clear() => _result.Clear();

    // true if ok
    public override bool PrepareUnits(IEnumerable<string> units) => true;
    public override void SetColorGroups(Dictionary<string, int> colorGroups) {}

    public override List<List<string>> Result => _result;
    
    // Below members are per-table

    private Table _current;

    public override void Start(string title, string xlabel) { _current = new(title: title, xlabel: xlabel, series: new()); }
    public override void AddSeries(string title, string unit, string colorFamilyKey, string colorFamilyId, List<(XValue x, double? y)> data)
        => _current.series.Add(new(title: title, unit: unit, data: data.Select(pair => new DataPoint(pair.x, pair.y)).ToList()));

    private int MaxTokenLength(string phrase) => phrase.Split(' ').Select(s => s.Length).Max();
    protected string NDashes(int n) => new string('-', n);
    protected string NSpaces(int n) => new string(' ', n);
    protected void PadLeft(StringBuilder sb, int width) => sb.Insert(0, NSpaces(width - sb.Length));

    protected abstract string MakeTitle(string title);
    protected abstract string? StartTable();
    protected abstract IEnumerable<string> HeaderLines(IEnumerable<string> headerValues, IEnumerable<int> widths);
    protected abstract string? HeaderBorder(IEnumerable<int> widths);
    protected abstract IEnumerable<string> DataLine(XValue xvalue, IEnumerable<DataPoint?> values, IEnumerable<int> widths);
    protected abstract string? EndTable();

    protected const string lineStart = "| ";
    protected const string lineDelim = " | ";
    protected const string lineEnd = " |";

    private static void AddIfNotNull(List<string> list, string? value)
    {
        if (value != null) list.Add(value);
    }

    public override void Finish(XArrangement xArrangement)
    {

        int xWidth = _current.series.SelectMany(series => series.data.Select(d => d.x.ToString().Length))
            .Append(MaxTokenLength(_current.xlabel))
            .Max();
        List<int> seriesWidths = _current.series.Select(
            series => series.data.Select(d => d.y?.ToString("N3").Length ?? 0)
                .Append((this is RawTextPresenter) ? MaxTokenLength(series.title) : series.title.Length)
                .Append(series.unit.Length + 2) // "(<unit>)"
                .Max());
        var allWidths = seriesWidths.Prepend(xWidth);
        List<string> tableText = new();
        tableText.Add(MakeTitle(_current.title));
        tableText.Add("");
        AddIfNotNull(tableText, StartTable());

        var headerValues = _current.series.Select(series => series.title).Prepend(_current.xlabel);
        tableText.AddRange(HeaderLines(headerValues, allWidths));
        AddIfNotNull(tableText, HeaderBorder(allWidths));

        int numSeries = _current.series.Count;
        int[] nextIndices = new int[numSeries]; // all zeroes
        DataPoint?[] candidates = new DataPoint?[numSeries];
        string[] elements = new string[numSeries + 1]; // includes X
        while (true)
        {
            // Find next xvalue, if it exists.
            for (int i = 0; i < numSeries; ++i)
            {
                while ((nextIndices[i] < _current.series[i].data.Count)
                    && !_current.series[i].data[nextIndices[i]].y.HasValue)
                {
                    nextIndices[i]++;
                }

                candidates[i] = nextIndices[i] < _current.series[i].data.Count
                    ? _current.series[i].data[nextIndices[i]]
                    : null;
            }
            XValue? next = xArrangement.ChooseNext(candidates.Select(p => p?.x));
            if (!next.HasValue) break;

            // Get values
            for (int i = 0; i < numSeries; ++i)
            {
                if (!candidates[i].HasValue) continue;
                if (!next.Value.Equals(candidates[i].Value.x))
                {
                    candidates[i] = null;
                    continue;
                }
                nextIndices[i]++;
            }
            if (!candidates.Any(NotNull)) throw new Exception("internal error - no candidate used");

            tableText.AddRange(DataLine(next.Value, candidates, allWidths));
        }

        AddIfNotNull(tableText, EndTable());
        _result.Add(tableText);
    }
}

public class RawTextPresenter : TextPresenter
{
    private const string borderStart = "+ ";
    private const string borderDelim = "-+-";
    private const string borderEnd = " +";

    protected override string MakeTitle(string title) => title;
    protected override string? StartTable() => null;
    
    private List<string> MakeLines(string phrase, int width)
    {
        List<string> result = new();

        string[] tokens = phrase.Split(' ');
        StringBuilder current = new();
        foreach (string token in tokens)
        {
            if (token.Length > width) throw new Exception("Tokenization inconsistent");
            if ((current.Length + token.Length + 1) > width)
            {
                PadLeft(current, width);
                result.Add(current.ToString());
                current = new();
            }

            if (current.Length > 0) current.Append(' ');
            current.Append(token);
        }
        PadLeft(current, width);
        result.Add(current.ToString());

        return result;
    }

    protected override IEnumerable<string> HeaderLines(IEnumerable<string> headerValues, IEnumerable<int> widths)
    {
        var headerCells = headerValues.Zip(widths).Select(headerAndWidth => MakeLines(headerAndWidth.First, headerAndWidth.Second));
        int maxHeaderLines = headerCells.Select(lines => lines.Count).Max();
        foreach ((List<string> cell, int width) in headerCells.Zip(widths))
        {
            while (cell.Count < maxHeaderLines) cell.Insert(0, NSpaces(width));
        }
        for (int i = 0; i < maxHeaderLines; ++i)
        {
            yield return (lineStart + string.Join(lineDelim, headerCells.Select(cell => cell[i])) + lineEnd);
        }
    }

    protected override string? HeaderBorder(IEnumerable<int> widths)
        => borderStart + string.Join(borderDelim, widths.Select(n => NDashes(n))) + borderEnd;

    protected override IEnumerable<string> DataLine(XValue xvalue, IEnumerable<DataPoint?> values, IEnumerable<int> widths)
    {
        var cells = values.Select(p => p?.y?.ToString("N3")).Prepend(xvalue.ToString())
            .Zip(widths).Select(p => (p.First ?? "").PadLeft(p.Second));
        yield return lineStart + string.Join(lineDelim, cells) + lineEnd;
    }

    protected override string? EndTable() => null;

    public override void Display()
    {
        foreach (List<string> table in Result)
        {

            Console.WriteLine();
            foreach (string line in table)
            {
                Console.WriteLine(line);
            }
        }
    }
}

public class MarkdownPresenter : TextPresenter
{
    protected override string MakeTitle(string title) => $"### {title}";
    protected override string? StartTable() => null;

    protected override IEnumerable<string> HeaderLines(IEnumerable<string> headerValues, IEnumerable<int> widths)
    {
        yield return lineStart + string.Join(lineDelim, headerValues.Zip(widths).Select(pair => pair.First.PadLeft(pair.Second))) + lineEnd;
    }

    protected override string? HeaderBorder(IEnumerable<int> widths)
        => lineStart + string.Join(lineDelim, widths.Select(n => NDashes(n-1) + ":")) + lineEnd;

    protected override IEnumerable<string> DataLine(XValue xvalue, IEnumerable<DataPoint?> values, IEnumerable<int> widths)
    {
        var cells = values.Select(p => p?.y?.ToString("N3")).Prepend(xvalue.ToString())
            .Zip(widths).Select(p => (p.First ?? "").PadLeft(p.Second));
        yield return lineStart + string.Join(lineDelim, cells) + lineEnd;
    }

    protected override string? EndTable() => null;

    public override void Display()
    {
        foreach (List<string> table in Result)
        {
            string.Join("\n", table).DisplayAs("text/markdown");
        }
    }
}

public class HtmlPresenter : TextPresenter
{
    protected override string MakeTitle(string title) => $"<h3>{title}</h3>";
    protected override string? StartTable() => "<table>";
    protected override IEnumerable<string> HeaderLines(IEnumerable<string> headerValues, IEnumerable<int> widths)
    {
        yield return "<tr>";
        foreach (string value in headerValues)
        {
            yield return $"  <th>{value}</th>";
        }
        yield return "</tr>";
    }

    protected override string? HeaderBorder(IEnumerable<int> widths) => null;

    protected override IEnumerable<string> DataLine(XValue xvalue, IEnumerable<DataPoint?> values, IEnumerable<int> widths)
    {
        var cells = values.Select(p => p?.y?.ToString("N3")).Prepend(xvalue.ToString());
        yield return "<tr>";
        foreach (string value in cells)
        {
            yield return $"  <th>{value}</th>";
        }
        yield return "</tr>";
    }

    protected override string? EndTable() => "</table>";

    public override void Display()
    {
        foreach (List<string> table in Result)
        {
            string.Join("\n", table).DisplayAs("text/html");
        }
    }
}

public class CsvPresenter : TextPresenter
{
    protected override string MakeTitle(string title) => $"# {title}";
    protected override string? StartTable() => null;
    protected override IEnumerable<string> HeaderLines(IEnumerable<string> headerValues, IEnumerable<int> widths)
    {
        yield return string.Join(",", headerValues);
    }

    protected override string? HeaderBorder(IEnumerable<int> widths) => null;

    protected override IEnumerable<string> DataLine(XValue xvalue, IEnumerable<DataPoint?> values, IEnumerable<int> widths)
    {
        var cells = values.Select(p => p?.y?.ToString("N3")).Prepend(xvalue.ToString());
        yield return string.Join(",", cells);
    }

    protected override string? EndTable() => null;

    public override void Display()
    {
        foreach (List<string> table in Result)
        {
            string.Join("\n", table).DisplayAs("text/csv");
        }
    }
}

public class ChartPresenter : DataPresenter<List<PlotlyChart>>
{
    private string _scatterMode;
    private List<string> _uniqueUnits;
    private ColorProvider _colorProvider;
    private List<PlotlyChart> _charts = new();

    public ChartPresenter(string scatterMode = null)
    {
        _scatterMode = scatterMode;
    }

    public override void Clear() => _charts.Clear();

    public override bool PrepareUnits(IEnumerable<string> units)
    {
        _uniqueUnits = new();
        foreach (string unit in units)
        {
            if (!_uniqueUnits.Contains(unit)) _uniqueUnits.Add(unit);
        }
        if (_uniqueUnits.Count > 2)
        {
            Console.WriteLine($"Too many units: {string.Join(", ", _uniqueUnits)}");
            return false;
        }
        return true;
    }

    private int yaxis(string unit) => _uniqueUnits.IndexOf(unit);

    public override void SetColorGroups(Dictionary<string, int> colorGroups)
    {
        _colorProvider = new(colorGroups);
        if (Debug) _colorProvider.DumpColorGroups();
    }

    public override void Display()
    {
        foreach (PlotlyChart chart in Result) chart.Display();
    }

    public override List<PlotlyChart> Result => _charts;

    // Below members are per-chart

    private Layout.Layout _layout;
    private List<Scatter> _scatters;

    public override void Start(string title, string xlabel)
    {
        _layout = new Layout.Layout
            {
                xaxis = new Xaxis { title = xlabel },
                yaxis = new Yaxis { title = _uniqueUnits[0] },
                title = title,
                // margin = new Margin() { r = 123 },
            };

        if (_uniqueUnits.Count > 1)
        {
            _layout.yaxis2 = new Yaxis { title = _uniqueUnits[1], side = "right", overlaying = "y" };
        }

        _scatters = new();
    }

    public override void AddSeries(string title, string unit, string colorFamilyKey, string colorFamilyId, List<(XValue x, double? y)> data)
    {
        Scatter scatter =
            new Scatter {
                name = title,
                x = data[0].x.HasName ? data.Select(d => d.x.GetName()) : data.Select(d => d.x.GetValue()),
                y = data.Select(d => d.y),
            };
        if (_scatterMode != null) scatter.mode = _scatterMode;
        if (yaxis(unit) == 1) scatter.yaxis = "y2";
        _colorProvider.SetMarker(scatter, colorFamilyKey, colorFamilyId);
        // scatter.marker will throw if marker hasn't been set.
        // ShouldSerializemarker appears to check if it has been set.
        if (Debug) Console.WriteLine($"color '{colorFamilyKey}': '{(scatter.ShouldSerializemarker() ? scatter.marker.color : "")}'");
        _scatters.Add(scatter);
    }

    public override void Finish(XArrangement xArrangement) => _charts.Add(Chart.Plot(_scatters, _layout));
}

TResult ChartInternal<TData, TResult>(DataPresenter<TResult> presenter, ChartType<TData> chartType,
    DataManager dataManager, List<Metric<TData>> metrics,
    Filter runFilter = null, Filter configFilter = null, Filter benchmarkFilter = null, IntFilter iterationFilter = null,
    ConfigIterationFilter configIterationFilter = null, Func<TData, bool> dataFilter = null, Func<string, string> benchmarkMap = null,
    BaseMetric<(string, TData), XValue> xMetric = null, XArrangement xArrangement = null,
    NameSimplifier configNameSimplifier = null, bool includeRunName = false,
    bool display = true, bool debug = false)
{
    runFilter = runFilter ?? Filter.All;
    configFilter = configFilter ?? Filter.All;
    benchmarkFilter = benchmarkFilter ?? Filter.All;
    iterationFilter = iterationFilter ?? IntFilter.All;
    // configIterationFilter is not set to an empty dictionary as that would exclude everything
    dataFilter = dataFilter ?? (data => true);
    benchmarkMap = benchmarkMap ?? chartType.DefaultBenchmarkMap;
    xMetric = xMetric ?? chartType.DefaultXMetric;
    xArrangement = xArrangement ?? XArrangements.Default;

    presenter.Clear();
    presenter.Debug = debug;

    if (metrics.Count == 0)
    {
        Console.WriteLine("No metrics");
        return default(TResult);
    }

    List<string> configs = dataManager.GetConfigs(runFilter: runFilter, configFilter: configFilter).Select(tuple => tuple.config).Distinct().ToList();
    if (configs.Count == 0)
    {
        Console.WriteLine("No configs afer filtering");
        return default(TResult);
    }

    if (debug) Console.WriteLine("Simplify config names");
    Dictionary<string, string> configDisplayNames = null;
    string configPrefix = null;
    if (configNameSimplifier != null)
    {
        (configPrefix, configDisplayNames) = configNameSimplifier.Simplify(configs);
    }
    
    if (debug) Console.WriteLine("Prepare units");
    presenter.PrepareUnits(metrics.Select(metric => metric.Unit));

    Dictionary<string, List<string>> benchmarkGroups = new();
    HashSet<string> benchmarkSet = new();
    foreach ((string run, string config, string benchmark) in
        dataManager.GetBenchmarks(runFilter: runFilter, configFilter: configFilter, benchmarkFilter: benchmarkFilter, iterationFilter: iterationFilter,
            configIterationFilter: configIterationFilter))
    {
        if (!benchmarkSet.Add(benchmark)) continue;

        string benchmarkGroup = (benchmarkMap != null) ? benchmarkMap(benchmark) : benchmark;
        benchmarkGroups.GetOrAdd(benchmarkGroup, new());
        benchmarkGroups[benchmarkGroup].Add(benchmark);
    }

    foreach (var (groupName, benchmarkList) in benchmarkGroups)
    {
        benchmarkList.Sort();

        if (debug)
        {
            Console.Write($"{groupName}:");
            foreach (var benchmark in benchmarkList)
            {
                Console.Write($" {benchmark}");
            }
            Console.WriteLine();
        }
    }

    foreach (var (benchmarkGroup, benchmarkList) in benchmarkGroups)
    {
        if (debug) Console.WriteLine("Initialize colors");

        // Consider moving 'colorGroups' to the presenter
        Dictionary<string, int> colorGroups = new();
        foreach (SeriesInfo<TData> info in
            chartType.GetSeries(dataManager, metrics, runFilter: runFilter, configFilter: configFilter, benchmarkFilter: benchmarkFilter,
                iterationFilter: iterationFilter, configIterationFilter: configIterationFilter, benchmarkList: benchmarkList))
        {
            string colorFamilyKey = chartType.GetColorFamilyKey(info, multipleMetrics: metrics.Count > 1, includeRunName: includeRunName, multipleConfigs: configs.Count > 1,
                configDisplayNames: configDisplayNames, multipleBenchmarks: benchmarkList.Count > 1);

            colorGroups[colorFamilyKey] = colorGroups.GetValueOrDefault(colorFamilyKey, 0) + 1;
        }

        presenter.SetColorGroups(colorGroups);

        {
            List<Scatter> scatters = new();

            string xlabel = xArrangement.GetNewTitle(xMetric.Title);

            string titlePrefix = chartType.GetChartTitle();
            List<string> titleParts = new();
            if (!string.IsNullOrWhiteSpace(benchmarkGroup)) titleParts.Add(benchmarkGroup);
            if (metrics.Count == 1) titleParts.Add(metrics[0].Title);
            if (configPrefix != null) titleParts.Add(configPrefix);
            else if (configs.Count == 1) titleParts.Add(configDisplayNames?.GetValueOrDefault(configs[0]) ?? configs[0]);
            string titleWithoutPrefix = string.Join(" / ", titleParts);
            string title = string.Join(" / ", titleParts.Prepend(titlePrefix));
            presenter.Start(title: title, xlabel: xlabel);

            List<(XValue x, double? y)> firstDataPreSorted = null;
            double firstDataMin = 0;
            HashSet<XValue> firstDataSet = new();

            foreach ((SeriesInfo<TData> info, int indexForOffsetting) in
                chartType.GetSeries(dataManager, metrics, runFilter: runFilter, configFilter: configFilter, benchmarkFilter: benchmarkFilter,
                    iterationFilter: iterationFilter, configIterationFilter: configIterationFilter, benchmarkList: benchmarkList).WithIndex())
            {
                string colorFamilyKey = chartType.GetColorFamilyKey(info, multipleMetrics: metrics.Count > 1, includeRunName: includeRunName, multipleConfigs: configs.Count > 1,
                    configDisplayNames: configDisplayNames, multipleBenchmarks: benchmarkList.Count > 1);
                string seriesTitle = chartType.GetSeriesTitle(info, colorFamilyKey, metrics.Count > 1);
                if (debug) Console.Write($"series title: {seriesTitle}, ");

                List<KeyValuePair<string, TData>> dataSource;
                try { dataSource = chartType.GetDataSource(info, benchmarkFilter: benchmarkFilter, iterationFilter: iterationFilter,
                    configIterationFilter: configIterationFilter, dataFilter: dataFilter); }
                catch (Exception e) { Console.WriteLine($"Exception {e} processing data source for {title} / {seriesTitle}"); dataSource = null; }
                if (dataSource == null)
                {
                    Console.WriteLine($"No data for {titleWithoutPrefix} / {seriesTitle}");
                    continue;
                }
                int dataSourceCount = dataSource.Count;
                if (debug) Console.Write($"source count = {dataSourceCount}, ");

                List<(XValue x, double? y)> data;
                // Theory: For numeric x values, null y values need to be filtered or "mode==lines" won't show
                //         values that have null neighbors.
                // Theory: For non-numeric x values, null y values are needed to avoid shuffling of the x values
                //         (Example: if series 1 has "a" "c" and series 2 has "a" "b" "c", then 2 will be displayed
                //         "a" "c" "b" -AND- "mode==lines" will connect the "a" to the "b" to the "c")
                //         TODO: We probably need to add fake entries to the first (?) series if the different
                //         series have different sets of x values. The existing code will work if the x value
                //         exists in the DataManager but the metrics don't. (Example: we have ASP.NET metrics but
                //         no GC trace for a benchmark, but the chart contains GC metrics)
                info.Metric.ResetDiagnostics();
                try { data = dataSource.Select(b => (x: xMetric.DoExtract((b.Key, b.Value)), y: info.Metric.DoExtract(b.Value, indexForOffsetting))).ToList(); }
                catch { Console.WriteLine($"Exception processing data items for {title} / {seriesTitle}"); data = null; }
                info.Metric.DisplayDiagnostics($"{titleWithoutPrefix} / {seriesTitle}");
                if (debug) Console.Write($"data count = {data.Count}, ");
                if (!data.Any(d => d.y != null))
                {
                    Console.WriteLine($"No data items for {titleWithoutPrefix} / {seriesTitle}");
                    continue;
                }

                // This should probably be factored into CombinedSortedXArrangement.  The idea is that firstDataPreSorted
                // contains the first series' data so that each series can be merged into it, sorted the same way, and
                // then all displayed in the same order of x values.  However, the first series might not have all of the
                // values, so this tacks them on the end arbitrarily.
                if (firstDataPreSorted == null)
                {
                    firstDataPreSorted = new(data); // make a copy so that edits don't change the original
                    firstDataMin = firstDataPreSorted.Select(pair => pair.y).Where(NotNull).Min(y => y.Value);
                    firstDataSet = new(firstDataPreSorted.Select(pair => pair.x));
                }
                foreach (var d in data)
                {
                    if (firstDataSet.Add(d.x))
                    {
                        // The "--" is a hack to produce lower values.  This should be fixed to be clearer.
                        firstDataPreSorted.Add((d.x, --firstDataMin));
                    }
                }

                data = xArrangement.Arrange(data, firstDataPreSorted);

                // See above comment.  If x values are numeric, remove ones without y values.
                // Note that xarrangement can change the x value type.
                if (data[0].x.HasValue)
                {
                    data = data.Where(d => d.y != null);
                }

                if (debug) Console.Write($"data count = {data.Count}, ");
                if (data.Count == 0)
                {
                    Console.WriteLine($"No data items after filtering nulls for {titleWithoutPrefix} / {seriesTitle}");
                    continue;
                }

                string colorFamilyId = chartType.GetColorFamilyId(info, multipleMetrics: metrics.Count > 1);
                presenter.AddSeries(title: seriesTitle, unit: info.Metric.Unit, colorFamilyKey: colorFamilyKey, colorFamilyId: colorFamilyId, data: data);
            }

            presenter.Finish(xArrangement);
        }
    }

    if (display)
    {
        presenter.Display();
    }

    return presenter.Result;
}

List<List<string>> TableBenchmarks(DataManager dataManager, List<Metric<BenchmarkData>> metrics, TextPresenter textPresenter = null,
    Filter runFilter = null, Filter configFilter = null, Filter benchmarkFilter = null, IntFilter iterationFilter = null,
    ConfigIterationFilter configIterationFilter = null, Func<BenchmarkData, bool> dataFilter = null,
    Func<string, string> benchmarkMap = null, BaseMetric<(string, BenchmarkData), XValue> xMetric = null, XArrangement xArrangement = null,
    NameSimplifier configNameSimplifier = null, bool includeRunName = false,
    bool display = true, bool debug = false)
    => ChartInternal(textPresenter ?? TextPresenter.RawText, new BenchmarksChartType(),
        dataManager, metrics,
        runFilter: runFilter, configFilter: configFilter, benchmarkFilter: benchmarkFilter, iterationFilter: iterationFilter,
        configIterationFilter: configIterationFilter, dataFilter: dataFilter,
        benchmarkMap: benchmarkMap, xMetric: xMetric, xArrangement: xArrangement,
        configNameSimplifier: configNameSimplifier, includeRunName: includeRunName,
        display: display, debug: debug);

List<List<string>> TableBenchmarks(DataManager dataManager, Metric<BenchmarkData> metric, TextPresenter textPresenter = null,
    Filter runFilter = null, Filter configFilter = null, Filter benchmarkFilter = null, IntFilter iterationFilter = null,
    ConfigIterationFilter configIterationFilter = null, Func<BenchmarkData, bool> dataFilter = null,
    Func<string, string> benchmarkMap = null, BaseMetric<(string, BenchmarkData), XValue> xMetric = null, XArrangement xArrangement = null,
    NameSimplifier configNameSimplifier = null, bool includeRunName = false,
    bool display = true, bool debug = false)
    => TableBenchmarks(dataManager, ML(metric), textPresenter,
        runFilter: runFilter, configFilter: configFilter, benchmarkFilter: benchmarkFilter, iterationFilter: iterationFilter,
        configIterationFilter: configIterationFilter, dataFilter: dataFilter,
        benchmarkMap: benchmarkMap, xMetric: xMetric, xArrangement: xArrangement,
        configNameSimplifier: configNameSimplifier, includeRunName: includeRunName,
        display: display, debug: debug);

List<List<string>> TableIterations(DataManager dataManager, List<Metric<IterationData>> metrics, TextPresenter textPresenter = null,
    Filter runFilter = null, Filter configFilter = null, Filter benchmarkFilter = null, IntFilter iterationFilter = null,
    ConfigIterationFilter configIterationFilter = null, Func<IterationData, bool> dataFilter = null,
    Func<string, string> benchmarkMap = null, BaseMetric<(string, IterationData), XValue> xMetric = null, XArrangement xArrangement = null,
    NameSimplifier configNameSimplifier = null, bool includeRunName = false,
    bool display = true, bool debug = false)
    => ChartInternal(textPresenter ?? TextPresenter.RawText, new IterationsChartType(),
        dataManager, metrics,
        runFilter: runFilter, configFilter: configFilter, benchmarkFilter: benchmarkFilter, iterationFilter: iterationFilter,
        configIterationFilter: configIterationFilter, dataFilter: dataFilter,
        benchmarkMap: benchmarkMap, xMetric: xMetric, xArrangement: xArrangement,
        configNameSimplifier: configNameSimplifier, includeRunName: includeRunName,
        display: display, debug: debug);

List<List<string>> TableIterations(DataManager dataManager, Metric<IterationData> metric, TextPresenter textPresenter = null,
    Filter runFilter = null, Filter configFilter = null, Filter benchmarkFilter = null, IntFilter iterationFilter = null,
    ConfigIterationFilter configIterationFilter = null, Func<IterationData, bool> dataFilter = null,
    Func<string, string> benchmarkMap = null, BaseMetric<(string, IterationData), XValue> xMetric = null, XArrangement xArrangement = null,
    NameSimplifier configNameSimplifier = null, bool includeRunName = false,
    bool display = true, bool debug = false)
    => TableIterations(dataManager, ML(metric), textPresenter,
        runFilter: runFilter, configFilter: configFilter, benchmarkFilter: benchmarkFilter, iterationFilter: iterationFilter,
        configIterationFilter: configIterationFilter, dataFilter: dataFilter,
        benchmarkMap: benchmarkMap, xMetric: xMetric, xArrangement: xArrangement,
        configNameSimplifier: configNameSimplifier, includeRunName: includeRunName,
        display: display, debug: debug);

List<List<string>> TableGCData(DataManager dataManager, List<Metric<TraceGC>> metrics, TextPresenter textPresenter = null,
    Filter runFilter = null, Filter configFilter = null, Filter benchmarkFilter = null, IntFilter iterationFilter = null,
    ConfigIterationFilter configIterationFilter = null, Func<TraceGC, bool> dataFilter = null,
    Func<string, string> benchmarkMap = null, BaseMetric<(string, TraceGC), XValue> xMetric = null, XArrangement xArrangement = null,
    NameSimplifier configNameSimplifier = null, bool includeRunName = false,
    bool display = true, bool debug = false)
    => ChartInternal(textPresenter ?? TextPresenter.RawText, new TraceGCChartType(),
        dataManager, metrics,
        runFilter: runFilter, configFilter: configFilter, benchmarkFilter: benchmarkFilter, iterationFilter: iterationFilter,
        configIterationFilter: configIterationFilter, dataFilter: dataFilter,
        benchmarkMap: benchmarkMap, xMetric: xMetric, xArrangement: xArrangement,
        configNameSimplifier: configNameSimplifier, includeRunName: includeRunName,
        display: display, debug: debug);

List<List<string>> TableGCData(DataManager dataManager, Metric<TraceGC> metric, TextPresenter textPresenter = null,
    Filter runFilter = null, Filter configFilter = null, Filter benchmarkFilter = null, IntFilter iterationFilter = null,
    ConfigIterationFilter configIterationFilter = null, Func<TraceGC, bool> dataFilter = null,
    Func<string, string> benchmarkMap = null, BaseMetric<(string, TraceGC), XValue> xMetric = null, XArrangement xArrangement = null,
    NameSimplifier configNameSimplifier = null, bool includeRunName = false,
    bool display = true, bool debug = false)
    => TableGCData(dataManager, ML(metric), textPresenter,
        runFilter: runFilter, configFilter: configFilter, benchmarkFilter: benchmarkFilter, iterationFilter: iterationFilter,
        configIterationFilter: configIterationFilter, dataFilter: dataFilter,
        benchmarkMap: benchmarkMap, xMetric: xMetric, xArrangement: xArrangement,
        configNameSimplifier: configNameSimplifier, includeRunName: includeRunName,
        display: display, debug: debug);

List<PlotlyChart> ChartBenchmarks(DataManager dataManager, List<Metric<BenchmarkData>> metrics,
    Filter runFilter = null, Filter configFilter = null, Filter benchmarkFilter = null, IntFilter iterationFilter = null,
    ConfigIterationFilter configIterationFilter = null, Func<BenchmarkData, bool> dataFilter = null,
    Func<string, string> benchmarkMap = null, BaseMetric<(string, BenchmarkData), XValue> xMetric = null, XArrangement xArrangement = null,
    NameSimplifier configNameSimplifier = null, bool includeRunName = false,
    bool display = true, bool debug = false)
    => ChartInternal(new ChartPresenter(scatterMode: null), new BenchmarksChartType(),
        dataManager, metrics,
        runFilter: runFilter, configFilter: configFilter, benchmarkFilter: benchmarkFilter, iterationFilter: iterationFilter,
        configIterationFilter: configIterationFilter, dataFilter: dataFilter,
        benchmarkMap: benchmarkMap, xMetric: xMetric, xArrangement: xArrangement,
        configNameSimplifier: configNameSimplifier, includeRunName: includeRunName,
        display: display, debug: debug);

List<PlotlyChart> ChartBenchmarks(DataManager dataManager, Metric<BenchmarkData> metric,
    Filter runFilter = null, Filter configFilter = null, Filter benchmarkFilter = null, IntFilter iterationFilter = null,
    ConfigIterationFilter configIterationFilter = null, Func<BenchmarkData, bool> dataFilter = null,
    Func<string, string> benchmarkMap = null, BaseMetric<(string, BenchmarkData), XValue> xMetric = null, XArrangement xArrangement = null,
    NameSimplifier configNameSimplifier = null, bool includeRunName = false,
    bool display = true, bool debug = false)
    => ChartBenchmarks(dataManager, ML(metric),
        runFilter: runFilter, configFilter: configFilter, benchmarkFilter: benchmarkFilter, iterationFilter: iterationFilter,
        configIterationFilter: configIterationFilter, dataFilter: dataFilter,
        benchmarkMap: benchmarkMap, xMetric: xMetric, xArrangement: xArrangement,
        configNameSimplifier: configNameSimplifier, includeRunName: includeRunName,
        display: display, debug: debug);

List<PlotlyChart> ChartIterations(DataManager dataManager, List<Metric<IterationData>> metrics,
    Filter runFilter = null, Filter configFilter = null, Filter benchmarkFilter = null, IntFilter iterationFilter = null,
    ConfigIterationFilter configIterationFilter = null, Func<IterationData, bool> dataFilter = null,
    Func<string, string> benchmarkMap = null, BaseMetric<(string, IterationData), XValue> xMetric = null, XArrangement xArrangement = null,
    NameSimplifier configNameSimplifier = null, bool includeRunName = false,
    bool display = true, bool debug = false)
    => ChartInternal(new ChartPresenter(scatterMode: "markers"), new IterationsChartType(),
        dataManager, metrics,
        runFilter: runFilter, configFilter: configFilter, benchmarkFilter: benchmarkFilter, iterationFilter: iterationFilter,
        configIterationFilter: configIterationFilter, dataFilter: dataFilter,
        benchmarkMap: benchmarkMap, xMetric: xMetric, xArrangement: xArrangement,
        configNameSimplifier: configNameSimplifier, includeRunName: includeRunName,
        display: display, debug: debug);

List<PlotlyChart> ChartIterations(DataManager dataManager, Metric<IterationData> metric,
    Filter runFilter = null, Filter configFilter = null, Filter benchmarkFilter = null, IntFilter iterationFilter = null,
    ConfigIterationFilter configIterationFilter = null, Func<IterationData, bool> dataFilter = null,
    Func<string, string> benchmarkMap = null, BaseMetric<(string, IterationData), XValue> xMetric = null, XArrangement xArrangement = null,
    NameSimplifier configNameSimplifier = null, bool includeRunName = false,
    bool display = true, bool debug = false)
    => ChartIterations(dataManager, ML(metric),
        runFilter: runFilter, configFilter: configFilter, benchmarkFilter: benchmarkFilter, iterationFilter: iterationFilter,
        configIterationFilter: configIterationFilter, dataFilter: dataFilter,
        benchmarkMap: benchmarkMap, xMetric: xMetric, xArrangement: xArrangement,
        configNameSimplifier: configNameSimplifier, includeRunName: includeRunName,
        display: display, debug: debug);

List<PlotlyChart> ChartGCData(DataManager dataManager, List<Metric<TraceGC>> metrics,
    Filter runFilter = null, Filter configFilter = null, Filter benchmarkFilter = null, IntFilter iterationFilter = null,
    ConfigIterationFilter configIterationFilter = null, Func<TraceGC, bool> dataFilter = null,
    Func<string, string> benchmarkMap = null, BaseMetric<(string, TraceGC), XValue> xMetric = null, XArrangement xArrangement = null,
    NameSimplifier configNameSimplifier = null, bool includeRunName = false,
    bool display = true, bool debug = false)
    => ChartInternal(new ChartPresenter(scatterMode: null), new TraceGCChartType(),
        dataManager, metrics,
        runFilter: runFilter, configFilter: configFilter, benchmarkFilter: benchmarkFilter, iterationFilter: iterationFilter,
        configIterationFilter: configIterationFilter, dataFilter: dataFilter,
        benchmarkMap: benchmarkMap, xMetric: xMetric, xArrangement: xArrangement,
        configNameSimplifier: configNameSimplifier, includeRunName: includeRunName,
        display: display, debug: debug);

List<PlotlyChart> ChartGCData(DataManager dataManager, Metric<TraceGC> metric,
    Filter runFilter = null, Filter configFilter = null, Filter benchmarkFilter = null, IntFilter iterationFilter = null,
    ConfigIterationFilter configIterationFilter = null, Func<TraceGC, bool> dataFilter = null,
    Func<string, string> benchmarkMap = null, BaseMetric<(string, TraceGC), XValue> xMetric = null, XArrangement xArrangement = null,
    NameSimplifier configNameSimplifier = null, bool includeRunName = false,
    bool display = true, bool debug = false)
    => ChartGCData(dataManager, ML(metric),
        runFilter: runFilter, configFilter: configFilter, benchmarkFilter: benchmarkFilter, iterationFilter: iterationFilter,
        configIterationFilter: configIterationFilter, dataFilter: dataFilter,
        benchmarkMap: benchmarkMap, xMetric: xMetric, xArrangement: xArrangement,
        configNameSimplifier: configNameSimplifier, includeRunName: includeRunName,
        display: display, debug: debug);

In [ ]:
// Benchmark lists

// scoutList is a list of ASP.NET benchmarks identified by looking at allocation rates.
// scoutList2 adds some tests that Maoni identified.
// smallList is for very quick looks.

// Often a test infra run will have been limited to a smaller set of tests when desired,
// in which case these aren't necessary.  However, these predefined lists can be used to
// help load (or chart after loading) a subset of a run when desired.

List<string> scoutList = ML(
    "ConnectionClose",
    "ConnectionCloseHttps",
    "ConnectionCloseHttpsHttpSys",
    "ConnectionCloseHttpSys",
    "Fortunes",
    "FortunesDapper",
    "FortunesEf",
    "FortunesPlatform",
    "FortunesPlatformDapper",
    "FortunesPlatformEF",
    "Json",
    "JsonHttps",
    "JsonHttpsHttpSys",
    "JsonMin",
    "JsonMvc",
    "MultipleQueriesPlatform",
    "PlaintextMvc",
    "PlaintextQueryString",
    "PlaintextWithParametersEmptyFilter",
    "PlaintextWithParametersNoFilter",
    "SingleQueryPlatform",
    "Stage1",
    "Stage1Grpc",
    "Stage2",
    "UpdatesPlatform"
);

List<string> scoutList2 = scoutList.Concat(ML("CachingPlatform", "JsonMapAction", "Stage1TrimR2RSingleFile")).ToList();
List<string> smallList = ML("Fortunes", "JsonHttpsHttpSys", "PlaintextQueryString", "Stage2", "PlaintextMvc");

# Examples

In [ ]:
string rootDir = @"C:\home\repro\decommit";
var hugeDM = new DataManager();
foreach (string config in ML("rcbase", "rc1", "alt-draft-log3"))
{
    string configDir = $"gcperfsim_{config}_gc";
    foreach (string benchBase in ML("100mb", "100mb-10mb", "10mb-100mb"))
    {
        foreach (string benchSuffix in ML("", "-low"))
        {
            string bench = benchBase + benchSuffix;
            string etl = $@"{rootDir}\{configDir}\{bench}\{bench}.{config}.0.etl";
            if (!File.Exists(etl))
            {
                Console.WriteLine($"{etl} does not exist");
                continue;
            }
            hugeDM.AddGCTrace(etl, ML("corerun"), config: config, loadMultipleProcesses: false);
        }
    }
}

In [ ]:
hugeDM

In [ ]:
string sds_process_name = "Microsoft.M365.Core.Sds.Service";
string store_worker_process_name = "Microsoft.Exchange.Store.Worker";
var store_work_process_list = ML(sds_process_name, store_worker_process_name);
var sdsDM = DataManager.CreateGCTraces(@"c:\home\repro\notebook-gcapi", store_work_process_list);

(sdsDM.Data.Runs.First().Value.Configs.First().Value.Benchmarks.Skip(1).First().Value.Iterations[0].GCProcessData.ProcessID,
sdsDM.Data.Runs.First().Value.Configs.First().Value.Benchmarks.Skip(1).First().Value.Iterations[0].GCProcessData.CommandLine)

In [ ]:
var diffDataManager = DataManager.CreateAspNetData(ML(
    @"C:\home\repro\hc\asp_traceplus3_gc",
    @"C:\home\repro\hc\asp_tp3-m4_gc",
    @"C:\home\repro\hc\asp_slope_gc",
    @"C:\home\repro\hc\asp_evaldecr_gc"
));

In [ ]:
var cardsDM = DataManager.CreateGCTraces(@"c:\home\repro\2401310010004275", pertinentProcesses: ML("EXCEL"));

In [ ]:
var low4DM = DataManager.CreateAspNetData(@"c:\home\repro\hc\asp_v2-fixrearranged-mult-max_gc"
    //, benchmarkFilter: Filter.RE("Stage.*|Json.*");
    //, iterationFilter: IntFilter.Values(1)
);

(low4DM.Data.Runs["asp_v2-fixrearranged-mult-max_gc"].Configs.Keys,
 low4DM.Data.Runs["asp_v2-fixrearranged-mult-max_gc"].Configs["v2-fixrearranged-mult-max-h4"].Benchmarks["Stage2"].Iterations[1].GCSummaryInfo.MaxHeapCount,
 low4DM.Data.Runs["asp_v2-fixrearranged-mult-max_gc"].Configs["v2-fixrearranged-mult-max"].Benchmarks["Stage2"].Iterations[1].GCSummaryInfo.MaxHeapCount,
 low4DM.GetConfigs(Filter.All, Filter.RE("max")))


In [ ]:
var rc3DataManager = DataManager.CreateAspNetData(@"C:\home\repro\hc\asp_v2-fixrearranged_gc");
rc3DataManager.AddAspNetData(@"C:\home\repro\hc\asp_v2-tune_gc");

rc3DataManager.GetConfigs(Filter.All, Filter.All)

In [ ]:
var net6dm = DataManager.CreateGCTrace(@"C:\home\repro\2046032\Microsoft.MWC.Workload.OneLake.Service.EntryPoint.exe_WithGCon.nettrace", ML("Microsoft.MWC.Workload.OneLake.Service.EntryPoint"));
net6dm.AddGCTrace(@"C:\home\repro\2046032\TESTenvCST350 with perf.nettrace", ML("Microsoft.MWC.Workload.OneLake.Service.EntryPoint"));

Charting examples

In [ ]:
ChartBenchmarks(sdsDM, ML(Metrics.B.AveragePercentPauseTimeInGC, Metrics.B.MaxHeapCount)
    //, benchmarkFilter: Filter.Names("JsonMapAction")
    , configNameSimplifier: NameSimplifier.PrefixDashed
    );

In [ ]:
TableBenchmarks(low4DM, ML(Metrics.B.AveragePercentPauseTimeInGC, Metrics.B.MaxHeapCount)
    //, benchmarkFilter: Filter.Names("JsonMapAction")
    , configNameSimplifier: NameSimplifier.PrefixDashed
    );

In [ ]:
hugeDM.Data.Runs.First().Value.Configs.First().Value.Benchmarks.First().Value.Iterations[0].GCProcessData.GCs

In [ ]:
ChartGCData(hugeDM, ML(Metrics.G.CommittedBeforeInUse, Metrics.G.CommittedBeforeInFree, Metrics.G.CommittedBeforeInGlobalFree, Metrics.G.CommittedBeforeInGlobalDecommit)
  , configNameSimplifier: NameSimplifier.PrefixDashed
  , debug: true
  );

In [ ]:
foreach (var xarr in new[] { (XArrangement) XArrangements.Default, XArrangements.Sorted, XArrangements.CombinedSorted })
{
TableBenchmarks(low4DM,
    ML(Metrics.B.MaxHeapCount,
        Metrics.Promote(Metrics.I.MaxHeapCount, Aggregation.Min),
        Metrics.Promote(Metrics.I.MaxHeapCount, Aggregation.Range),
        Metrics.Promote(Metrics.I.NumberOfHeapCountSwitches, Aggregation.Range),
        Metrics.Promote(Metrics.I.NumberOfHeapCountDirectionChanges, Aggregation.Range)),
    configNameSimplifier: NameSimplifier.PrefixDashed,
    xArrangement: xarr,
    configFilter: new Filter(excludeRE: "h4")
    //, debug: true
    );
}

In [ ]:
foreach (var xarr in new[] { (XArrangement) XArrangements.Default, XArrangements.Sorted, XArrangements.CombinedSorted })
{
TableBenchmarks(diffDataManager,
    ML((Metrics.B.MaxHeapCount),
        Metrics.Promote(Metrics.I.MaxHeapCount, Aggregation.Min),
        Metrics.Promote(Metrics.I.MaxHeapCount, Aggregation.Range),
        Metrics.Promote(Metrics.I.NumberOfHeapCountSwitches, Aggregation.Range),
        Metrics.Promote(Metrics.I.NumberOfHeapCountDirectionChanges, Aggregation.Range)),
    //configNameSimplifier: NameSimplifier.PrefixDashed,
    xArrangement: xarr);
}

In [ ]:
TableIterations(diffDataManager, Metrics.I.MaxHeapCount
    , configFilter: Filter.Names("traceplus3", "tp3-m4")
);

In [ ]:
foreach (var lat in Metrics.I.LatencyMSList)
{
    TableBenchmarks(diffDataManager, Metrics.Promote(lat, Aggregation.Average)
        // , benchmarkFilter: Filter.RE("Stage.*")
        // configNameSimplifier: NameSimplifier.PrefixDashed,
        // types: B_XType.All,
        //configFilter: new Filter(excludeRE: ".*h4")
        );
}

In [ ]:
ChartIterations(diffDataManager, ML(/*Metrics.I.GCScore,*/ Metrics.I.RequestsPerMSec));

In [ ]:
TableGCData(net6dm,
    ML(
        Metrics.G.HeapSizeAfter,
        Metrics.G.HeapSizeBefore,
        new Metric<TraceGC>(gc => gc.HeapStats.GCHandleCount, "GC Handles", "#"),
        Metrics.G.GenLargeSizeBefore,
        Metrics.G.GenLargeSizeAfter,
        Metrics.G.GenLargeObjSizeAfter,
        new Metric<TraceGC>(gc => gc.HeapStats.TotalHeapSize / 1000000, "Total heap size", "MB")
    )
    , textPresenter: TextPresenter.Html
    , configNameSimplifier: new ListSimplifier(("Microsoft.MWC.Workload.OneLake.Service.EntryPoint.exe_WithGCon", "Entry"), ("TESTenvCST350 with perf", "TES"))
    , dataFilter: gc => gc.Number > 300 && gc.Number < 310
    );

In [ ]:
ChartGCData(low4DM
    , metrics: ML(Metrics.G.PauseDuration)
    , benchmarkFilter: Filter.Names("Stage2")
    , configNameSimplifier: NameSimplifier.PrefixDashed
    , xMetric: Metrics.X.StartRelativeMSec
    , dataFilter: gc => gc.StartRelativeMSec > 2000
    , iterationFilter: IntFilter.Values(0)
    , xArrangement: XArrangements.Relative
);

In [ ]:
ChartGCData(low4DM
    , metrics: ML(Metrics.G.HctMtcp, Metrics.G.NumHeaps)
    , benchmarkFilter: Filter.RE("Stage2$")
    , configNameSimplifier: NameSimplifier.PrefixDashed
    , iterationFilter: IntFilter.Values(0)
);

In [ ]:
ChartGCData(low4DM, Metrics.G.NumHeaps, configNameSimplifier: NameSimplifier.PrefixDashed, debug: false);

In [ ]:
var low4CompRuns = ML(("v2-fixrearranged-mult-max", "base"), ("v2-fixrearranged-mult-max-h4", "max4"),
    ("v2-fixrearranged-mult-max-svr", "svr"), ("v2-fixrearranged-mult-max-svr4", "svr4"),
    ("v2-fixrearranged-mult-max-mult8", "mult8"), ("v2-fixrearranged-mult-max-mult32", "mult32"),
    ("v2-fixrearranged-mult-max-mult8x10", "m8x10"), ("v2-fixrearranged-mult-max-mult32x10", "m32x10"),
    ("v2-fixrearranged-mult-max-x10", "x10"));

ChartGCData(low4DM, Metrics.G.HctMtcp
    , configFilter: Filter.ExcludeRE("svr")
    , configNameSimplifier: new ListSimplifier(low4CompRuns)
    );

In [ ]:
ChartGCData(
    sdsDM
    , metrics: ML(Metrics.G.AllocRateMBSec, Metrics.G.PauseDuration)
    //, benchmarkFilter: Filter.RE("Run32")
    , configNameSimplifier: NameSimplifier.PrefixDashed
);

In [ ]:
ChartGCData(
    cardsDM
    , metrics: ML(Metrics.G.AllocRateMBSec, Metrics.G.PauseDuration)
    , configFilter: Filter.RE("Run32")
);

In [ ]:
ChartGCData(
    cardsDM,
    metrics: ML(Metrics.G.PauseDuration.WithCap(100), Metrics.G.PauseStack.WithCap(100), Metrics.G.PauseFQ, Metrics.G.PauseHandles.WithCap(100), Metrics.G.PauseCards, Metrics.G.Suspend,
        new Metric<TraceGC>(gc => gc.HeapStats.GCHandleCount, "GC Handles", "#"),
        new Metric<TraceGC>(gc => gc.HeapStats.FinalizationPromotedCount, "F promoted", "#"))
    //, dataFilter: gc => gc.Generation == 0
    , configFilter: Filter.RE("Only")
    , xMetric: Metrics.X.GCIndex
    , configNameSimplifier: new ListSimplifier(("2401310010004275", "a"))
    );

In [ ]:
ChartGCData(rc3DataManager, Metrics.G.HeapSizeBefore, benchmarkFilter: Filter.RE("Stage2$"));

In [ ]:
ChartGCData(rc3DataManager, Metrics.G.NumHeaps
    , benchmarkFilter: Filter.Names("Fortunes", "FortunesDapper", "JsonHttpsHttpSys", "PlaintextQueryString", "Stage1", "Stage2", "PlaintextMvc")
    , benchmarkMap: x => (x == "Stage1" || x == "Stage2" ? "S1/2" : x));

In [ ]:
var rc3RearrNoBaseRuns = ML(("v2-rc3", "rc3"), ("v2-fixrearranged", "rc3rearr"), ("v2-tune", "rc3tune"));

ChartGCData(rc3DataManager, Metrics.G.NumHeaps
    , configNameSimplifier: new ListSimplifier(rc3RearrNoBaseRuns)
    , benchmarkFilter: new Filter(includeNames: scoutList2)
);

## Obsolete stuff - for temporary reference

In [ ]:
// Old comparison/summary code (commented out)
    /*
    public LoadInfo GetComparison(LoadInfo baseline, LoadInfo comparand)
    {
        return new LoadInfo
        {
            MaxWorkingSetMB    = DeltaPercent(baseline.MaxWorkingSetMB, comparand.MaxWorkingSetMB),
            P99WorkingSetMB = DeltaPercent(baseline.P99WorkingSetMB, comparand.P99WorkingSetMB),
            P95WorkingSetMB    = DeltaPercent(baseline.P95WorkingSetMB, comparand.P95WorkingSetMB),
            P90WorkingSetMB    = DeltaPercent(baseline.P90WorkingSetMB, comparand.P90WorkingSetMB),
            P75WorkingSetMB    = DeltaPercent(baseline.P75WorkingSetMB, comparand.P75WorkingSetMB),
            P50WorkingSetMB    = DeltaPercent(baseline.P50WorkingSetMB, comparand.P50WorkingSetMB),

            MaxPrivateMemoryMB = DeltaPercent(baseline.MaxPrivateMemoryMB, comparand.MaxPrivateMemoryMB),
            P99PrivateMemoryMB = DeltaPercent(baseline.P99PrivateMemoryMB, comparand.P99PrivateMemoryMB),
            P95PrivateMemoryMB    = DeltaPercent(baseline.P95PrivateMemoryMB, comparand.P95PrivateMemoryMB),
            P90PrivateMemoryMB    = DeltaPercent(baseline.P90PrivateMemoryMB, comparand.P90PrivateMemoryMB),
            P75PrivateMemoryMB    = DeltaPercent(baseline.P75PrivateMemoryMB, comparand.P75PrivateMemoryMB),
            P50PrivateMemoryMB    = DeltaPercent(baseline.P50PrivateMemoryMB, comparand.P50PrivateMemoryMB),
            
            Latency50thMS   = DeltaPercent(baseline.Latency50thMS, comparand.Latency50thMS),
            Latency75thMS   = DeltaPercent(baseline.Latency75thMS, comparand.Latency75thMS),
            Latency90thMS   = DeltaPercent(baseline.Latency90thMS, comparand.Latency90thMS), 
            Latency99thMS   = DeltaPercent(baseline.Latency99thMS, comparand.Latency99thMS),  
            MeanLatencyMS   = DeltaPercent(baseline.MeanLatencyMS, comparand.MeanLatencyMS),
            RequestsPerMSec = DeltaPercent(baseline.RequestsPerMSec, comparand.RequestsPerMSec),
            TotalSuspensionTimeMSec = DeltaPercent(baseline.TotalSuspensionTimeMSec, comparand.TotalSuspensionTimeMSec),
            PercentPauseTimeInGC = DeltaPercent(baseline.PercentPauseTimeInGC, comparand.PercentPauseTimeInGC),
            PercentTimeInGC = DeltaPercent(baseline.PercentTimeInGC, comparand.PercentTimeInGC),
            MeanHeapSizeBeforeMB = DeltaPercent(baseline.MeanHeapSizeBeforeMB, comparand.MeanHeapSizeBeforeMB),
            MaxHeapSizeMB = DeltaPercent(baseline.MaxHeapSizeMB, comparand.MaxHeapSizeMB),
            TotalAllocationsMB = DeltaPercent(baseline.TotalAllocationsMB, comparand.TotalAllocationsMB),
            GCScore         = DeltaPercent(baseline.GCScore, comparand.GCScore),
            MaxHeapCount = DeltaPercent(baseline.MaxHeapCount, comparand.MaxHeapCount),
            NumberOfHeapCountSwitches = DeltaPercent(baseline.NumberOfHeapCountSwitches, comparand.NumberOfHeapCountSwitches),
            NumberOfHeapCountDirectionChanges = DeltaPercent(baseline.NumberOfHeapCountDirectionChanges, comparand.NumberOfHeapCountDirectionChanges),
            Data = baseline.Data,
            Data2 = comparand.Data,
            Run = $"{baseline.Run} vs. {comparand.Run}",
            Benchmark = baseline.Benchmark,
            Id = $"{baseline.Run} vs. {comparand.Run} for {baseline.Benchmark}"
        };
    }

    public Dictionary<string, LoadInfo>? GetAllBenchmarksForRun(string run)
    {
        if (!_runToBenchmarkData.TryGetValue(run, out var benchmarksForRun))
        {
            Console.WriteLine($"No benchmarks found for run: {run}");
            return null;
        }

        return benchmarksForRun;
    }

    public void SaveBenchmarkData(string outputPath = "")
    {
        if (string.IsNullOrEmpty(outputPath))
        {
            outputPath = _basePath;
        }

        StringBuilder sb = new();
        sb.AppendLine($"Run,Benchmark,Max Working Set (MB), Max Private Memory (MB), Request/MSec, Mean Latency (MSec), Latency 50th Percentile MSec, Latency 75th Percentile MSec, Latency 90th Percentile MSec, Latency 99th Percentile MSec");
        foreach (var b in _data)
        {
            var val = b.Value; 
            sb.AppendLine($"{val.Run},{val.Benchmark},{val.MaxWorkingSetMB},{val.MaxPrivateMemoryMB},{val.RequestsPerMSec},{val.MeanLatencyMS},{val.Latency50thMS},{val.Latency75thMS},{val.Latency90thMS},{val.Latency99thMS}");
        }

        File.WriteAllText(Path.Combine(outputPath, "AllBenchmarks.csv"), sb.ToString());
    }

    public Dictionary<string, LoadInfo>? GetAllRunsForBenchmark(string benchmark)
    {
        if (!_benchmarkToRunData.TryGetValue(benchmark, out var runsForBenchmark))
        {
            Console.WriteLine($"No runs found for benchmark: {benchmark}");
            return null;
        }

        return runsForBenchmark;
    }
*/

/*
    public LoadInfo? GetBenchmarkData(string benchmark, string run)
    {
        if (!_benchmarkToRunData.TryGetValue(benchmark, out var runData))
        {
            Console.WriteLine($"Benchmark: {benchmark} not found!");
            return null;
        }

        if (!runData.TryGetValue(run, out var loadInfo))
        {
            Console.WriteLine($"Run: {run} not found!");
            return null;
        }

        return loadInfo;
    }
*/

    /*
    public Dictionary<string, LoadInfo> GetBenchmarkToComparison(string baselineRun, string comparandRun)
    {
        Dictionary<string, LoadInfo> comparisons = new();

        Dictionary<string, LoadInfo> baselineData = new();
        Dictionary<string, LoadInfo> comparandData = new();
        HashSet<string> allBenchmarks = new();

        foreach (var d in _data)
        {
            allBenchmarks.Add(d.Value.Benchmark);

            string run = d.Key.Split("|", StringSplitOptions.RemoveEmptyEntries | StringSplitOptions.TrimEntries)[0];

            if (string.CompareOrdinal(run, baselineRun) == 0 && !baselineData.TryGetValue(d.Key, out var baselineInfo))
            {
                baselineInfo = baselineData[d.Value.Benchmark] = d.Value;
            }

            else if (string.CompareOrdinal(run, comparandRun) == 0 && !comparandData.TryGetValue(d.Key, out var comparandInfo))
            {
                comparandInfo = comparandData[d.Value.Benchmark] = d.Value;
            }
        }

        foreach (var benchmark in allBenchmarks)
        {
            if (!baselineData.TryGetValue(benchmark, out var baselineBenchmarkInfo))
            {
                Console.WriteLine($"Benchmark: {benchmark} not found on the baseline: {baselineRun}");
                continue;
            }

            if (!comparandData.TryGetValue(benchmark, out var comparandBenchmarkInfo))
            {
                Console.WriteLine($"Benchmark: {benchmark} not found on the comparand: {comparandRun}");
                continue;
            }

            LoadInfo comparison = GetComparison(baselineBenchmarkInfo, comparandBenchmarkInfo);
            comparisons[benchmark] = comparison;
        }
        
        return comparisons;
    }
    */

    // Haven't used this in a while - writes a summary file to disk

/*
    public void SummarizeResults(DataManager dataManager, string outFile, Dictionary<string, LoadInfo> info = null)
    {
        if (info == null)
        {
            info = dataManager._data;
        }

        using (StreamWriter sw = new StreamWriter(outFile))
    {
        sw.WriteLine("{0,12} | {1,35} | {2, 5} | {3, 5:0.00} | {4, 5} | {5, 5:0.00} | {6, 5} | {7, 5:0.00} | {8, 5} | {9, 5:0.00} | {10, 10:0.00} | {11, 10:0.00} | {12, 5:0.00} | {13, 10:0.00} | {14, 10:0.00} |", 
                        "run", "benchmark", "gen0", "pause", "gen1", "pause", "ngc2", "pause", "bgc", "pause", "allocMB", "alloc/gc", "pct", "peakMB", "meanMB");
        sw.WriteLine("{0,12} | {1,35} | {2, 5} | {3, 5:0.00} | {4, 5} | {5, 5:0.00} | {6, 5} | {7, 5:0.00} | {8, 5} | {9, 5:0.00} | {10, 10:0.00} | {11, 10:0.00} | {12, 5:0.00} | {13, 10:0.00} | {14, 10:0.00} |", 
                        "", "", "", "susp", "", "susp", "", "susp", "", "susp", "", "", "", "totalcpu", "meancpu");
        sw.WriteLine("{0}", new String('-', 174));
        foreach (var kvp in info)
        {
            List<TraceGC> gcs = kvp.Value?.Data?.GCs;
            if (gcs == null || gcs.Count == 0)
            {
                continue;
            }

            int[] gc_counts = new int[4];
            double[] gc_pauses = new double[4];
            double[] gc_susps = new double[4];
            for (int i = 0; i < gcs.Count; i++)
            {
                TraceGC gc = gcs[i];
                //if (gc.SuspendDurationMSec > 5) sw.WriteLine($"i={gc.Number} gen={gc.Generation} suspension={gc.SuspendDurationMSec} totalpause={gc.PauseDurationMSec}");
                if (gc.Generation < 2)
                {
                    gc_counts[gc.Generation]++;
                    gc_pauses[gc.Generation] += gc.PauseDurationMSec;
                    gc_susps[gc.Generation] += gc.SuspendDurationMSec;
                }
                else
                {
                    if (gc.Type == GCType.BackgroundGC)
                    {
                        gc_counts[3]++;
                        gc_pauses[3] += gc.PauseDurationMSec;
                        gc_susps[3] += gc.SuspendDurationMSec;
                    }
                    else
                    {
                        gc_counts[2]++;
                        gc_pauses[2] += gc.PauseDurationMSec;
                        gc_susps[2] += gc.SuspendDurationMSec;
                    }
                }
            }
            
            for (int i = 0; i < 4; i++)
            {
                if (gc_counts[i] > 0)
                {
                    gc_pauses[i] /= gc_counts[i];
                    gc_susps[i] /= gc_counts[i];
                }
            }
            
            sw.WriteLine("{0,12} | {1,35} | {2, 5} | {3, 5:0.00} | {4, 5} | {5, 5:0.00} | {6, 5} | {7, 5:0.00} | {8, 5} | {9, 5:0.00} | {10, 10:0.00} | {11, 10:0.00} | {12, 5:0.00} | {13, 10:0.00} | {14, 10:0.00} |",
                kvp.Value.Run, kvp.Value.Benchmark, gc_counts[0], gc_pauses[0], gc_counts[1], gc_pauses[1], gc_counts[2], gc_pauses[2], gc_counts[3], gc_pauses[3],
                kvp.Value.Data.Stats.TotalAllocatedMB, (kvp.Value.Data.Stats.TotalAllocatedMB / gcs.Count), kvp.Value.Data.Stats.GetGCPauseTimePercentage(), kvp.Value.Data.Stats.MaxSizePeakMB, kvp.Value.Data.Stats.MeanSizePeakMB);
            sw.WriteLine("{0,12} | {1,35} | {2, 5} | {3, 5:0.00} | {4, 5} | {5, 5:0.00} | {6, 5} | {7, 5:0.00} | {8, 5} | {9, 5:0.00} | {10, 10:0.00} | {11, 10:0.00} | {12, 5:0.00} | {13, 10:0.00} | {14, 10:0.00} |",
                "", "", "", gc_susps[0], "", gc_susps[1], "", gc_susps[2], "", gc_susps[3],
                "", "", "", kvp.Value.Data.Stats.TotalCpuMSec, kvp.Value.Data.Stats.MeanCpuMSec);
        }
    }}

class MeanDataComparison
{
    public string bench { get; set; }
    public double baselineMaxPrivateMemoryMB { get; set; }
    public double baselineP50PrivateMemoryMB { get; set; }
    public double baselineRequestsPerMSec { get; set; }
    public double avgMaxPrivateMemoryMBDiff { get; set; }
    public double avgP50PrivateMemoryMBDiff { get; set; }
    public double avgRequestsPerMSecDiff { get; set; }
    public double baselineCVMaxPrivateMemoryMB { get; set; }
    public double baselineCVP50PrivateMemoryMB { get; set; }
    public double baselineCVRequestsPerMSec { get; set; }
    public double fixCVMaxPrivateMemoryMB { get; set; }
    public double fixCVP50PrivateMemoryMB { get; set; }
    public double fixCVRequestsPerMSec { get; set; }
    public double cvMaxPrivateMemoryMBDiff { get; set; }
    public double cvP50PrivateMemoryMBDiff { get; set; }
    public double cvRequestsPerMSecDiff { get; set; }
}

double GetCV(List<double> dataPoints, out double avg)
{
    // for (int i = 0; i < dataPoints.Count; i++)
    // {
    //     Console.WriteLine("item {0}: {1}", i, dataPoints[i]);
    // }
    double mean = dataPoints.Average();
    avg = mean;
    double sumOfSquaredDifferences = dataPoints.Sum(val => Math.Pow(val - mean, 2));
    double populationStandardDeviation = Math.Sqrt(sumOfSquaredDifferences / dataPoints.Count);
    double coefficientOfVariation = (populationStandardDeviation / mean) * 100;
    return coefficientOfVariation;
}

// accommodates when there are different numbers of iterations in first and second run.
// returns a list of benchmarks we added to the comparison data
List<MeanDataComparison> SummarizeResultsByBench(DataManager dataManager, List<string> runNames, string benchName = null)
{
    Dictionary<string, Dictionary<string, LoadInfo>> benchmarkToRunData = dataManager._benchmarkToRunData;
    Console.WriteLine("benchmarkToRunData has {0} tests\n", benchmarkToRunData.Count);

    //bool fLogDetail = false;
    bool fLogDetail = true;

    string strSeparator = new String('-', 223);
    Console.WriteLine("{0}", strSeparator);

    // key is the name of the run, eg, "baseline" or "fix". For each run, we add its summary data to a list.
    Dictionary<string, List<BenchmarkSummaryData>> summaryDataForRuns = new Dictionary<string, List<BenchmarkSummaryData>>(2);
    List<MeanDataComparison> comparisonData = new List<MeanDataComparison>(51);

    foreach (var benchmarkData in benchmarkToRunData)
    {
        // // Console.WriteLine("benchmark is {0}", benchmarkData.Key);

        if ((benchName == null) || benchmarkData.Key.Equals(benchName, StringComparison.OrdinalIgnoreCase))
        {
            summaryDataForRuns.Clear();

            if (fLogDetail)
            {
                Console.WriteLine("{0,25} | {1,35} | {2, 5} | {3, 5:0.00} | {4, 5} | {5, 5:0.00} | {6, 5} | {7, 5:0.00} | {8, 5} | {9, 5:0.00} | {10, 10:0.00} | {11, 10:0.00} | {12, 5:0.00} | {13, 8:0.00} | {14, 8:0.00} | {15, 8:0.00} | {16, 8:0.00} | {17, 8:0.00} | {18, 4:0.00} | {19, 10} |",
                                "run", "benchmark", "gen0", "pause", "gen1", "pause", "ngc2", "pause", "bgc", "pause", "allocMB", "alloc/gc", "pct", "peakMB", "meanMB", "max mem", "rps", "latency", "hc", "gc count");
                Console.WriteLine("{0}", strSeparator);                                
            }

            // if no runs observed an hc change, we don't keep it in the summary data.
            int totalHCChanges = 0;

            foreach (var kvp in benchmarkData.Value)
            {
                List<TraceGC> gcs = kvp.Value?.Data?.GCs;
                // We don't look at benchmarks that did very few GCs
                if ((gcs == null) || (gcs.Count == 0))
                {
                    continue;
                }

                int[] gc_counts = new int[4];
                double[] gc_pauses = new double[4];
                for (int i = 0; i < gcs.Count; i++)
                {
                    TraceGC gc = gcs[i];
                    if (gc.Generation < 2)
                    {
                        gc_counts[gc.Generation]++;
                        gc_pauses[gc.Generation] += gc.PauseDurationMSec;
                    }
                    else
                    {
                        if (gc.Type == GCType.BackgroundGC)
                        {
                            gc_counts[3]++;
                            gc_pauses[3] += gc.PauseDurationMSec;
                        }
                        else
                        {
                            gc_counts[2]++;
                            gc_pauses[2] += gc.PauseDurationMSec;
                        }
                    }
                }

                for (int i = 0; i < 4; i++)
                {
                    if (gc_counts[i] > 0)
                    {
                        gc_pauses[i] /= gc_counts[i];
                    }
                }

                if (fLogDetail)
                {
                    Console.WriteLine("{0,25} | {1,35} | {2, 5} | {3, 5:0.00} | {4, 5} | {5, 5:0.00} | {6, 5} | {7, 5:0.00} | {8, 5} | {9, 5:0.00} | {10, 10:0.00} | {11, 10:0.00} | {12, 5:0.00} | {13, 8:0.00} | {14, 8:0.00} | {15, 8:0.00} | {16, 8:0.00} | {17, 8:0.00} | {18, 4} | {19, 10} |",
                        kvp.Value.Run, kvp.Value.Benchmark, gc_counts[0], gc_pauses[0], gc_counts[1], gc_pauses[1], gc_counts[2], gc_pauses[2], gc_counts[3], gc_pauses[3],
                        kvp.Value.Data.Stats.TotalAllocatedMB, (kvp.Value.Data.Stats.TotalAllocatedMB / gcs.Count), kvp.Value.Data.Stats.GetGCPauseTimePercentage(), kvp.Value.Data.Stats.MaxSizePeakMB, kvp.Value.Data.Stats.MeanSizePeakMB,
                        kvp.Value.MaxPrivateMemoryMB, kvp.Value.RequestsPerMSec, kvp.Value.MeanLatencyMS, kvp.Value.NumberOfHeapCountSwitches, kvp.Value.Data.Stats.Count);
                }

                totalHCChanges += (int)kvp.Value.NumberOfHeapCountSwitches;

                for (int runIdx = 0; runIdx < runNames.Count; runIdx++)
                {
                    if (kvp.Value.Run.StartsWith(runNames[runIdx]))
                    {
                        BenchmarkSummaryData data = new BenchmarkSummaryData 
                        {
                            MaxPrivateMemoryMB = kvp.Value.MaxPrivateMemoryMB,
                            P50PrivateMemoryMB = kvp.Value.P50PrivateMemoryMB,
                            RequestsPerMSec = kvp.Value.RequestsPerMSec,
                        };

                        if (summaryDataForRuns.ContainsKey(runNames[runIdx]))
                        {
                            summaryDataForRuns[runNames[runIdx]].Add(data);
                        }
                        else
                        {
                            List<BenchmarkSummaryData> listData = new List<BenchmarkSummaryData>(3);
                            listData.Add(data);
                            summaryDataForRuns.Add(runNames[runIdx], listData);
                        }
                        break;
                    }
                }
            }

            if (fLogDetail)
            {
                Console.WriteLine("{0}", strSeparator);
            }

            if (totalHCChanges == 0)
            {
                //Console.WriteLine("don't do comparison for bench {0}! no HC changes", benchmarkData.Key);
                continue;
            }

            // Now write some summary stuff
            //Console.WriteLine("summary dictionary has {0} elements", summaryDataForRuns.Count);

            if (fLogDetail)
            {
                Console.WriteLine("{0,45} | {1,10} | {2,10} | {3,10} | {4,10} | {5,10} | {6,10} |", "data", "max mem", "CV%", "p50 mem", "CV%", "rps", "CV%");
            }

            int numRuns = summaryDataForRuns.Count;
            double[] avgMaxPrivateMemoryMBForRuns = new double [numRuns];
            double[] avgP50PrivateMemoryMBForRuns = new double [numRuns];
            double[] avgRequestsPerMSecForRuns = new double [numRuns];
            double[] cvMaxPrivateMemoryMBForRuns = new double [numRuns];
            double[] cvP50PrivateMemoryMBForRuns = new double [numRuns];
            double[] cvRequestsPerMSecForRuns = new double [numRuns];

            for (int i = 0; i < summaryDataForRuns.Count; i++)
            {
                //Console.WriteLine("bench {0} has {1} iteration in run {2}", benchmarkData.Key, summaryDataForRuns.ElementAt(i).Value.Count, summaryDataForRuns.ElementAt(i).Key);
                List<BenchmarkSummaryData> listData = summaryDataForRuns.ElementAt(i).Value;

                // for (int runIdx  = 0; runIdx < listData.Count; runIdx++)
                // {
                //     Console.WriteLine("run {0} iter {1} max mem {2}, rps {3}", summaryDataForRuns.ElementAt(i).Key, runIdx, listData[runIdx].MaxPrivateMemoryMB, listData[runIdx].RequestsPerMSec);
                // }
                List<double> listMaxPrivateMemoryMB = listData.Select(s => s.MaxPrivateMemoryMB).ToList();
                double avgMaxPrivateMemoryMB, avgP50PrivateMemoryMB, avgRequestsPerMSec;
                double cvMaxPrivateMemoryMB = GetCV(listMaxPrivateMemoryMB, out avgMaxPrivateMemoryMB);
                List<double> listP50PrivateMemoryMB = listData.Select(s => s.P50PrivateMemoryMB).ToList();
                double cvP50PrivateMemoryMB = GetCV(listP50PrivateMemoryMB, out avgP50PrivateMemoryMB);
                List<double> listRequestsPerMSec = listData.Select(s => s.RequestsPerMSec).ToList();
                double cvRequestsPerMSec = GetCV(listRequestsPerMSec, out avgRequestsPerMSec);

                avgMaxPrivateMemoryMBForRuns[i] = avgMaxPrivateMemoryMB;
                avgP50PrivateMemoryMBForRuns[i] = avgP50PrivateMemoryMB;
                avgRequestsPerMSecForRuns[i] = avgRequestsPerMSec;
                cvMaxPrivateMemoryMBForRuns[i] = cvMaxPrivateMemoryMB;
                cvP50PrivateMemoryMBForRuns[i] = cvP50PrivateMemoryMB;
                cvRequestsPerMSecForRuns[i] = cvRequestsPerMSec;
                
                if (fLogDetail)
                {
                    Console.WriteLine("{0,45} | {1,10:0.00} | {2,10:0.00} | {3,10:0.00} | {4,10:0.00} | {5,10:0.00} | {6,10:0.00} |", ("-" + benchmarkData.Key + "-" + summaryDataForRuns.ElementAt(i).Key), 
                        avgMaxPrivateMemoryMB, cvMaxPrivateMemoryMB, avgP50PrivateMemoryMB, cvP50PrivateMemoryMB, avgRequestsPerMSec, cvRequestsPerMSec);
                }
            }

            // I'm just assuming we only have 2 runs.
            MeanDataComparison comp = new MeanDataComparison
            {
                bench = benchmarkData.Key,
                baselineMaxPrivateMemoryMB = avgMaxPrivateMemoryMBForRuns[0],
                baselineP50PrivateMemoryMB = avgP50PrivateMemoryMBForRuns[0],
                baselineRequestsPerMSec = avgRequestsPerMSecForRuns[0],
                avgMaxPrivateMemoryMBDiff = (avgMaxPrivateMemoryMBForRuns[1] - avgMaxPrivateMemoryMBForRuns[0]) * 100.0 / avgMaxPrivateMemoryMBForRuns[0],
                avgP50PrivateMemoryMBDiff = (avgP50PrivateMemoryMBForRuns[1] - avgP50PrivateMemoryMBForRuns[0]) * 100.0 / avgP50PrivateMemoryMBForRuns[0],
                avgRequestsPerMSecDiff = (avgRequestsPerMSecForRuns[1] - avgRequestsPerMSecForRuns[0]) * 100.0 / avgRequestsPerMSecForRuns[0],
                baselineCVMaxPrivateMemoryMB = cvMaxPrivateMemoryMBForRuns[0],
                baselineCVP50PrivateMemoryMB = cvP50PrivateMemoryMBForRuns[0],
                baselineCVRequestsPerMSec = cvRequestsPerMSecForRuns[0],
                fixCVMaxPrivateMemoryMB = cvMaxPrivateMemoryMBForRuns[1],
                fixCVP50PrivateMemoryMB = cvP50PrivateMemoryMBForRuns[1],
                fixCVRequestsPerMSec = cvRequestsPerMSecForRuns[1],
                cvMaxPrivateMemoryMBDiff = (cvMaxPrivateMemoryMBForRuns[1] - cvMaxPrivateMemoryMBForRuns[0]) * 100.0 / cvMaxPrivateMemoryMBForRuns[0],
                cvP50PrivateMemoryMBDiff = (cvP50PrivateMemoryMBForRuns[1] - cvP50PrivateMemoryMBForRuns[0]) * 100.0 / cvP50PrivateMemoryMBForRuns[0],
                cvRequestsPerMSecDiff = (cvRequestsPerMSecForRuns[1] - cvRequestsPerMSecForRuns[0]) * 100.0 / cvRequestsPerMSecForRuns[0],                
            };
            comparisonData.Add(comp);

            if (fLogDetail)
            {
                Console.WriteLine("{0}\n", strSeparator);
            }

            if (benchName != null)
            {
                break;
            }
        }
    }

    if (true)
    {
        Console.WriteLine("displaying {0} benches that observed HC changes", comparisonData.Count);

        Console.WriteLine("{0,35} | {1, 9} | {2,9} | {3,9} | {4,9} | {5,9} | {6,9} | {7,9} | {8,9} | {9,9} | {10,9} | {11,9} | {12,9} | {13,9} | {14,9} | {15,9} |",
            "bench", "b max mem", "max mem %", "b cv%", "f cv%", "CV% %", "b p50 mem", "p50 mem %", "b cv%", "f cv%", "CV% %", "b rps", "rps %", "b cv%", "f cv%", "CV% %");

        var sortedComparisonData = comparisonData.OrderByDescending(a => a.cvMaxPrivateMemoryMBDiff).ToList();
        //var sortedComparisonData = comparisonData.OrderBy(a => a.avgMaxPrivateMemoryMBDiff).ToList();
        for (int benchIdx = 0; benchIdx < sortedComparisonData.Count; benchIdx++)
        {
            MeanDataComparison currentComp = sortedComparisonData[benchIdx];
            Console.WriteLine("{0,35} | {1,9:0.00} | {2,9:0.00} | {3,9:0.00} | {4,9:0.00} | {5,9:0.00} | {6,9:0.00} | {7,9:0.00} | {8,9:0.00} | {9,9:0.00} | {10,9:0.00} | {11,9:0.00} | {12,9:0.00} | {13,9:0.00} | {14,9:0.00} | {15,9:0.00} |",
                currentComp.bench,
                currentComp.baselineMaxPrivateMemoryMB, currentComp.avgMaxPrivateMemoryMBDiff, currentComp.baselineCVMaxPrivateMemoryMB, currentComp.fixCVMaxPrivateMemoryMB, currentComp.cvMaxPrivateMemoryMBDiff,
                currentComp.baselineP50PrivateMemoryMB, currentComp.avgP50PrivateMemoryMBDiff, currentComp.baselineCVP50PrivateMemoryMB, currentComp.fixCVP50PrivateMemoryMB, currentComp.cvP50PrivateMemoryMBDiff,
                currentComp.baselineRequestsPerMSec, currentComp.avgRequestsPerMSecDiff, currentComp.baselineCVRequestsPerMSec, currentComp.fixCVRequestsPerMSec, currentComp.cvRequestsPerMSecDiff);
        }
    }

    return comparisonData;
}
*/

// I haven't used this in a while.  I'm not sure if it works.
/*
    public void SaveDifferences(DataManager dataManager, string baseline, string comparand, List<string> sortingCriteria = null)
    {
        // This function assumes the runs are all in:
        // {build}_{iteration} form.
        // Else, it will except.
using (StreamWriter sw = new StreamWriter(@"c:\home\repro\hc\hc-savediff.txt")) {
    sw.WriteLine("start");
        // Iteration -> LoadInfos
        Dictionary<int, List<LoadInfo>> iterationData = new();

        // Get the max iteration.
        int maxIteration = -1;
        foreach (var run in dataManager._runToBenchmarkData)
        {
            string runName = run.Key;
            int iteration = 0;
            if (run.Key.Contains("_"))
            {
                string[] split = run.Key.Split("_");
                Debug.Assert(split.Length == 2);
                string build = split[0];
                string iterationAsString = split[1];
                iteration = Convert.ToInt32(iterationAsString);
            }
            maxIteration = System.Math.Max(iteration, maxIteration);
        }
        sw.WriteLine(maxIteration);
        // Compute Average Diff
        // Build to Benchmark -> Data
        Dictionary<string, Dictionary<string, LoadInfo>> averageData = new();

        for (int i = 0; i <= maxIteration; i++)
        {
            sw.WriteLine(i);
            sw.WriteLine(maxIteration);
            string baselineIteration;
            string comparandIteration;
            if (maxIteration == 0)
            {
                baselineIteration = baseline;
                comparandIteration = comparand;
            }
            else
            {
                baselineIteration = baseline  + "_" + i.ToString();
                comparandIteration = comparand + "_" + i.ToString();
            }
            foreach (var x in dataManager._runToBenchmarkData.Keys) { sw.WriteLine(x); }
            Dictionary<string, LoadInfo> baselineIterationRuns  = dataManager._runToBenchmarkData[baselineIteration];
            Dictionary<string, LoadInfo> comparandIterationRuns = dataManager._runToBenchmarkData[comparandIteration];

            foreach (var b in baselineIterationRuns)
            {
                if (!iterationData.TryGetValue(i, out var benchmarks))
                {
                    iterationData[i] = benchmarks = new();
                }

                benchmarks.Add(dataManager.GetComparison(baselineIterationRuns[b.Key], comparandIterationRuns[b.Key]));
            }

            if (!averageData.TryGetValue(baseline, out var bVal))
            {
                averageData[baseline] = bVal = new();
                foreach (var benchmark in baselineIterationRuns)
                {
                    bVal[benchmark.Key] = new LoadInfo
                    {
                        Benchmark = benchmark.Key,
                        MaxWorkingSetMB    = benchmark.Value.MaxWorkingSetMB,
                        MaxPrivateMemoryMB = benchmark.Value.MaxPrivateMemoryMB,
                        P99PrivateMemoryMB = benchmark.Value.P99PrivateMemoryMB,
                        P95PrivateMemoryMB = benchmark.Value.P95PrivateMemoryMB,
                        P90PrivateMemoryMB = benchmark.Value.P90PrivateMemoryMB,
                        P75PrivateMemoryMB = benchmark.Value.P75PrivateMemoryMB,
                        P50PrivateMemoryMB = benchmark.Value.P50PrivateMemoryMB,
                        RequestsPerMSec = benchmark.Value.RequestsPerMSec,
                        MeanLatencyMS   = benchmark.Value.MeanLatencyMS,
                        Latency50thMS   = benchmark.Value.Latency50thMS, 
                        Latency75thMS   = benchmark.Value.Latency75thMS,
                        Latency90thMS   = benchmark.Value.Latency90thMS,
                        Latency99thMS   = benchmark.Value.Latency99thMS,
                        MaxHeapCount = benchmark.Value.MaxHeapCount,
                        NumberOfHeapCountSwitches = benchmark.Value.NumberOfHeapCountSwitches,
                        NumberOfHeapCountDirectionChanges = benchmark.Value.NumberOfHeapCountDirectionChanges,
                    };
                }
            }

            else
            {
                foreach (var benchmark in baselineIterationRuns)
                {
                    var data = bVal[benchmark.Key];
                    data.Benchmark = benchmark.Key;
                    data.MaxWorkingSetMB    += benchmark.Value.MaxWorkingSetMB;
                    data.MaxPrivateMemoryMB += benchmark.Value.MaxPrivateMemoryMB;
                    data.P99PrivateMemoryMB += benchmark.Value.P99PrivateMemoryMB;
                    data.P95PrivateMemoryMB += benchmark.Value.P95PrivateMemoryMB;
                    data.P90PrivateMemoryMB += benchmark.Value.P90PrivateMemoryMB;
                    data.P75PrivateMemoryMB += benchmark.Value.P75PrivateMemoryMB;
                    data.P50PrivateMemoryMB += benchmark.Value.P50PrivateMemoryMB;
                    data.RequestsPerMSec += benchmark.Value.RequestsPerMSec;
                    data.MeanLatencyMS   += benchmark.Value.MeanLatencyMS;
                    data.Latency50thMS   += benchmark.Value.Latency50thMS; 
                    data.Latency75thMS   += benchmark.Value.Latency75thMS;
                    data.Latency90thMS   += benchmark.Value.Latency90thMS;
                    data.Latency99thMS   += benchmark.Value.Latency99thMS;
                    data.MaxHeapCount += benchmark.Value.MaxHeapCount;
                    data.NumberOfHeapCountSwitches += benchmark.Value.NumberOfHeapCountSwitches;
                    data.NumberOfHeapCountDirectionChanges += benchmark.Value.NumberOfHeapCountDirectionChanges;
                }
            }

            if (!averageData.TryGetValue(comparand, out var cVal))
            {
                averageData[comparand] = cVal = new();
                foreach (var benchmark in comparandIterationRuns)
                {
                    cVal[benchmark.Key] = new LoadInfo
                    {
                        Benchmark = benchmark.Key,
                        MaxWorkingSetMB    = benchmark.Value.MaxWorkingSetMB,
                        MaxPrivateMemoryMB = benchmark.Value.MaxPrivateMemoryMB,
                        P99PrivateMemoryMB = benchmark.Value.P99PrivateMemoryMB,
                        P95PrivateMemoryMB = benchmark.Value.P95PrivateMemoryMB,
                        P90PrivateMemoryMB = benchmark.Value.P90PrivateMemoryMB,
                        P75PrivateMemoryMB = benchmark.Value.P75PrivateMemoryMB,
                        P50PrivateMemoryMB = benchmark.Value.P50PrivateMemoryMB,
                        RequestsPerMSec = benchmark.Value.RequestsPerMSec,
                        MeanLatencyMS   = benchmark.Value.MeanLatencyMS,
                        Latency50thMS   = benchmark.Value.Latency50thMS, 
                        Latency75thMS   = benchmark.Value.Latency75thMS,
                        Latency90thMS   = benchmark.Value.Latency90thMS,
                        Latency99thMS   = benchmark.Value.Latency99thMS,
                        MaxHeapCount    = benchmark.Value.MaxHeapCount,
                        NumberOfHeapCountSwitches = benchmark.Value.NumberOfHeapCountSwitches,
                        NumberOfHeapCountDirectionChanges = benchmark.Value.NumberOfHeapCountDirectionChanges,
                    };
                }
            }

            else
            {
                foreach (var benchmark in comparandIterationRuns)
                {
                    var data = cVal[benchmark.Key];
                    data.Benchmark = benchmark.Key;
                    data.MaxWorkingSetMB    += benchmark.Value.MaxWorkingSetMB;
                    data.MaxPrivateMemoryMB += benchmark.Value.MaxPrivateMemoryMB;
                    data.P99PrivateMemoryMB += benchmark.Value.P99PrivateMemoryMB;
                    data.P95PrivateMemoryMB += benchmark.Value.P95PrivateMemoryMB;
                    data.P90PrivateMemoryMB += benchmark.Value.P90PrivateMemoryMB;
                    data.P75PrivateMemoryMB += benchmark.Value.P75PrivateMemoryMB;
                    data.P50PrivateMemoryMB += benchmark.Value.P50PrivateMemoryMB;
                    data.RequestsPerMSec += benchmark.Value.RequestsPerMSec;
                    data.MeanLatencyMS   += benchmark.Value.MeanLatencyMS;
                    data.Latency50thMS   += benchmark.Value.Latency50thMS; 
                    data.Latency75thMS   += benchmark.Value.Latency75thMS;
                    data.Latency90thMS   += benchmark.Value.Latency90thMS;
                    data.Latency99thMS   += benchmark.Value.Latency99thMS;
                    data.MaxHeapCount    += benchmark.Value.MaxHeapCount;
                    data.NumberOfHeapCountSwitches += benchmark.Value.NumberOfHeapCountSwitches;
                    data.NumberOfHeapCountDirectionChanges += benchmark.Value.NumberOfHeapCountDirectionChanges;
                }
            }
        }

        foreach (var benchmark in dataManager._benchmarkToRunData)
        {
            foreach (var build in averageData)
            {
                var data = build.Value[benchmark.Key];
                data.Benchmark = benchmark.Key;
                data.MaxWorkingSetMB    /= (maxIteration + 1); 
                data.MaxPrivateMemoryMB /=  (maxIteration + 1);
                data.P99PrivateMemoryMB /=  (maxIteration + 1);
                data.P95PrivateMemoryMB /=  (maxIteration + 1);
                data.P90PrivateMemoryMB /=  (maxIteration + 1);
                data.P75PrivateMemoryMB /=  (maxIteration + 1);
                data.P50PrivateMemoryMB /=  (maxIteration + 1);
                data.RequestsPerMSec /=  (maxIteration + 1);
                data.MeanLatencyMS   /= (maxIteration + 1);
                data.Latency50thMS   /= (maxIteration + 1);
                data.Latency75thMS   /= (maxIteration + 1);
                data.Latency90thMS   /= (maxIteration + 1);
                data.Latency99thMS   /= (maxIteration + 1);
                data.MaxHeapCount /= (maxIteration + 1);
                data.NumberOfHeapCountSwitches /= (maxIteration + 1);
                data.NumberOfHeapCountDirectionChanges /= (maxIteration + 1);
            }
        }

        string DisplayDetailsForABenchmark(LoadInfo val) =>
            $"{val.Benchmark},{val.MaxWorkingSetMB},{val.MaxPrivateMemoryMB},{val.RequestsPerMSec},{val.MeanLatencyMS},{val.Latency50thMS},{val.Latency75thMS},{val.Latency90thMS},{val.Latency99thMS},{val.NumberOfHeapCountSwitches},{val.MaxHeapCount}";

        if (sortingCriteria == null)
        {
            sortingCriteria = new() { nameof(LoadInfo.MaxPrivateMemoryMB) };
        }

        foreach (var s in sortingCriteria)
        {
            Func<LoadInfo, double> sortingFunctor = null;
            Func<KeyValuePair<string, LoadInfo>, double> selectionFunctor = null;

            switch (s)
            {
                case nameof(LoadInfo.MaxWorkingSetMB):
                    sortingFunctor = (data) => data.MaxWorkingSetMB;
                    selectionFunctor = (data) => data.Value.MaxWorkingSetMB;
                    break;
                case nameof(LoadInfo.MaxPrivateMemoryMB):
                    sortingFunctor = (data) => data.MaxPrivateMemoryMB;
                    selectionFunctor = (data) => data.Value.MaxPrivateMemoryMB;
                    break;
                case nameof(LoadInfo.P99PrivateMemoryMB):
                    sortingFunctor = (data) => data.P99PrivateMemoryMB;
                    selectionFunctor = (data) => data.Value.P99PrivateMemoryMB;
                    break;
                case nameof(LoadInfo.P95PrivateMemoryMB):
                    sortingFunctor = (data) => data.P95PrivateMemoryMB;
                    selectionFunctor = (data) => data.Value.P95PrivateMemoryMB;
                    break;
                case nameof(LoadInfo.P90PrivateMemoryMB):
                    sortingFunctor = (data) => data.P90PrivateMemoryMB;
                    selectionFunctor = (data) => data.Value.P90PrivateMemoryMB;
                    break;
                case nameof(LoadInfo.P75PrivateMemoryMB):
                    sortingFunctor = (data) => data.P75PrivateMemoryMB;
                    selectionFunctor = (data) => data.Value.P75PrivateMemoryMB;
                    break;
                case nameof(LoadInfo.P50PrivateMemoryMB):
                    sortingFunctor = (data) => data.P50PrivateMemoryMB;
                    selectionFunctor = (data) => data.Value.P50PrivateMemoryMB;
                    break;
                case nameof(LoadInfo.RequestsPerMSec):
                    sortingFunctor = (data) => data.RequestsPerMSec;
                    selectionFunctor = (data) => data.Value.RequestsPerMSec;
                    break;
                case nameof(LoadInfo.MeanLatencyMS):
                    sortingFunctor = (data) => data.MeanLatencyMS;
                    selectionFunctor = (data) => data.Value.MeanLatencyMS;
                    break;
                case nameof(LoadInfo.Latency50thMS):
                    sortingFunctor = (data) => data.Latency50thMS;
                    selectionFunctor = (data) => data.Value.Latency50thMS;
                    break;
                case nameof(LoadInfo.Latency75thMS):
                    sortingFunctor = (data) => data.Latency75thMS;
                    selectionFunctor = (data) => data.Value.Latency75thMS;
                    break;
                case nameof(LoadInfo.Latency90thMS):
                    sortingFunctor = (data) => data.Latency90thMS;
                    selectionFunctor = (data) => data.Value.Latency90thMS;
                    break;
                case nameof(LoadInfo.Latency99thMS):
                    sortingFunctor = (data) => data.Latency99thMS;
                    selectionFunctor = (data) => data.Value.Latency99thMS;
                    break;
                case nameof(LoadInfo.MaxHeapCount):
                    sortingFunctor = (data) => data.MaxHeapCount;
                    selectionFunctor = (data) => data.Value.MaxHeapCount;
                    break;
                case nameof(LoadInfo.NumberOfHeapCountSwitches):
                    sortingFunctor = (data) => data.NumberOfHeapCountSwitches;
                    selectionFunctor = (data) => data.Value.NumberOfHeapCountSwitches;
                    break;
                case nameof(LoadInfo.NumberOfHeapCountDirectionChanges):
                    sortingFunctor = (data) => data.NumberOfHeapCountDirectionChanges;
                    selectionFunctor = (data) => data.Value.NumberOfHeapCountDirectionChanges;
                    break;

                case nameof(BenchmarkSummaryData.TotalSuspensionTimeMSec):
                    sortingFunctor = (data) => data.TotalSuspensionTimeMSec;
                    selectionFunctor = (data) => data.Value.TotalSuspensionTimeMSec;
                    break;
                case nameof(BenchmarkSummaryData.PercentPauseTimeInGC):
                    sortingFunctor = (data) => data.PercentPauseTimeInGC;
                    selectionFunctor = (data) => data.Value.PercentPauseTimeInGC;
                    break;
                case nameof(BenchmarkSummaryData.PercentTimeInGC):
                    sortingFunctor = (data) => data.PercentTimeInGC;
                    selectionFunctor = (data) => data.Value.PercentTimeInGC;
                    break;
                case nameof(BenchmarkSummaryData.MeanHeapSizeBeforeMB):
                    sortingFunctor = (data) => data.MeanHeapSizeBeforeMB;
                    selectionFunctor = (data) => data.Value.MeanHeapSizeBeforeMB;
                    break;
                case nameof(BenchmarkSummaryData.MaxHeapSizeMB):
                    sortingFunctor = (data) => data.MaxHeapSizeMB;
                    selectionFunctor = (data) => data.Value.MaxHeapSizeMB;
                    break;
                case nameof(BenchmarkSummaryData.TotalAllocationsMB):
                    sortingFunctor = (data) => data.TotalAllocationsMB;
                    selectionFunctor = (data) => data.Value.TotalAllocationsMB;
                    break;
                case nameof(BenchmarkSummaryData.GCScore):
                    sortingFunctor = (data) => data.GCScore;
                    selectionFunctor = (data) => data.Value.GCScore;
                    break;

                default:
                    throw new Exception($"unexpected {s}");
            }

            List<List<LoadInfo>> sortedLoadInfo = new(); 
            foreach (var iteration in iterationData)
            {
                sortedLoadInfo.Add(iteration.Value.OrderByDescending(sortingFunctor).ToList());
            }

            List<LoadInfo> sortedAverages = new();

            foreach (var benchmark in averageData[baseline])
            {
                LoadInfo baselineInfo   = benchmark.Value;
                LoadInfo comparandInfo  = averageData[comparand][benchmark.Key];
                LoadInfo comparisonInfo = dataManager.GetComparison(baselineInfo, comparandInfo);
                sortedAverages.Add(comparisonInfo);
            }
            sortedAverages = sortedAverages.OrderByDescending(sortingFunctor).ToList();

            // Create CSV.
            StringBuilder top = new();

            // Iterate over each of the runs.
            const int singleBuildColumnSize = 11;
            int numberOfIterations = maxIteration + 1;
            string columnHeader = "Benchmark Name,WorkingSetMB,PrivateMemoryMB,RequestsPerMSec,MeanLatencyMS,Latency50thMS,Latency75thMS,Latency90thMS,Latency99thMS,# HC Switches";

            int totalCountOfBenchmarks = sortedLoadInfo.First().Count;

            string first = string.Join("", Enumerable.Range(0, numberOfIterations).Select(build => build + string.Join("", Enumerable.Repeat(",", singleBuildColumnSize))));
            string second = string.Join(",,", Enumerable.Repeat(columnHeader, numberOfIterations));

            // Add the average diff.
            first  += "Average Diff %" + string.Join("", Enumerable.Repeat(",", singleBuildColumnSize));
            second += ",," + string.Join(",,", columnHeader);

            top.AppendLine(first);
            top.AppendLine(second);

            for (int benchmarkIdx = 0; benchmarkIdx < totalCountOfBenchmarks; benchmarkIdx++)
            {
                string benchmarkData = string.Join(",,", Enumerable.Range(0, numberOfIterations).Select(iteration => DisplayDetailsForABenchmark(sortedLoadInfo[iteration][benchmarkIdx])));
                benchmarkData += $",,{DisplayDetailsForABenchmark(sortedAverages[benchmarkIdx])}";

                top.AppendLine(benchmarkData);
            }

            File.WriteAllText(Path.Combine(dataManager._basePath, $"Difference_{s}.csv"), top.ToString());

            var layout = new Layout.Layout
            {
                xaxis = new Xaxis { title = "Benchmark Name" },
                yaxis = new Yaxis { title = $"{s}" },
                width = 1500,
                title = $"Raw values of {s} for Runs"
            };

            List<Scatter> scatters = new();

            const int baseColor = 150;

            for (int iterationIdx = 0; iterationIdx <= maxIteration; iterationIdx++)
            {
                string baselineIteration;
                string comparandIteration;
            if (maxIteration == 0)
            {
                baselineIteration = baseline;
                comparandIteration = comparand;
            }
            else
            {
                baselineIteration = baseline  + "_" + iterationIdx.ToString();
                comparandIteration = comparand + "_" + iterationIdx.ToString();
            }

                Dictionary<string, LoadInfo> baselineData  = dataManager._runToBenchmarkData[baselineIteration];
                Dictionary<string, LoadInfo> comparandData = dataManager._runToBenchmarkData[comparandIteration];

                if (iterationIdx == 0)
                {
                    var sortedBaseline = baselineData.Values.OrderByDescending(sortingFunctor);
                    baselineData = sortedBaseline.ToDictionary(d => d.Benchmark);
                }

                Scatter baselineScatter = new()
                {
                    x = baselineData.Select(b => b.Key),
                    y = baselineData.Select(selectionFunctor),
                    name = $"{baselineIteration} - {s}",
                    mode = "markers",
                    marker = new Marker { color = $"rgb({baseColor + iterationIdx * 50}, 0, 0)" } 
                };

                Scatter comparandScatter = new()
                {
                    x = comparandData.Select(b => b.Key),
                    y = comparandData.Select(selectionFunctor),
                    name = $"{comparandIteration} - {s}",
                    mode = "markers",
                    marker = new Marker { color = $"rgb(0, 0, {baseColor + iterationIdx * 50})" } 
                };

                scatters.Add(baselineScatter);
                scatters.Add(comparandScatter);
            }

            Chart.Plot(scatters, layout).Display();
        }
    }
    }
    */

        /*
    public class BuildNameComparer : IEqualityComparer<BuildName>
    {
        public bool Equals(BuildName b1, BuildName b2) => b1.InData == b2.InData;
        public int GetHashCode(BuildName b) => b.InData.GetHashCode();
    }
    public record PerBuildData((DataType, string) Criteria, string Unit, BuildName BuildName, Func<BenchmarkSummaryData, double> Selector, List<BenchmarkSummaryData> Data);
    
    public Func<List<double>, double>[] summarizers = new Func<List<double>, double>[] { ComputeVolatility, ComputeMin, ComputeMax, ComputeAverage, ComputeRange, ComputeGeoMean };
    public void SaveData(DataManager dataManager, List<BuildName> builds, List<(DataType, string)> chartCriteria = null)
        => SaveData(dataManager, builds, chartCriteria?.Select(s => new List<(DataType, string)> {s}).ToList());

    public void SaveData(DataManager dataManager, List<BuildName> builds, DataType dataType, List<string> chartCriteria = null)
        => SaveData(dataManager, builds, chartCriteria?.Select(s => (dataType, s)).ToList());
    public void SaveData(DataManager dataManager, List<BuildName> builds, DataType dataType, List<List<string>> chartCriteria = null)
        => SaveData(dataManager, builds, chartCriteria?.Select(s => s.Select(s2 => (dataType, s2)).ToList()).ToList());
    public void SaveDataOne(DataManager dataManager, List<BuildName> builds, DataType dataType, List<string> chartCriteria = null)
        => SaveData(dataManager, builds, new List<List<(DataType, string)>>() { chartCriteria?.Select(s => (dataType, s)).ToList() });

    public void SaveData(DataManager dataManager, List<BuildName> builds, List<List<(DataType, string)>> chartCriteria = null)
    {
        // Build Parent -> < Run -> < Benchmark -> Data >>>
        Dictionary<BuildName, Dictionary<string, Dictionary<string, LoadInfo>>> listOfData = new(new BuildNameComparer());

        foreach (var build in builds)
        {
            if (!listOfData.TryGetValue(build, out var b))
            {
                listOfData[build] = b = new();
            }

            foreach (var run in dataManager._runToBenchmarkData)
            {
                if (run.Key.Contains(build.InData))
                {
                    b.Add(run.Key, run.Value);
                }
            }
        }

        // At this point all the data has been categorized.

        // Build Parent -> < DataType -> < Benchmark -> BenchmarkSummaryData >>
        Dictionary<BuildName, Dictionary<string, BenchmarkSummaryData>[]> buildToBenchmarkSummaryData = new(new BuildNameComparer());
            //summarizers.Select(_ => new Dictionary<string, Dictionary<string, BenchmarkSummaryData>>()).ToArray();

        // Get the Summary Data Per Build.
        foreach (var b in listOfData)
        {
            if (!buildToBenchmarkSummaryData.TryGetValue(b.Key, out var data))
            {
                buildToBenchmarkSummaryData[b.Key] = data = summarizers.Select(_ => new Dictionary<string, BenchmarkSummaryData>()).ToArray();
            }

            foreach (var br in dataManager._benchmarkToRunData)
            {
                for (DataType type = DataType.MIN_VALUE; type < DataType.COUNT; ++type)
                {
                    data[(int) type][br.Key] = new();
                }
            }

            Dictionary<string, List<LoadInfo>> benchmarkToData = new();
            foreach (var run in b.Value)
            {
                foreach (var benchmark in run.Value)
                {
                    if (!benchmarkToData.TryGetValue(benchmark.Key, out var d))
                    {
                        benchmarkToData[benchmark.Key] = d = new();
                    }

                    d.Add(benchmark.Value);
                }
            }
        }

        //string DisplayDetailsForABenchmark(BenchmarkSummaryData val) =>
        //    $"{val.Benchmark},{val.MaxWorkingSetMB},{val.MaxPrivateMemoryMB},{val.RequestsPerMSec},{val.MeanLatencyMS},{val.Latency50thMS},{val.Latency75thMS},{val.Latency90thMS},{val.Latency99thMS},{val.NumberOfHeapCountSwitches},{val.MaxHeapCount}";
        if (chartCriteria == null)
        {
            chartCriteria = new() { new() { (DataType.Volatility, nameof(LoadInfo.MaxPrivateMemoryMB)) } };
        }

        foreach (var (group, criteriaIndex) in chartCriteria.WithIndex())
        {
            Func<KeyValuePair<string, BenchmarkSummaryData>, double> sortingFunctor = null;
            List<Func<BenchmarkSummaryData, double>> selectionFunctors = new();
            List<string> units = new();

            foreach (var ((type, s), index) in group.WithIndex())
            {
            Func<KeyValuePair<string, BenchmarkSummaryData>, double> thisSortingFunctor = null;
            Func<BenchmarkSummaryData, double> selectionFunctor = null;
            string unit = null;
            switch (s)
            {
                //case nameof()
                case nameof(BenchmarkSummaryData.MaxWorkingSetMB):
                    thisSortingFunctor = (data) => data.Value.MaxWorkingSetMB;
                    selectionFunctor = (data) => data.MaxWorkingSetMB;
                    unit = "MB";
                    break;
                case nameof(BenchmarkSummaryData.MaxPrivateMemoryMB):
                    thisSortingFunctor = (data) => data.Value.MaxPrivateMemoryMB;
                    selectionFunctor = (data) => data.MaxPrivateMemoryMB;
                    unit = "MB";
                    break;
                case nameof(BenchmarkSummaryData.P99PrivateMemoryMB):
                    thisSortingFunctor = (data) => data.Value.P99PrivateMemoryMB;
                    selectionFunctor = (data) => data.P99PrivateMemoryMB;
                    unit = "MB";
                    break;
                case nameof(BenchmarkSummaryData.P95PrivateMemoryMB):
                    thisSortingFunctor = (data) => data.Value.P95PrivateMemoryMB;
                    selectionFunctor = (data) => data.P95PrivateMemoryMB;
                    unit = "MB";
                    break;
                case nameof(BenchmarkSummaryData.P90PrivateMemoryMB):
                    thisSortingFunctor = (data) => data.Value.P90PrivateMemoryMB;
                    selectionFunctor = (data) => data.P90PrivateMemoryMB;
                    unit = "MB";
                    break;
                case nameof(BenchmarkSummaryData.P75PrivateMemoryMB):
                    thisSortingFunctor = (data) => data.Value.P75PrivateMemoryMB;
                    selectionFunctor = (data) => data.P75PrivateMemoryMB;
                    unit = "MB";
                    break;
                case nameof(BenchmarkSummaryData.P50PrivateMemoryMB):
                    thisSortingFunctor = (data) => data.Value.P50PrivateMemoryMB;
                    selectionFunctor = (data) => data.P50PrivateMemoryMB;
                    unit = "MB";
                    break;
                case nameof(BenchmarkSummaryData.RequestsPerMSec):
                    thisSortingFunctor = (data) => data.Value.RequestsPerMSec;
                    selectionFunctor = (data) => data.RequestsPerMSec;
                    unit = "Req/sec";
                    break;
                case nameof(BenchmarkSummaryData.MeanLatencyMS):
                    thisSortingFunctor = (data) => data.Value.MeanLatencyMS;
                    selectionFunctor = (data) => data.MeanLatencyMS;
                    unit = "ms";
                    break;
                case nameof(BenchmarkSummaryData.Latency50thMS):
                    thisSortingFunctor = (data) => data.Value.Latency50thMS;
                    selectionFunctor = (data) => data.Latency50thMS;
                    unit = "ms";
                    break;
                case nameof(BenchmarkSummaryData.Latency75thMS):
                    thisSortingFunctor = (data) => data.Value.Latency75thMS;
                    selectionFunctor = (data) => data.Latency75thMS;
                    unit = "ms";
                    break;
                case nameof(BenchmarkSummaryData.Latency90thMS):
                    thisSortingFunctor = (data) => data.Value.Latency90thMS;
                    selectionFunctor = (data) => data.Latency90thMS;
                    unit = "ms";
                    break;
                case nameof(BenchmarkSummaryData.Latency99thMS):
                    thisSortingFunctor = (data) => data.Value.Latency99thMS;
                    selectionFunctor = (data) => data.Latency99thMS;
                    unit = "ms";
                    break;
                case nameof(BenchmarkSummaryData.MaxHeapCount):
                    thisSortingFunctor = (data) => data.Value.MaxHeapCount;
                    selectionFunctor = (data) => data.MaxHeapCount;
                    unit = "heap count";
                    break;
                case nameof(BenchmarkSummaryData.NumberOfHeapCountSwitches):
                    thisSortingFunctor = (data) => data.Value.NumberOfHeapCountSwitches;
                    selectionFunctor = (data) => data.NumberOfHeapCountSwitches;
                    unit = "hc switches";
                    break;
                case nameof(BenchmarkSummaryData.NumberOfHeapCountDirectionChanges):
                    thisSortingFunctor = (data) => data.Value.NumberOfHeapCountDirectionChanges;
                    selectionFunctor = (data) => data.NumberOfHeapCountDirectionChanges;
                    unit = "hc dir changes";
                    break;
                case nameof(BenchmarkSummaryData.TotalSuspensionTimeMSec):
                    thisSortingFunctor = (data) => data.Value.TotalSuspensionTimeMSec;
                    selectionFunctor = (data) => data.TotalSuspensionTimeMSec;
                    unit = "ms";
                    break;
                case nameof(BenchmarkSummaryData.PercentPauseTimeInGC):
                    thisSortingFunctor = (data) => data.Value.PercentPauseTimeInGC;
                    selectionFunctor = (data) => data.PercentPauseTimeInGC;
                    unit = "%";
                    break;
                case nameof(BenchmarkSummaryData.PercentTimeInGC):
                    thisSortingFunctor = (data) => data.Value.PercentTimeInGC;
                    selectionFunctor = (data) => data.PercentTimeInGC;
                    unit = "%";
                    break;
                case nameof(BenchmarkSummaryData.MeanHeapSizeBeforeMB):
                    thisSortingFunctor = (data) => data.Value.MeanHeapSizeBeforeMB;
                    selectionFunctor = (data) => data.MeanHeapSizeBeforeMB;
                    unit = "MB";
                    break;
                case nameof(BenchmarkSummaryData.MaxHeapSizeMB):
                    thisSortingFunctor = (data) => data.Value.MaxHeapSizeMB;
                    selectionFunctor = (data) => data.MaxHeapSizeMB;
                    unit = "MB";
                    break;
                case nameof(BenchmarkSummaryData.TotalAllocationsMB):
                    thisSortingFunctor = (data) => data.Value.TotalAllocationsMB;
                    selectionFunctor = (data) => data.TotalAllocationsMB;
                    unit = "MB";
                    break;
                case nameof(BenchmarkSummaryData.GCScore):
                    thisSortingFunctor = (data) => data.Value.GCScore;
                    selectionFunctor = (data) => data.GCScore;
                    unit = "score";
                    break;

                default:
                    throw new Exception($"unexpected {s}");
                    //thisSortingFunctor = (data) => data.Value.MaxPrivateMemoryMB;
                    //selectionFunctor = (data) => data.MaxPrivateMemoryMB;
                    //unit = "MB";
                    //break;
            }
            sortingFunctor = sortingFunctor ?? thisSortingFunctor; // keep first one
            selectionFunctors.Add(selectionFunctor);
            units.Add(unit);
            }

            var uniqueUnits = units.Zip(group.Select(t => t.Item1)).Select(p => p.Item2 == DataType.Volatility ? "Volatility Score" : p.Item1).Distinct();
            if (uniqueUnits.Count() > 2) throw new Exception("More than two units in chart");

            List<PerBuildData> pairedPerBuildData = new();
            List<PerBuildData> sortedPerBuildData = new();

            foreach (BuildName build in buildToBenchmarkSummaryData.Keys)
            {
                for (int groupIndex = 0; groupIndex < group.Count; ++groupIndex)
                {
                    var b = buildToBenchmarkSummaryData[build][(int) group[groupIndex].Item1];
                    var pairedData = b.Zip(buildToBenchmarkSummaryData[buildToBenchmarkSummaryData.Keys.First()][(int) group[0].Item1]).OrderByDescending(pair => sortingFunctor(pair.Second)).Select(pair => pair.First.Value);
                    //pairedPerBuildData.Add(new PerBuildData(group[groupIndex], units[groupIndex], build, selectionFunctors[groupIndex], b.OrderByDescending(sortingFunctor).Select(k => k.Value).ToList()));
                    sortedPerBuildData.Add(new PerBuildData(group[groupIndex], units[groupIndex], build, selectionFunctors[groupIndex], pairedData)); // b.OrderByDescending(sortingFunctor).Select(k => k.Value).ToList() ));
                    //sortedPerBuildData.Add(new PerBuildData(group[groupIndex], units[groupIndex], build, selectionFunctors[groupIndex], b.OrderByDescending(sortingFunctor).Select(k => k.Value).ToList()));
                }
            }


//            // Create CSV.
//            StringBuilder top = new();
//
//            // Iterate over each of the runs.
//            const int singleBuildColumnSize = 10;
//            int numberOfBuilds = buildToBenchmarkSummaryData.Count;
//            string columnHeader = "Benchmark Name,MaxWorkingSetMB,MaxPrivateMemoryMB,RequestsPerMSec,MeanLatencyMS,Latency50thMS,Latency75thMS,Latency90thMS,Latency99thMS,# HC Switches";
//
//            // Assumption: the same benchmarks are present for all runs.
//            int totalCountOfBenchmarks = buildToBenchmarkSummaryData.First().Value.Count;
//
//            string first = string.Join(",", namesOfBuilds.Select(build => build + string.Join("", Enumerable.Repeat(",", singleBuildColumnSize))));
//            string second = string.Join(",,", Enumerable.Repeat(columnHeader, numberOfBuilds));
//
//            top.AppendLine(first);
//            top.AppendLine(second);
//
//            for (int benchmarkIdx = 0; benchmarkIdx < totalCountOfBenchmarks; benchmarkIdx++)
//            {
//                top.AppendLine(string.Join(",,", namesOfBuilds.Select(buildName => DisplayDetailsForABenchmark(sortedPerBuildVolatility[buildName][benchmarkIdx]))));
//            }
//
//            File.WriteAllText(Path.Combine(dataManager._basePath, $"Volatility_{group[0]}.csv"), top.ToString());

            // Chart the sorted % Vol Results.

            ColorProvider colorProvider = new();
            //colorProvider.StartColors(builds.Select(build => build.InData));
            List<Scatter> scatters = new();
            //string mode = "markers";
            string mode = "lines+markers";
            string firstUnit = sortedPerBuildData[0].Unit;

            var layout = new Layout.Layout
            {
                xaxis = new Xaxis { title = "Benchmark Name" },
                yaxis = new Yaxis { title = firstUnit },
                width = 1200,
                title = $"GCMetrcs Sorted by {group[0].Item1} of {group[0].Item2} for {builds[0].ToDisplay} (by test)"
            };

            foreach (var (b, index) in sortedPerBuildData.WithIndex())
            {
                var scatter = new Scatter
                {
                    x = b.Data.Select(s => s.Benchmark),
                    y = b.Data.Select(v => b.Selector(v)),// + 0.1 * index),
                    mode = mode,
                    name = $"{b.BuildName.ToDisplay}: {b.Criteria.Item1.ToString()} of {b.Criteria.Item2}",
                };

                if (b.Unit != firstUnit)
                {
                    layout.yaxis2 = new Yaxis { title = b.Unit, side = "right", overlaying = "y" };
                    scatter.yaxis = "y2";
                }

                colorProvider.SetMarker(scatter, b.BuildName.InData, sortedPerBuildData.Count());
                scatters.Add(scatter);
            }

            Chart.Plot(scatters, layout).Display();

//            scatters.Clear();
//            layout = new Layout.Layout
//            {
//                xaxis = new Xaxis { title = "Benchmark Index" },
//                yaxis = new Yaxis { title = firstUnit },
//                width = 1200,
//                title = $"GCMetrcs Sorted by {group[0].Item1} of {group[0].Item2} for {builds[0].ToDisplay} (by index)"
//            };

//            //colorProvider.StartColors(builds.Select(build => build.InData));
//            foreach (var b in sortedPerBuildData)
//            {
//                var sortedData = b.Data.OrderByDescending(b.Selector);
//                var scatter = new Scatter
//                {
//                    x = Enumerable.Range(0, sortedData.Count()),
//                    y = sortedData.Select(b.Selector),
//                    mode = mode,
//                    name = $"{b.BuildName.ToDisplay}: {b.Criteria.Item1.ToString()} of {b.Criteria.Item2}",
//                    text = sortedData.Select(ss => ss.Benchmark),
//                };

//                if (b.Unit != firstUnit)
//                {
//                    layout.yaxis2 = new Yaxis { title = b.Unit, side = "right", overlaying = "y" };
//                    scatter.yaxis = "y2";
//                }

//                colorProvider.SetMarker(scatter, b.BuildName.InData, sortedPerBuildData.Count());
//                scatters.Add(scatter);
//            }
            
//            Chart.Plot(scatters, layout).Display();
        }
    }
*/

// CompareFull is used to compare different builds.

/*
public class CollectedBenchmarkData
{
    public List<double> Data = new();
    public double Sum => Data.Sum(x => x);
    public double Prod => Data.Aggregate(1.0, (prod, next) => prod * next);
    public double Average => Sum / Data.Count();
    public double GeoMean => Math.Pow(Prod, 1.0 / Data.Count());

    public void Add(double value) => Data.Add(value);
}

public class Blob // rename this...
{
    public CollectedBenchmarkData Baseline = new();
    public List<CollectedBenchmarkData> Diffs = new();
    public double Ratio(int i) => Diffs[i].GeoMean / Baseline.GeoMean;
}

void CheckAdd(string benchmark, CollectedBenchmarkData data, Func<LoadInfo, double> selector, string includeRE, string excludeRE)
{
    if ((includeRE != null) && !Regex.Match(benchmark, includeRE).Success) return;
    if ((excludeRE != null) && Regex.Match(benchmark, excludeRE).Success) return;
    if (!data.TryGetValue(benchmark, out var blob)) data[benchmark] = blob = new Blob();

}

// selector -> to extract the data to CompareFull
// includeRE -> which benchmarks to include (regex), all if null
// excludeRE -> which benchmarks to exclude (regex), none if null
// baseline/diffs -> names of builds to compare
// includeIndiv -> whether to include the individual benchmark comparisons (probably use true)
void CompareFull(DataManager dataManager, Func<LoadInfo, double> selector, string includeRE, string excludeRE, string baseline, List<string> diffs, bool includeIndiv)
{
    HashSet<string> seen = new();
    // benchmark -> Blob
    Dictionary<string, Blob> data = new();
    foreach (var (run, benchmarkData) in dataManager._runToBenchmarkData)
    {
        var build = run.Substring(0, run.LastIndexOf('_'));
        //if (!seen.Contains(build)) { Console.WriteLine(build); seen.Add(build); }
        if (build != baseline && build != diff) continue;
        foreach (var (benchmark, loadInfo) in benchmarkData)
        {
            if ((includeRE != null) && !Regex.Match(benchmark, includeRE).Success) continue;
            if ((excludeRE != null) && Regex.Match(benchmark, excludeRE).Success) continue;
            if (!data.TryGetValue(benchmark, out var blob)) data[benchmark] = blob = new Blob();
            if (build == baseline) blob.Baseline.Add(selector(loadInfo));
            else 
            blob.GetData(build == baseline).Add(selector(loadInfo));
        }
    }

    List<double> ratios = new();
    Console.WriteLine($"Baseline: {baseline}");
    foreach (var (d, i) in diffs.WithIndex())
    {
        Console.WriteLine($"Diff{i}: {d}");
    }
    {
        Console.WriteLine($"{"Benchmark",35} | {"D/B",5} | {"Base",8} | {"Diff",8}");
        Console.WriteLine($"{new string('-', 35)}-+-{new string('-', 5)}-+-{new string('-', 8)}-+-{new string('-', 8)}");
        foreach (var (benchmark, value) in data.OrderByDescending(kvp => kvp.Value.Ratio))
        {
            if (includeIndiv)
            {
                Console.WriteLine($"{benchmark,35} | {value.Ratio,5:N3} | {value.Baseline.GeoMean,8:N2} | {value.Diff.GeoMean,8:N2}");
            }
            ratios.Add(value.Ratio);
        }
    }

    Console.WriteLine($"{new string('-', 35)}-+-{new string('-', 5)}-+-{new string('-', 8)}-+-{new string('-', 8)}");
    double baseGeoMean = ComputeGeoMean(data.Select(kvp => kvp.Value.Baseline.GeoMean));
    double diffGeoMean = ComputeGeoMean(data.Select(kvp => kvp.Value.Diff.GeoMean));
    Console.WriteLine($"{"GeoMean",35} | {diffGeoMean / baseGeoMean,5:N3} | {baseGeoMean,8:N2} | {diffGeoMean,8:N2}");
    Console.WriteLine($"{"ArithMean",35} | {ComputeAverage(ratios),5:N3} | {"",8} | {"",8}");
    Console.WriteLine();
}
*/

// Display individual benchmark runs

// extract -> to extract the data to CompareFull
// benchmark -> benchmarks(s) to include
// exactMatch -> impact matching of benchmark - odd behavior.. see code

/*
void ProcessDataMean(DataManager dataManager, Func<LoadInfo, double> extract, string benchmarkName, bool exactMatch = false)
{
    Console.WriteLine("Benchmark {0}", benchmarkName);
    var names = dataManager.Data.Keys;
    // build -> (sum of GCMetrc, count)
    Dictionary<string, (double sum, int count)> GCMetrcByBuild = new(2);
    foreach (var name in names)
    {
        bool matched = (exactMatch ? name.EndsWith(benchmarkName) : name.Contains(benchmarkName));
        if (matched)
        {
            string[] fields = name.Split(new Char[] { '_' }, StringSplitOptions.RemoveEmptyEntries);
            string buildname = fields[0];
            LoadInfo info = dataManager.Data[name];
            //Console.WriteLine("build name is {0}", buildname);
            if (GCMetrcByBuild.TryGetValue(buildname, out (double sum, int count) p))
            {
                metricByBuild[buildname] = (p.sum + extract(info), p.count + 1);
            }
            else
            {
                metricByBuild.Add(buildname, (extract(info), 1));
            }

            //Console.WriteLine("metric is now {0:0.00}", metricByBuild[buildname]);

            Console.WriteLine("{0,60}: mean latency {1:0.00} ms, time in GC {2:0.00}%, heap switches {3}, max HC {4,2}, maxHeapMB {5:0.0}",
                name, info.MeanLatencyMS, info.PercentPauseTimeInGC,
                info.NumberOfHeapCountSwitches, info.MaxHeapCount, info.MaxHeapSizeMB);
        }
    }

    foreach (var (build, (sum, count)) in metricByBuild)
    {
        Console.WriteLine("build {0, 10}: {1:0.00}", build, sum / count);
    }
}

void ProcessDataMeanLatency(DataManager dataManager, string benchmarkName, bool exactMatch = false)
    => ProcessDataMean(dataManager, loadInfo => loadInfo.MeanLatencyMS, benchmarkName, exactMatch);
*/

// Shows benchmark runs that increment the metric two times in a row

// builds -> the build name (with trailing _, like "fix_")
// extract -> data to examine
// benchmarkFilterRE -> benchmarks to include
// listAll -> show all data points instead of just the changes - usually too much
/*
public void DisplayDoubleIncrement(DataManager dataManager, IEnumerable<string> builds, Func<TraceGC, double> extract, string benchmarkFilterRE, bool listAll = false)
{
    foreach ((string benchmark, var allRuns) in dataManager._benchmarkToRunData)
    {
        bool firstForBenchmark = true;
        if (!Regex.IsMatch(benchmark, benchmarkFilterRE)) continue;
        foreach ((string run, var results) in allRuns)
        {
            if (!builds.Any(b => run.StartsWith(b))) continue;
            if (results.Data == null)
            {
                Console.WriteLine($"No data for {benchmark} {run}");
                continue;
            }
            var doubleIncr =
                results.Data.GCs
                    .Where(gc => gc.GlobalHeapHistory != null)
                    .Select(extract)
                    .SlidingRange(10)
                    .SkipWhile(window => ((window[0] + 1) != window[3]) || ((window[3] + 1) != window[6])|| ((window[6] + 1) != window[9]));
            bool anyDouble = doubleIncr.Any(x => true);
            if (!anyDouble) continue;

            if (firstForBenchmark)
            {
                Console.WriteLine($"{benchmark}:");
                firstForBenchmark = false;
            }
            Console.Write($" {run,12}:");
            if (listAll)
            {
                foreach (int num in
                    results.Data.GCs
                        .Where(gc => gc.GlobalHeapHistory != null)
                        .Select(extract)
                        .SlidingWindow(-1)
                        .Where(window => window.PrevItem != window.CurrentItem)
                        .Select(window => window.CurrentItem))
                {
                    Console.Write($" {num}");
                }
            }
            Console.WriteLine();
        }
    }
}


// Shows benchmark runs that decrease a metric (ever)

// builds -> the build name (with trailing _, like "fix_")
// extract -> data to examine
// amount -> threshold of decrease to include
// benchmarkFilterRE -> benchmarks to include
// listAll -> show all data points instead of just the changes - usually too much

public void DisplayChangesDown(DataManager dataManager, IEnumerable<string> builds, Func<TraceGC, double> extract, double amount, string benchmarkFilterRE, bool listAll = false)
{
    foreach ((string benchmark, var allRuns) in dataManager._benchmarkToRunData)
    {
        bool firstForBenchmark = true;
        if (!Regex.IsMatch(benchmark, benchmarkFilterRE)) continue;
        foreach ((string run, var results) in allRuns)
        {
            if (!builds.Any(b => run.StartsWith(b))) continue;
            if (results.Data == null)
            {
                Console.WriteLine($"No data for {benchmark} {run}");
                continue;
            }
            var dec =
                results.Data.GCs
                    .Where(gc => gc.GlobalHeapHistory != null)
                    .Select(extract)
                    .SlidingWindow(-1)
                    .SkipWhile(window => window.PrevItem <= window.CurrentItem + amount);
            bool anyDecrease = dec.Any(x => true);
            if (!anyDecrease) continue;

            var incAfterDec = dec.SkipWhile(window => window.PrevItem >= window.CurrentItem - amount);
            var anyIncAfterDec = incAfterDec.Any(x => true);
            if (!anyIncAfterDec) continue;

            if (firstForBenchmark)
            {
                Console.WriteLine($"{benchmark}:");
                firstForBenchmark = false;
            }
            Console.Write($" {run,12}:");
            if (listAll)
            {
                foreach (int num in
                    results.Data.GCs
                        .Where(gc => gc.GlobalHeapHistory != null)
                        .Select(extract)
                        .SlidingWindow(-1)
                        .Where(window => window.PrevItem != window.CurrentItem)
                        .Select(window => window.CurrentItem))
                {
                    Console.Write($" {num}");
                }
            }
            Console.WriteLine();
        }
    }
}

public void DisplayHeapChangesDown(DataManager dataManager, IEnumerable<string> builds, string benchmarkFilterRE)
    => DisplayChangesDown(dataManager, builds, gc => gc.GlobalHeapHistory.NumHeaps, 0, benchmarkFilterRE, listAll: true);

// This is used to reduce a list of charts into a small enough number for the notebook to fully display.
// I have a habit of calling GetPage(0) for small lists, but this isn't needed - just display the whole list.
// (When displaying all benchmarks, having that and changing the 0 to 1, 2, etc., can be useful)
static IEnumerable<PlotlyChart> GetPage(this IEnumerable<PlotlyChart> groups, int page, int numPerPage = 18)
    => groups.Skip(numPerPage * page).Take(numPerPage);
*/


In [ ]:
// Old examples - use as resources then delete

/*

SummarizeResultsByBench(low4DM, ML("v2-fixrearranged-all", "v2-fixrearranged-all-svr"));
SummarizeResultsByBench(low4DM, ML("v2-rc3", "v2-fixrearranged"));

SummarizeResults(diffDataManager, latestPath + @"\summarize.txt");

// The specific values are busted here, but more paths can be added to an existing DataManager.
// Note: Adding/overwriting more benchmarks to an existing loaded directory is untested/etc.
// This is intended for adding a new run when you already have a baseline or previous run
// loaded and don't want to wait to read it again.

dataManager.AddData(new[] { slopePath, evalDecrPath }, scoutList.ToList());

// Again, the values are busted, but you can speed up loading if you only want to look at
// certain benchmarks.

var x = new DataManager(new[] { evalDecrPath }, filter: debugList);

var low4BaseRun = ML(new BuildName("v2-fixrearranged-mult-max_", "base"));
var low4Run = ML(new BuildName("v2-fixrearranged-mult-max-h4_", "max4"));
var svrRun = ML(new BuildName("v2-fixrearranged-mult-max-svr_", "svr"));
var svr4Run = ML(new BuildName("v2-fixrearranged-mult-max-svr4_", "svr4"));
var mult8Run = ML(new BuildName("v2-fixrearranged-mult-max-mult8_", "mult8"));
var mult32Run = ML(new BuildName("v2-fixrearranged-mult-max-mult32_", "mult32"));
var mult8max10Run = ML(new BuildName("v2-fixrearranged-mult-max-mult8x10_", "m8x10"));
var max10Run = ML(new BuildName("v2-fixrearranged-mult-max-x10_", "x10"));
var low4CompRuns = Concat(low4BaseRun, low4Run, svrRun, svr4Run, mult8Run, mult32Run, mult8max10Run, max10Run);

string compareBase = "v2-fixrearranged-mult-max-svr4";
string compareDiff = "v2-fixrearranged-mult-max-mult8";

//var extract = (LoadInfo loadInfo) => loadInfo.RequestsPerMSec;
//var extract = (LoadInfo loadInfo) => loadInfo.Latency50thMS;
var extract = (LoadInfo loadInfo) => loadInfo.MeanLatencyMS;
//var extract = (LoadInfo loadInfo) => loadInfo.MaxPrivateMemoryMB;
//var extract = (LoadInfo loadInfo) => loadInfo.P50PrivateMemoryMB;
//var extract = (LoadInfo loadInfo) => loadInfo.PercentPauseTimeInGC;

string includeRE = null; // scoutREListShort2;
string excludeRE = null; // "ConnectionClose";
CompareFull(low4DM, extract, includeRE, excludeRE, compareBase, compareDiff, true);
//CompareFull(rc3DataManager, (LoadInfo loadInfo) => Math.Max(5, loadInfo.PercentPauseTimeInGC), scoutREListShort, null, compareBase, compareDiff, true);

foreach (string benchmark in new[] { "ConnectionClose", "SingleQueryPlatform" })
{
    ProcessDataMeanLatency(rc2DataManager, benchmark, true);
}

// Not tested for a while.

DisplayDoubleIncrement(rc2DataManager, ML("fix_"), gc => gc.GlobalHeapHistory.NumHeaps, "", true)

// Not tested for a while

DisplayChangesDown(rc2DataManager, ML("fix_"), gc => gc.HeapSizeAfterMB, 3, "")

// Leftover code - manually displays heap changes

foreach (string build in ML("v2-fixrearranged-all_"))
{
    for (int i = 0; i < 3; ++i)
    {
        Console.Write($"{build}{i}:");
        foreach (int num in
            low4DM.Data[$"{build}{i} | Stage1"].Data.GCs
                .Where(gc => gc.GlobalHeapHistory != null)
                .Select(gc => gc.GlobalHeapHistory.NumHeaps)
                .SlidingWindow(-1)
                .Where(window => window.PrevItem != window.CurrentItem)
                .Select(window => window.CurrentItem))
                {
                    Console.Write($" {num}");
                }
        Console.WriteLine();
    }
}

*/

In [ ]:
// Old charting examples
/*
SaveData(low4DM, low4CompRuns, ML((DataType.Average, nameof(LoadInfo.P90PrivateMemoryMB)), (DataType.Max, nameof(LoadInfo.MaxPrivateMemoryMB)), (DataType.Max, nameof(LoadInfo.MaxHeapCount))));
SaveData(low4DM, low4CompRuns, DataType.Average, ML(nameof(LoadInfo.RequestsPerMSec)));
SaveData(dataManager, allRuns, DataType.Average, ML(nameof(LoadInfo.RequestsPerMSec), nameof(LoadInfo.Latency50thMS)));
SaveData(rc3DataManager, rc3RearrangedRun,
    ML(ML((DataType.Max, nameof(LoadInfo.MaxPrivateMemoryMB)), (DataType.Min, nameof(LoadInfo.MaxPrivateMemoryMB))),
        ML((DataType.Max, nameof(LoadInfo.P99PrivateMemoryMB)), (DataType.Min, nameof(LoadInfo.P99PrivateMemoryMB))),
        ML((DataType.Max, nameof(LoadInfo.P50PrivateMemoryMB)), (DataType.Min, nameof(LoadInfo.P50PrivateMemoryMB)))));
SaveData(rc3DataManager, rc3Runs, DataType.Average, priMemList);
SaveData(diffDataManager, vsBaseRuns, priMemList.Select(m => ML((DataType.Min, m), (DataType.Max, m))));
SaveDataOne(v2DataManager, v2Runs, DataType.Average, priMemList);
SaveDataOne(diffDataManager, allRuns, DataType.Volatility, volList);

// Using the DataManager - I haven't been using this section.

// The following cells demonstrates how to make use of the ``DataManager``. 

// The name of the run from the yaml file for which the ASP.NET run is created for.
string runName = "base_0";

Dictionary<string, LoadInfo> run = dataManager.GetAllBenchmarksForRun(runName);
dataManager.Data.Display();
List<KeyValuePair<string, LoadInfo>> runsWithGCData = dataManager.GetAllBenchmarksForRun(runName).Where(gc => gc.Value.Data != null);

string benchmarkName = "Stage2";
LoadInfo benchmarkData = dataManager.GetBenchmarkData(benchmark: benchmarkName, run: runName);
benchmarkData.Id

Dictionary<string, LoadInfo> allRunsForBenchmark = dataManager.GetAllRunsForBenchmark(benchmark: benchmarkName);
allRunsForBenchmark.Keys

dataManager.SaveBenchmarkData()

// ## Build to Build Comparison and Volatility Analysis

// I haven't been using this section, but it is an obvious one to start using again.

var run1_vs_run2 = diffDataManager.GetBenchmarkToComparison("tp3-m_0", "tp3-m_1");

static bool IsNotInvalidDouble(double val) => 
    !double.IsNaN(val) && 
    !double.IsInfinity(val) && 
    !double.IsPositiveInfinity(val) && 
    !double.IsNegativeInfinity(val);

public class SummaryTable
{
    public SummaryTable(Dictionary<string, Dictionary<string, LoadInfo>> comparisons)
    {
        Comparisons = comparisons;
    }

    private string GenerateSummaryForComparison(string comparisonKey, Dictionary<string, LoadInfo> comparison)
    {
        double averageWorkingSet = comparison.Where(a => IsNotInvalidDouble(a.Value.MaxWorkingSetMB)).Average(a => a.Value.MaxWorkingSetMB);
        double privateMemory = comparison.Where(a => IsNotInvalidDouble(a.Value.MaxPrivateMemoryMB)).Average(a => a.Value.MaxPrivateMemoryMB);
        double throughput = comparison.Where(a => IsNotInvalidDouble(a.Value.RequestsPerMSec)).Average(a => a.Value.RequestsPerMSec);
        double meanLatency = comparison.Where(a => IsNotInvalidDouble(a.Value.MeanLatencyMS)).Average(a => a.Value.MeanLatencyMS);

        double p50Latency = comparison.Where(a => IsNotInvalidDouble(a.Value.Latency50thMS)).Average(a => a.Value.Latency50thMS);
        double p75Latency = comparison.Where(a => IsNotInvalidDouble(a.Value.Latency75thMS)).Average(a => a.Value.Latency75thMS);
        double p90Latency = comparison.Where(a => IsNotInvalidDouble(a.Value.Latency90thMS)).Average(a => a.Value.Latency90thMS);
        double p99Latency = comparison.Where(a => IsNotInvalidDouble(a.Value.Latency99thMS)).Average(a => a.Value.Latency99thMS);

        return $"{comparisonKey},{averageWorkingSet},{privateMemory},{throughput},{meanLatency},{p50Latency},{p75Latency},{p90Latency},{p99Latency}";
    }

    public string GenerateSummaryForComparisons()
    {
        StringBuilder sb = new();
        sb.AppendLine("Build to Build,Average Max Working Set (MB) %, Average Max Private Memory (MB) %, Average Request/MSec %, Average Mean Latency (MSec), Average P50 Latency (MSec) %, Average P75 Latency (MSec) %, Average P90 Latency (MSec) %, Average P99 Latency (MSec) %");
        foreach (var comparison in Comparisons)
        {
            sb.AppendLine(GenerateSummaryForComparison(comparison.Key, comparison.Value));
        }

        return sb.ToString();
    }

    private int GetCountOfRegressions(List<double> selected, double thresholdPercentage, bool lessIsBetter = true)
    {
        // If throughput, less is worse => threshold <= -5%.
        var comparison = selected.Where(d => IsNotInvalidDouble(d) && ( (lessIsBetter) ? (d >= thresholdPercentage) : (d <= -thresholdPercentage)));
        return comparison.Count;
    }

    private int GetCountOfAbsRegressions(List<double> selected, double thresholdPercentage)
    {
        var comparison = selected.Where(d => IsNotInvalidDouble(d) && Math.Abs(d) >= thresholdPercentage);
        return comparison.Count;
    }

    // # of benchmarks with throughput regressed by >= 5% and 10%
    private string GenerateRegressionSummary(string comparisonKey, Dictionary<string, LoadInfo> comparison)
    {
        List<double> workingSet  = comparison.Select(c => c.Value.MaxWorkingSetMB);
        int workingSetCountGT_5  = GetCountOfRegressions(workingSet, 5);
        int workingSetCountGT_10 = GetCountOfRegressions(workingSet, 10);

        List<double> privateMemory  = comparison.Select(c => c.Value.MaxPrivateMemoryMB);
        int privateMemoryCountGT_5  = GetCountOfRegressions(privateMemory, 5);
        int privateMemoryCountGT_10 = GetCountOfRegressions(privateMemory, 10);

        List<double> throughput  = comparison.Select(a => a.Value.RequestsPerMSec);
        int throughputCountGT_5  = GetCountOfRegressions(throughput, 5, false);
        int throughputCountGT_10 = GetCountOfRegressions(throughput, 10, false);

        List<double> meanLatency  = comparison.Select(a => a.Value.MeanLatencyMS);
        int meanLatencyCountGT_5  = GetCountOfRegressions(meanLatency, 5);
        int meanLatencyCountGT_10 = GetCountOfRegressions(meanLatency, 10);

        List<double> p50Latency  = comparison.Select(a => a.Value.Latency50thMS);
        int p50LatencyCountGT_5  = GetCountOfRegressions(p50Latency, 5);
        int p50LatencyCountGT_10 = GetCountOfRegressions(p50Latency, 10);

        List<double> p75Latency  = comparison.Select(a => a.Value.Latency75thMS);
        int p75LatencyCountGT_5  = GetCountOfRegressions(p75Latency, 5);
        int p75LatencyCountGT_10 = GetCountOfRegressions(p75Latency, 10);

        List<double> p90Latency  = comparison.Select(a => a.Value.Latency90thMS);
        int p90LatencyCountGT_5  = GetCountOfRegressions(p90Latency, 5);
        int p90LatencyCountGT_10 = GetCountOfRegressions(p90Latency, 10);
        
        List<double> p99Latency  = comparison.Select(a => a.Value.Latency99thMS);
        int p99LatencyCountGT_5  = GetCountOfRegressions(p99Latency, 5);
        int p99LatencyCountGT_10 = GetCountOfRegressions(p99Latency, 10);

        return $"{comparisonKey},{workingSetCountGT_5},{workingSetCountGT_10},{privateMemoryCountGT_5},{privateMemoryCountGT_10},{throughputCountGT_5},{throughputCountGT_10},{meanLatencyCountGT_5},{meanLatencyCountGT_10},{p50LatencyCountGT_5},{p50LatencyCountGT_10},{p75LatencyCountGT_5},{p75LatencyCountGT_10},{p90LatencyCountGT_5},{p90LatencyCountGT_10},{p99LatencyCountGT_5},{p99LatencyCountGT_10}";
    }

    public string GenerateRegressionSummaryForComparisons()
    {
        StringBuilder sb = new();
        sb.AppendLine("Build to Build,Reg. Count - Working Set (MB),Large Reg. Count - Working Set (MB),Reg. Count - Max Private Memory (MB),Large Reg. Count - Max Private Memory (MB),Reg. Count - Throughput, Large Reg. Count - Throughput,Reg. Count - Mean Latency,Large Reg. Count - Mean Latency,Reg. Count - P50 Latency, Large Reg. Count - P50 Latency, Reg. Count - P75 Latency, Large Reg. Count - P75 Latency,Reg. Count - P90 Latency, Large Reg. Count - P90 Latency,Reg. Count - P99 Latency, Large Reg. Count - P99 Latency");
        foreach (var comparison in Comparisons)
        {
            sb.AppendLine(GenerateRegressionSummary(comparison.Key, comparison.Value));
        }

        return sb.ToString();
    }

    public Dictionary<string, string> GenerateRegressionAnalysisForComparison(string comparisonKey)
    {
        StringBuilder sb = new();
        Dictionary<string, string> csvData = new();
        Dictionary<string, LoadInfo> comparison = Comparisons[comparisonKey];

        string header = "Benchmark,MaxWorkingSetMB,MaxPrivateMemoryMB,RequestsPerMSec,MeanLatencyMS,Latency50thMS,Latency75thMS,Latency90thMS,Latency99thMS";

        // Generate Memory Regressions.
        StringBuilder memRegressions = new();
        memRegressions.AppendLine(header);
        foreach (var benchmark in comparison.Where(c => c.Value.MaxWorkingSetMB >= 10 || c.Value.MaxPrivateMemoryMB >= 10 ))
        {
            memRegressions.AppendLine($"{benchmark.Key},{benchmark.Value.MaxWorkingSetMB},{benchmark.Value.MaxPrivateMemoryMB},{benchmark.Value.RequestsPerMSec},{benchmark.Value.MeanLatencyMS},{benchmark.Value.Latency50thMS},{benchmark.Value.Latency75thMS},{benchmark.Value.Latency90thMS},{benchmark.Value.Latency99thMS}");
        }
        csvData["memory"] = memRegressions.ToString();

        // Generate Throughput Regressions.
        StringBuilder throughputRegressions = new();
        throughputRegressions.AppendLine(header);
        foreach (var benchmark in comparison.Where(c => c.Value.RequestsPerMSec <= -10))
        {
            throughputRegressions.AppendLine($"{benchmark.Key},{benchmark.Value.MaxWorkingSetMB},{benchmark.Value.MaxPrivateMemoryMB},{benchmark.Value.RequestsPerMSec},{benchmark.Value.MeanLatencyMS},{benchmark.Value.Latency50thMS},{benchmark.Value.Latency75thMS},{benchmark.Value.Latency90thMS},{benchmark.Value.Latency99thMS}");
        }
        csvData["throughput"] = throughputRegressions.ToString();

        // Generate Latency Regressions.
        StringBuilder latencyRegressions = new();
        latencyRegressions.AppendLine(header);
        foreach (var benchmark in comparison.Where(c => c.Value.MeanLatencyMS >= 10 || 
                                                        c.Value.Latency50thMS >= 10 || 
                                                        c.Value.Latency75thMS >= 10 || 
                                                        c.Value.Latency90thMS >= 10 || 
                                                        c.Value.Latency99thMS >= 10 ))
        {
            latencyRegressions.AppendLine($"{benchmark.Key},{benchmark.Value.MaxWorkingSetMB},{benchmark.Value.MaxPrivateMemoryMB},{benchmark.Value.RequestsPerMSec},{benchmark.Value.MeanLatencyMS},{benchmark.Value.Latency50thMS},{benchmark.Value.Latency75thMS},{benchmark.Value.Latency90thMS},{benchmark.Value.Latency99thMS}");
        }
        csvData["latency"] = latencyRegressions.ToString();

        // All.
        StringBuilder all = new();
        all.AppendLine(header);
        foreach (var benchmark in comparison)
        {
            all.AppendLine($"{benchmark.Key},{benchmark.Value.MaxWorkingSetMB},{benchmark.Value.MaxPrivateMemoryMB},{benchmark.Value.RequestsPerMSec},{benchmark.Value.MeanLatencyMS},{benchmark.Value.Latency50thMS},{benchmark.Value.Latency75thMS},{benchmark.Value.Latency90thMS},{benchmark.Value.Latency99thMS}");
        }
        csvData["all"] = all.ToString();

        return csvData;
    }

    public void SaveComparisons(string basePath)
    {
        // Add Summary for Comparisons.
        string summaryOfComparisons = GenerateSummaryForComparisons();
        File.WriteAllText(Path.Combine(basePath, "SummaryOfComparisons.csv"), summaryOfComparisons);

        // Add Regression Summary for Comparisons.
        string regressionSummary = GenerateRegressionSummaryForComparisons();
        File.WriteAllText(Path.Combine(basePath, "RegressionSummary.csv"), regressionSummary);

        // Add Large Regression Analysis for Comparison.
        string perComparisonDataPath = Path.Combine(basePath, "PerComparisonData");
        if (!Directory.Exists(perComparisonDataPath))
        {
            Directory.CreateDirectory(perComparisonDataPath);
        }

        foreach (var comparison in Comparisons)
        {
            string comparisonPath = Path.Combine(perComparisonDataPath, comparison.Key);
            Directory.CreateDirectory(comparisonPath);

            Dictionary<string, string> regressionComparisons = GenerateRegressionAnalysisForComparison(comparison.Key);

            // Memory
            File.WriteAllText(Path.Combine(comparisonPath, "MemoryRegressions.csv"), regressionComparisons["memory"]);

            // Throughput
            File.WriteAllText(Path.Combine(comparisonPath, "ThroughputRegressions.csv"), regressionComparisons["throughput"]);

            // Latency
            File.WriteAllText(Path.Combine(comparisonPath, "LatencyRegressions.csv"), regressionComparisons["latency"]);

            // All
            File.WriteAllText(Path.Combine(comparisonPath, "All.csv"), regressionComparisons["all"]);
        }
    }

    public Dictionary<string, Dictionary<string, LoadInfo>> Comparisons { get; }
}


//var datas3_vs_datas_4 = baseDataManager.GetBenchmarkToComparison("base_0", "base_1");

Dictionary<string, Dictionary<string, LoadInfo>> comparisons = new()
{
    { nameof(run1_vs_run2), run1_vs_run2 },
};

SummaryTable summaryTable = new(comparisons);
summaryTable.SaveComparisons(diffPath);


// I don't use this anymore (or GCCharting at all)

void ChartProperty(LoadInfo baseline, LoadInfo comparand, string nameOfProperty)
{
    GCProcessData baselineGC = baseline.Data;
    GCProcessData comparandGC = comparand.Data;

    List<(string scatterName, List<TraceGC> gcs)> gcData = 
        new()
        {
            { ( scatterName :  $"{nameOfProperty} for {baseline.Id}" , gcs : baselineGC.GCs )},
            { ( scatterName :  $"{nameOfProperty} for {comparand.Id}" , gcs : comparandGC.GCs )}
        };

    GCCharting.ChartGCData(gcData          : gcData, 
                           title           : $"{nameOfProperty} Comparison Between {baseline.Run} and {comparand.Run}", 
                           isXAxisRelative : false,
                           fieldName       : nameOfProperty).Display();

}

void ChartProperty(LoadInfo comparison, string nameOfProperty)
{
    GCProcessData baselineGC = comparison.Data;
    GCProcessData comparandGC = comparison.Data2;

    List<(string scatterName, List<TraceGC> gcs)> gcData = 
        new()
        {
            { ( scatterName :  $"{nameOfProperty} for Baseline" , gcs : baselineGC.GCs )},
            { ( scatterName :  $"{nameOfProperty} for Comparand" , gcs : comparandGC.GCs )}
        };

    GCCharting.ChartGCData(gcData          : gcData, 
                           title           : $"{nameOfProperty} Comparison", 
                           isXAxisRelative : false,
                           fieldName       : nameOfProperty).Display();

}

void ChartProperty(IEnumerable<LoadInfo> info, string nameOfProperty)
{
    List<(string scatterName, List<TraceGC> gcs)> gcData =
        info.Select(li => (scatterName: $"{nameOfProperty}", gcs: li.Data.GCs)).ToList();
    GCCharting.ChartGCData(gcData: gcData, title: "${nameOfProperty} Comparison", isXAxisRelative: false, fieldName: nameOfProperty).Display();
}


var run1_Benchmark = diffDataManager.GetBenchmarkData(benchmark: "CachingPlatform", "tp3-m_0");
var run2_Benchmark = diffDataManager.GetBenchmarkData(benchmark: "CachingPlatform", "tp3-m_1");

// Chart the PauseDurationMSec for the run1 vs. run2.
ChartProperty(baseline: run1_Benchmark, comparand: run2_Benchmark, nameof(TraceGC.HeapCount))


// Leftover code that bucketed ranges of values for metrics and displayed them in columns
// - probably out-of-date (and very hardwired to the data I was looking at) - probably ignore this

int[] ranges = {1,11,12,13};

for (int i = 0; i < 4; ++i)
{
    string trace = "fixed-newlinear-nosmooth_" + i;
    Console.Write($"{trace}: ");
    //var cpData = diffDataManager.GetBenchmarkData("MultipleQueriesPlatform", trace);
    var cpData = noDataManager.GetBenchmarkData("Fortunes", trace);

    int prevNumHeaps = -1;
    int count = 0;
    int nextRangeIndex = 0;
    foreach (int numHeaps in cpData.Data.GCs.Select(gc => gc.GlobalHeapHistory?.NumHeaps).Where(x => x.HasValue).Append(-1))
    {
        if (numHeaps == prevNumHeaps)
        {
            count++;
            continue;
        }

        if (count != 0)
        {
            int skip = ranges.Skip(nextRangeIndex).TakeWhile(r => prevNumHeaps > r).Count();
            Console.Write(new string(' ', skip * 13));
            nextRangeIndex += skip + 1;
            Console.Write($"{count,5} @ {prevNumHeaps,2} {(numHeaps == -1 ? ' ' : (numHeaps > prevNumHeaps ? '^' : 'v'))} ");
        }
        prevNumHeaps = numHeaps;
        count = 1;
    }
    Console.WriteLine();
}
*/

## Debugging

In [ ]:
System.Diagnostics.Process.GetCurrentProcess().Id

In [ ]:
#!about